<a href="https://colab.research.google.com/github/lamaimam/Hakawati/blob/main/Evaluation_for_Baseline_and_Fine_tuned_LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Our Graduation Project: **"Automatic Arabic Story Generation - Hakawaty"**
# this notebook shows the evaluation for both baseline llama3.1 and our fine-tuned model llama3.1 with evalution metrics(
        "Semantic Coherence": semantic_coherence,
        "Perplexity": perplexity,
        "Distinct 1": distinct_1,
        "Distinct 2": distinct_2
)

# important **installations**

In [ ]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


**install unsloth**

unsloth is a library that makes  fine-tuning LLMs  x2 faster, use 70% less memory, and with no degradation in accuracy!

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# **Dataset after merging**

In [ ]:
import pandas as pd

file1 = pd.read_excel("/content/MSA_data.xlsx")
file2 = pd.read_excel("/content/125_generated_stories.xlsx")


merged_data = pd.concat([file1, file2], ignore_index=True)

# this will save new excel for the new data'merged'
merged_data.to_excel("merged_file.xlsx", index=False)

print("Files have been merged successfully!")


file_path = "merged_file.xlsx"
data = pd.read_excel(file_path)

# Select only the required columns
data = data[['Prompt', 'Story']]


Files have been merged successfully!


In [ ]:

import pandas as pd

# Load the Excel file
file_path = "/content/merged_file.xlsx"  # Replace with your Excel file path
data = pd.read_excel(file_path)

# Select only the required columns
data = data[['Prompt', 'Story']]

 prepare dataset in a **converasation format** that is compatible with HuggingFace and Llama (LLaMA expects a conversation-based input format).


In [ ]:
from datasets import Dataset

# Prepare data in HuggingFace format ('role','contant')
formatted_data = [
    {
        "conversations": [
            {"role": "user", "content": row['Prompt']},
            {"role": "assistant", "content": row['Story']}
        ]
    }
    for _, row in data.iterrows()
]

# Convert to HuggingFace Dataset
full_dataset = Dataset.from_list(formatted_data)


**split the formatted data** into training, validation, and test sets.
**Split dataset into:**
*   Train: 80%
*   Validation: 10%
*   Test: 10%

 **convert to hugging face objects** which ensures compatibility with tools provided by HuggingFace, such as tokenizers and trainers.


 **Setting the chat template for Llama** (it expect a conversation), where it tells the tokenizer how to encode the data so the model interprets it properly,**then apply it to the data splits**.



> Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```



In [ ]:
from unsloth.chat_templates import get_chat_template
from sklearn.model_selection import train_test_split
# split data into train, val, and test
train_data, temp_data = train_test_split(
    formatted_data, test_size=0.2, random_state=42
)

val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42
)
dataset = Dataset.from_list(train_data) #train datasett
dataset1 = Dataset.from_list(val_data)  #valdition dataset
dataset2 = Dataset.from_list(test_data) #test dataset

# Convert to HuggingFace Datasets objects
dataset = Dataset.from_list(train_data)
dataset2 = Dataset.from_list(test_data)


# Set the chat template to LLaMA-3.1
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1"
)

# Apply the template to format the dataset
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}
# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)  # Train
dataset1 = dataset1.map(formatting_prompts_func, batched=True)  # Validation
dataset2 = dataset2.map(formatting_prompts_func, batched=True)  # Test

# print to confirm sizes:   80% of 1120 , 10% of 1120 , 10% of 1120 -> should be 896, 112, 112
print("Train dataset size:", len(dataset))
print("Validation dataset size:", len(dataset1))
print("Test dataset size:", len(dataset2))

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Train dataset size: 896
Validation dataset size: 112
Test dataset size: 112


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

# loding llama3.1 instruction based 8 billion parameter

In [ ]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 1024# our dataset pompt-story has a maximum sequence length of 995 (closest to 1024) , that is why we choose 1024
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True# Use 4bit quantization to reduce memory usage=.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",

    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",# the model that we choose
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

# Test and evualte the baseline model

In [ ]:
# Enable fast inference
FastLanguageModel.for_inference(pretrained_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference (from unsloth)


prompt = """أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 5-8.
أحداث القصة يجب أن تدور في منزل ضخم.
نهاية القصة يجب أن تكون سعيدة.
يجب أن يكون هناك حوار.
عدد الشخصيات في القصة يجب أن يكون 2.
بلد الأحداث هي المملكة العربية السعودية.
الأحداث تدور في فصل الصيف.
يجب أن تحتوي القصة على درس مفيد.
أكتب القصة مباشرة باللغة العربية."""

# Prepare the input message to he Llama format given the prompt
messages = [
    {"role": "user", "content": prompt},
]

# Apply the Llama chat template and tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True, # Adds something like [Assistant: ] at the end, to let or help the model to start its response.
    return_tensors="pt", # pytorch tensor (so the tokenizer ensures the inputs are directly compatible with PyTorch-based models)
).to("cuda")


# attention mask (to ignore padding tokens)
attention_mask = (inputs != tokenizer.pad_token_id).long()

outputs = model.generate(input_ids = inputs,attention_mask=attention_mask, max_new_tokens = 1024, use_cache = True,
                         temperature = 0.5, min_p = 0.3)
tokenizer.batch_decode(outputs,skip_special_tokens=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nأكتب قصة قصيرة تكون مفهومة للأطفال في عمر 5-8.\nأحداث القصة يجب أن تدور في منزل ضخم.\nنهاية القصة يجب أن تكون سعيدة.\nيجب أن يكون هناك حوار.\nعدد الشخصيات في القصة يجب أن يكون 2.\nبلد الأحداث هي المملكة العربية السعودية.\nالأحداث تدور في فصل الصيف.\nيجب أن تحتوي القصة على درس مفيد.\nأكتب القصة مباشرة باللغة العربية.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nفي منزل ضخم وبسيط في أحد أنحاء المملكة العربية السعودية، حيث تحلق الأطياف الملونة بين غرفه والمناظر الخلابة، كان يعيش الصديقين خالد وفراس. كان الصيف قد حل بحبه الدفء والأشعة الزاهية التي تغلف المدينة بألوانها العجيبة.\n\nذات يوم، أثناء الاستمتاع ب لعبة العاب خارجية، اقترحت فراس على خالد أن يبحثوا معاً عن حديقة صغيرة يمكن أن يجمعاها لنفسي، ويدرسا عن النباتات في المملكة.\n\n"هذا ما نحتاج إليه حقا!" صاحت فراس بحماس، "حديقة صغيرة ملهمة، ل

In [ ]:
import re
import pandas as pd
from transformers import TextStreamer, GPT2LMHeadModel, GPT2Tokenizer,AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# Initialize Sentence-BERT model for semantic relatedness (Coherence Metrics) -> resource: https://huggingface.co/sentence-transformers/paraphrase-MiniLM-L6-v2
semantic_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Load multilingual BART model (works with Arabic too) -> resource: https://huggingface.co/docs/transformers/en/model_doc/mbart,
mbart_tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
mbart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

# Evaluation metrics

In [ ]:
from transformers import TextStreamer


# Function to clean the generated story to select the stoty after assistant word -> resources that helped "https://stackoverflow.com/questions/53972642/how-to-search-for-regex-pattern-across-multiple-lines-of-text-with-re-dotall"
def clean_generated_story(generated_story):
    # Extract the text after 'assistant'
    story_match = re.search(r"assistant\n\n(.+)", generated_story, re.DOTALL)
    return story_match.group(1).strip() if story_match else ""


def generate_story(prompt):
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    attention_mask = (inputs != tokenizer.pad_token_id).long()

    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    output_ids = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, temperature=0.5, min_p=0.3, attention_mask= attention_mask)
    return output_ids


# Calculate Semantic Coherence (Cosine Similarity)
def calculate_semantic_relatedness(story):
    sentences = story.split('.') #splits the story into sentences
    embeddings = semantic_model.encode(sentences)
    similarity_scores = cosine_similarity(embeddings) # calc the cosine semilraty to these sentence
    return similarity_scores.mean()  # Average cosine similarity between all sentence pairs


# Calculate Perplexity using mBART -> resource that helped:  https://stackoverflow.com/questions/77433100/how-to-get-perplexity-per-token-rather-than-average-perplexity
def calculate_perplexity(text):
    encodings = mbart_tokenizer.encode(text, return_tensors='pt') #convert the text into numerical input
    with torch.no_grad(): #this used to reduce meamory usage
        output = mbart_model(encodings, labels=encodings) #this will [produce model output and losses]
    log_likelihood = output.loss * encodings.size(1)
    perplexity = torch.exp(log_likelihood / encodings.size(1)) #how well the model predicts the next word in a sequence
    return perplexity.item()

# Calculate Diversity (distinct n-gram scores) -> resource that helped: "https://github.com/neural-dialogue-metrics/Distinct-N/blob/main/distinct_n/metrics.py"
def calculate_diversity(generated_story):
    words = generated_story.split() #split the string into list using white space
    distinct_1 = len(set(words)) / len(words)  # Distinct 1-gram score we use set to remove the duplicate
    ngrams_2 = [tuple(words[i:i+2]) for i in range(len(words)-1)]
    distinct_2 = len(set(ngrams_2)) / len(ngrams_2)  # Distinct 2-gram score
    return distinct_1, distinct_2

In [ ]:

for example in dataset2:
    prompt = example['conversations'][0]['content']  # Get the prompt
    reference_story = example['conversations'][1]['content']  # Get the reference story
    print(f"Prompt:\n{prompt}\n")
    print(f"Reference Story:\n{reference_story}\n")

Prompt:
أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 8-10. أحداث القصة يجب أن تدور في المحيط. نهاية القصة يجب أن تكون حزينة. يجب أن يكون هناك حوار. عدد الشخصيات في القصة يجب أن يكون 1. موضوع القصة يجب أن يكون الأساطير والخرافات. بلد الأحداث هي  لبنان. النشاط الرسم. أضف تحولا غير متوقع في الأحداث: الكشف عن أن الحدث الرئيسي كان خدعة. أكتب القصة مباشرة.

Reference Story:
في قرية صغيرة على سواحل لبنان، كان يعيش طفل يُدعى عمر. عمر كان يحب الرسم كثيرًا، وكان يقضي ساعات طويلة وهو يرسم البحر والمحيط بألوانه الزاهية وأمواجه العاتية. رغم صغر سنه، كان لعمر خيال واسع يأخذه إلى عوالم بعيدة في رحلات مثيرة عبر رسوماته الجميلة.

ذات يوم، قرر عمر رسم أسطورة لبنانية قديمة سمعها من جده، عن مخلوق بحري غامض يعيش في أعماق المحيط قرب السواحل اللبنانية. قيل إن هذا المخلوق يظهر مرة كل مئة سنة، وأن من يتمكن من رؤيته سيُمنح حكمة ومعرفة لا تقدر بثمن.

عكف عمر على رسم ذلك المخلوق بكل تفاصيله التي تخيلها، مستخدمًا أروع الألوان وأدق الخطوط. بعد أيام من العمل المتواصل، أتم عمر رسمته وجلس يتأملها، متخيلًا لقاءه بذلك المخ

In [ ]:
import pandas as pd

# Prepare to collect the results for all test examples
test_results = []
count = 0


for example in dataset2:
    prompt = example['conversations'][0]['content']  # Get the prompt for all dataset
    reference_story = example['conversations'][1]['content']  # Get the reference story for all dataset


    generated_story_ids = generate_story(prompt)
    generated_story = tokenizer.decode(generated_story_ids[0], skip_special_tokens=True)
    count += 1 #increamnt the counter

    # Clean the generated story using clean_generated_story function
    cleaned_story = clean_generated_story(generated_story)
    print(f"Cleaned Story {count}:\n{cleaned_story}\n")

    # Store the initial results (without metrics) to excel this will reduce the memory usge
    test_results.append({
        "Prompt": prompt,
        "Reference Story": reference_story,
        "Generated Story": cleaned_story
    })

# Save the intermediate results to an Excel file before calculating metrics
results_df = pd.DataFrame(test_results)
results_df.to_excel("intermediate_test_results.xlsx", index=False)
print("Intermediate results saved to 'intermediate_test_results.xlsx'.")

Cleaned Story 1:
كانت هناك أسطورة قديمة في لبنان، تعود إلى زمن الأساطير، عن سمكة كبيرة اسمها "النمرة". كانت النمرة سمكة كبيرة، ذات لحية طويلة، تعيش في المحيط اللبناني. كانت معروفة بذكائها وذكاءها.

كانت النمرة تعيش في كهف تحت الماء، مع أصدقائها من الأسماك. كانت تحب الرسم، وتستخدم ذيلها كأداة للرسم. كانت تحب رسم الأسماك الأخرى، والشواطئ، والجبال.

أحد يومين، قالت النمرة لأصدقائها: "أنا سأرسم صورة كبيرة من المحيط اللبناني، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة من الجبال، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة من الشواطئ، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة من الأسماك الأخرى، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة مني، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة من المحيط اللبناني، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأرسم صورة كبيرة من الجبال، وسأضعها في وسط المحيط، حتى تروا جمالها".

وأضافت: "وأنا سأر

In [ ]:
import pandas as pd
import re

with open("/content/cleanedStories from llama3.1 baseline.txt", "r", encoding="utf-8") as file:
    cleaned_stories_text = file.read()

# here to extract only the story contetnt
pattern = r"(Cleaned Story \d+:)"
split_cleaned_stories = re.split(pattern, cleaned_stories_text)
cleaned_stories = [
    split_cleaned_stories[i + 1].strip()
    for i in range(1, len(split_cleaned_stories) - 1, 2)
]

# also extract from the test dataset
reference_stories = [item["content"] for convo in dataset2 for item in convo["conversations"] if item["role"] == "assistant"]

# Verify the number of stories matches
if len(cleaned_stories) != len(reference_stories):
    raise ValueError("The number of cleaned stories and reference stories do not match.")

# Create a DataFrame contaning the refernce and generated
data = pd.DataFrame({
    "Reference Story": reference_stories,
    "Generated Story": cleaned_stories
})


output_path = "generated_with_refernec_llama3baseline_stories.xlsx"
data.to_excel(output_path, index=False)

print(f"Merged stories saved to {output_path}")


Merged stories saved to generated_with_refernec_llama3baseline_stories.xlsx


# the result of **Baseline** model

In [ ]:
import pandas as pd


intermediate_results_df = pd.read_excel("/content/generated_with_refernec_llama3baseline_stories.xlsx")

#list for the result with all metrics
test_results_with_metrics = []

# Calculate metrics for each record'row' in the DataFrame
count = 0
for index, row in intermediate_results_df.iterrows():

    cleaned_story = row["Generated Story"]
    print(f"Metrices for Story {count}:\n")
    count += 1
    # Calculate metrics
    semantic_coherence = calculate_semantic_relatedness(cleaned_story)
    print(f"Semantic Coherence: {semantic_coherence}")
    perplexity = calculate_perplexity(cleaned_story)
    print(f"Perplexity: {perplexity}")
    distinct_1, distinct_2 = calculate_diversity(cleaned_story)
    print(f"Distinct 1-gram Score: {distinct_1}")
    print(f"Distinct 2-gram Score: {distinct_2}")

    # Add the metrics to the results
    test_results_with_metrics.append({
        "Semantic Coherence": semantic_coherence,
        "Perplexity": perplexity,
        "Distinct 1": distinct_1,
        "Distinct 2": distinct_2
    })

# Convert the updated results to a DataFrame
results_with_metrics_df = pd.DataFrame(test_results_with_metrics)

# Save the final results to an Excel file
results_with_metrics_df.to_excel("final_test_results_with_metrics.xlsx", index=False)
print("Final results with metrics saved to 'final_test_results_with_metrics.xlsx'.")


Metrices for Story 0:

Semantic Coherence: 0.9573985934257507


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpqvhx2myd.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


Grammar Errors: 25
Perplexity: 2.372913360595703
Distinct 1-gram Score: 0.17158176943699732
Distinct 2-gram Score: 0.22043010752688172
Metrices for Story 1:

Semantic Coherence: 0.9138842821121216
Grammar Errors: 7
Perplexity: 1.9387092590332031
Distinct 1-gram Score: 0.26681614349775784
Distinct 2-gram Score: 0.4247191011235955
Metrices for Story 2:

Semantic Coherence: 0.8973278403282166
Grammar Errors: 4
Perplexity: 1.834242343902588
Distinct 1-gram Score: 0.5189393939393939
Distinct 2-gram Score: 0.7832699619771863
Metrices for Story 3:

Semantic Coherence: 0.9195353984832764
Grammar Errors: 4
Perplexity: 1.8456629514694214
Distinct 1-gram Score: 0.19123505976095617
Distinct 2-gram Score: 0.2874251497005988
Metrices for Story 4:

Semantic Coherence: 0.8790943622589111
Grammar Errors: 11
Perplexity: 1.617571234703064
Distinct 1-gram Score: 0.47085201793721976
Distinct 2-gram Score: 0.6891891891891891
Metrices for Story 5:

Semantic Coherence: 0.8851078152656555
Grammar Errors: 17
Pe

# Evaluation for the **fine-tuned** model

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Lamiia/hakawaty", # here load our fine-tuned model from hugging face
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,

    )
    FastLanguageModel.for_inference(model)

prompt = """أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 5-8.
أحداث القصة يجب أن تدور في منزل ضخم.
نهاية القصة يجب أن تكون سعيدة.
يجب أن يكون هناك حوار.
عدد الشخصيات في القصة يجب أن يكون 2.
بلد الأحداث هي المملكة العربية السعودية.
الأحداث تدور في فصل الصيف.
يجب أن تحتوي القصة على درس مفيد.
أكتب القصة مباشرة باللغة العربية."""


messages = [
    {"role": "user", "content": prompt},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 0.5, min_p = 0.3)

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


في قلب المملكة العربية السعودية، كان هناك منزل ضخم يطل على واحة خضراء وجميلة. كان هذا المنزل ملاذاً لأب وابنة يُدعيان يوسف وعلياء. كان يوسف واليوسف عاطفان جداً ويشتركان في كل شيء، من اللعب إلى القراءة، وكانا يحبان الصيف كثيراً.

ذات يوم، بينما كان يوسف وعلياء يلعبون في الحديقة، قال يوسف بحماس: "يا علياء، هل تظن أن هناك مكاناً أجمل من هذا المنزل؟"

أجابت علياء بفضول: "لا، يا يوسف، هذا المنزل أجمل من أي مكان آخر. لكن، هل يمكننا أن نجد طريقة لجعل هذا المكان أجمل؟"

"بالطبع!"، قال يوسف بابتسامة. "لدي فكرة. دعونا نزرع بعض النباتات الجميلة هنا في الحديقة."

"ماذا نوع النباتات التي يجب أن نزرع؟" سألت علياء بحماس.

"أعتقد أن الزهور الأبيض والسماء الزرقاء سيكونان جميلان جدًا هنا!"، قال يوسف.

بدأ يوسف وعلياء بزراعة النباتات في الحديقة. كانوا يعملان معاً بجد، يزرعان كل نبتة بحب واهتمام. وبعد مرور بعض الأيام، بدأت الزهور تفتح، وأصبحت الحديقة أجمل مما كانت عليه من قبل.

"انظري يا علياء! لقد فعلناها!"، قال يوسف متحمساً.

"نعم، لقد فعلنا! وأنا فخورة بك يا يوسف!"، أجابت علياء.

"أنا فخور بك أيضًا يا 

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


# the results of our **fine-tuned** model

In [ ]:
# list for  the results for all test examples
test_results = []
count= 0
for example in dataset2:
    prompt = example['conversations'][0]['content']
    reference_story = example['conversations'][1]['content']


    generated_story_ids = generate_story(prompt)
    generated_story = tokenizer.decode(generated_story_ids[0], skip_special_tokens=True)
    count= count+1
    # Clean the generated story using clean_generated_story fucation
    cleaned_story = clean_generated_story(generated_story)
    print(f"Cleaned Story{count}:\n{cleaned_story}\n")


    # Calculate metrics
    semantic_coherence = calculate_semantic_relatedness(cleaned_story)
    perplexity = calculate_perplexity(cleaned_story)
    distinct_1, distinct_2 = calculate_diversity(cleaned_story)
    print(f"Semantic Coherence: {semantic_coherence}")
    print(f"Distinct 1-gram Score: {distinct_1}")
    print(f"Distinct 2-gram Score: {distinct_2}")
    print(f"Perplexity: {perplexity}\n")

    # Store the results
    test_results.append({
        "prompt": prompt,
        "reference_story": reference_story,
        "generated_story": cleaned_story,
        "semantic_coherence": semantic_coherence,
        "perplexity": perplexity,
        "distinct_1": distinct_1,
        "distinct_2": distinct_2
    })

# Convert the results to a DataFrame to convert it as excel
results_df = pd.DataFrame(test_results)


print(results_df)

#the results saved as a CSV file
results_df.to_csv("test_results.csv", index=False)


Cleaned Story1:
في قرية صغيرة على ساحل لبنان الجميل، كان يعيش صبي صغير يُدعى ياسين. ياسين كان يحب الرسم كثيرًا وكان دائمًا يحلم بأن يصبح رسامًا مشهورًا. لكن ما كان يخفي وراء حبه للرسم كان شغفه الأكبر وهو البحث عن الأساطير والخرافات التي تحكى عن المحيط الذي يحيط بقرية ياسين.

ذات يوم، بينما كان ياسين يرسم على الشاطئ، سمع صوتًا غريبًا يأتي من البحر. كان الصوت يبدو وكأنه يأخذ شكلًا من أشكال الأساطير التي كان ياسين يحلم بها. بقلب ممتلئ بالفضول، قرر ياسين أن يتبع الصوت.

"من هناك؟" صرخ ياسين، وهو يتجه نحو البحر.

"أنا... أنا نينو، بطل الأساطير الذي يحمي المحيط من الأخطار." أجاب الصوت.

"نينو؟ لقد كنت دائمًا يحلم بأن ألتقي بك!" قال ياسين بفضول.

"أنا هنا ليعلمك درسًا مهمًا، يا ياسين. المحيط ليس مجرد مكان للبحث عن الأساطير، بل هو عالم يجب أن نحافظ عليه." قال نينو.

بينما كان ياسين يستمع إلى نينو، بدأ البحر يتحول إلى لون ذهبي غامض. فجأة، اختفى نينو، تاركًا ياسين وحيدًا على الشاطئ.

"لماذا؟ لماذا تركك؟" سأل ياسين وهو يبكي.

"لقد كنت خدعة، يا ياسين. المحيط لا يمكن أن يظهر لك بطل الأساطير. لكن ال

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpv8tfaysy.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


BLEURT Score: 0.2856111526489258
Semantic Coherence: 0.780329704284668
Grammar Errors: 12
Distinct 1-gram Score: 0.6766917293233082
Distinct 2-gram Score: 0.9433962264150944
Perplexity: 1.9509146213531494

Cleaned Story2:
في قرية صغيرة بالصومال، كان يعيش طفلان صغيران، يوسف وأمينة. كان الشتاء قد حلّ بألوانه الزاهية والأمطار الغزيرة، مما جعل الأرض خضراء وجميلة.

ذات يوم، بينما كان يوسف وأمينة يلعبان خارج البيت، قال يوسف بحماس: "أمينة، هل تظن أن هناك كنزًا مخبأً في هذه القرية؟"

ابتسمت أمينة وقالت: "لا أعرف، لكن السحر في هذا الشتاء يجعل كل شيء ممكنًا!"

فكر يوسف وأمينة في كيفية البحث عن الكنز. قررا أن يبدأوا البحث في الحديقة التي تحيط بالبيت. كان هناك شجرة كبيرة، فقررا أن يبدأوا من هناك.

بينما كانا يبحثان، قال يوسف: "يبدو أننا بحاجة إلى بعض المساعدة. هل يمكننا أن نطلب من أحد سكان القرية المساعدة؟"

أجابت أمينة: "نعم، يمكننا أن نطلب من جدك المساعدة. هو يعرف كل شيء عن هذه القرية."

قاما يوسف وأمينة بزيارة جدته، الذي كان يعيش في بيت قريب. عندما سألتهما عن المساعدة، ابتسم جدته وقالت: "بالطبع

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-4d0667aaffb7>", line 10, in <cell line: 5>
    generated_story_ids = generate_story(prompt)
  File "<ipython-input-11-2bb29b3d77c1>", line 42, in generate_story
    output_ids = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, temperature=0.5, min_p=0.3, attention_mask= attention_mask)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/unsloth/models/llama.py", line 1494, in _fast_generate
    output = generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1838, in generate
    outputs = self.base_model.generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-pa

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-4d0667aaffb7>", line 10, in <cell line: 5>
    generated_story_ids = generate_story(prompt)
  File "<ipython-input-11-2bb29b3d77c1>", line 42, in generate_story
    output_ids = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, temperature=0.5, min_p=0.3, attention_mask= attention_mask)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/unsloth/models/llama.py", line 1494, in _fast_generate
    output = generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/peft/peft_model.py", line 1838, in generate
    outputs = self.base_model.generate(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-pa

TypeError: object of type 'NoneType' has no len()

# Result
The average values for the provided metrics are:

BLEURT Score: **0.2120**


Semantic Coherence: **0.8562**


Grammar Errors: **14.7471**


Distinct 1-gram Score: **0.6626**


Distinct 2-gram Score: **0.9102**


Perplexity: 1.8795 ​


Cleaned Story1:
في قرية صغيرة على ساحل لبنان الجميل، كان يعيش صبي صغير يُدعى ياسين. ياسين كان يحب الرسم كثيرًا وكان دائمًا يحلم بأن يصبح رسامًا مشهورًا. لكن ما كان يخفي وراء حبه للرسم كان شغفه الأكبر وهو البحث عن الأساطير والخرافات التي تحكى عن المحيط الذي يحيط بقرية ياسين.

ذات يوم، بينما كان ياسين يرسم على الشاطئ، سمع صوتًا غريبًا يأتي من البحر. كان الصوت يبدو وكأنه يأخذ شكلًا من أشكال الأساطير التي كان ياسين يحلم بها. بقلب ممتلئ بالفضول، قرر ياسين أن يتبع الصوت.

"من هناك؟" صرخ ياسين، وهو يتجه نحو البحر.

"أنا... أنا نينو، بطل الأساطير الذي يحمي المحيط من الأخطار." أجاب الصوت.

"نينو؟ لقد كنت دائمًا يحلم بأن ألتقي بك!" قال ياسين بفضول.

"أنا هنا ليعلمك درسًا مهمًا، يا ياسين. المحيط ليس مجرد مكان للبحث عن الأساطير، بل هو عالم يجب أن نحافظ عليه." قال نينو.

بينما كان ياسين يستمع إلى نينو، بدأ البحر يتحول إلى لون ذهبي غامض. فجأة، اختفى نينو، تاركًا ياسين وحيدًا على الشاطئ.

"لماذا؟ لماذا تركك؟" سأل ياسين وهو يبكي.

"لقد كنت خدعة، يا ياسين. المحيط لا يمكن أن يظهر لك بطل الأساطير. لكن الدرس الذي علمتك هو أننا يجب أن نحافظ على العالم الذي نحب." قال نينو من بعيد قبل أن يختفي تمامًا.

عاد ياسين إلى قريته، حزينًا لكنه أدرك درسًا قيمًا. قرر أن يستخدم رسوماته ليعلم الناس عن أهمية الحفاظ على البيئة والمحيط. ولكن، كان ياسين يشعر دائمًا بأن هناك شيئًا يفتقد إليه، شيئًا لم يكن يمكن أن يعود إليه.

"أتمنى لو كان بإمكاني رؤية نينو مرة أخرى،" قال ياسين وهو ينظر إلى البحر، وهو يدرك أن بعض الأساطير قد تكون خدعًا، لكن الدرس الذي تعلمه كان حقيقيًا.

Downloading LanguageTool 6.4: 100%|██████████| 246M/246M [00:03<00:00, 71.6MB/s]
INFO:language_tool_python.download_lt:Unzipping /tmp/tmpv8tfaysy.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.
BLEURT Score: 0.2856111526489258
Semantic Coherence: 0.780329704284668
Grammar Errors: 12
Distinct 1-gram Score: 0.6766917293233082
Distinct 2-gram Score: 0.9433962264150944
Perplexity: 1.9509146213531494

Cleaned Story2:
في قرية صغيرة بالصومال، كان يعيش طفلان صغيران، يوسف وأمينة. كان الشتاء قد حلّ بألوانه الزاهية والأمطار الغزيرة، مما جعل الأرض خضراء وجميلة.

ذات يوم، بينما كان يوسف وأمينة يلعبان خارج البيت، قال يوسف بحماس: "أمينة، هل تظن أن هناك كنزًا مخبأً في هذه القرية؟"

ابتسمت أمينة وقالت: "لا أعرف، لكن السحر في هذا الشتاء يجعل كل شيء ممكنًا!"

فكر يوسف وأمينة في كيفية البحث عن الكنز. قررا أن يبدأوا البحث في الحديقة التي تحيط بالبيت. كان هناك شجرة كبيرة، فقررا أن يبدأوا من هناك.

بينما كانا يبحثان، قال يوسف: "يبدو أننا بحاجة إلى بعض المساعدة. هل يمكننا أن نطلب من أحد سكان القرية المساعدة؟"

أجابت أمينة: "نعم، يمكننا أن نطلب من جدك المساعدة. هو يعرف كل شيء عن هذه القرية."

قاما يوسف وأمينة بزيارة جدته، الذي كان يعيش في بيت قريب. عندما سألتهما عن المساعدة، ابتسم جدته وقالت: "بالطبع، يا عزيزي. لكن الكنز الذي أنتما تبحثان عنه، قد يكون ليس كنزًا ماديًا، بل كنزًا في القلب."

فهم يوسف وأمينة ما قاله جدته. قاما بالبحث عن الأشياء الجميلة التي تجعل قلوبهم سعيدة. وجدا أصدقاء جدد، لعبوا معهم، وجدوا جمال الطبيعة التي تحيط بهم.

في نهاية اليوم، قال يوسف لأمينة: "لقد وجدنا الكنز، أمينة! الكنز في قلوبنا!"

أجابت أمينة بابتسامة: "نعم، يا يوسف. هذا هو الكنز الحقيقي."

عاد يوسف وأمينة إلى البيت، وهم يضحكان ويتشاركان قصص اليوم. كانت نهاية يومهم سعيدة، وأدركا أن السعادة تأتي من الصداقة والجمال الذي يحيط بهم.

ومنذ ذلك اليوم، كان يوسف وأمينة يبحثان عن الكنز في كل يوم، ليس في الحديقة فقط، بل في كل مكان يذهبان إليه. وأدركا أن السعادة الحقيقية تأتي من البحث عن الجمال والصداقة في كل لحظة.

BLEURT Score: 0.14631086587905884
Semantic Coherence: 0.8710048794746399
Grammar Errors: 16
Distinct 1-gram Score: 0.6377358490566037
Distinct 2-gram Score: 0.9166666666666666
Perplexity: 1.793131947517395

Cleaned Story3:
في يوم مشرق على شاطئ النهر في العراق القديم، كان هناك طفل يُدعى ياسين. ياسين كان يحب الشاطئ كثيرًا، وكان يحب اللعب على الرمال الذهبية والسباحة في المياه الزرقاء. لكن ياسين كان مختلفًا عن أصدقائه الآخرين؛ فكان يحب الأشغال اليدوية أكثر من اللعب.

ذات يوم، بينما ياسين يبني قلعة من الرمال على الشاطئ، لاحظ قديمًا من الحضارات القديمة يخرج من تحت الأرض. كان هذا القديم يبدو عليه الحزن واليأس.

"أين أصدقائي؟" سأل ياسين بحزن.

"أصدقاؤك ليسوا هنا، يا ياسين. كل منهم كان يعمل في وقت ما. الأصدقاء لا يلعبون طوال الوقت،" قال القديم بصوت حزين.

"لكن أنا لا أريد أن أترك اللعب!" قال ياسين بحزن.

"ياسين، هناك وقت للعمل ووقت للعب. الأشغال اليدوية مثل بناء قلعة من الرمال تعلمك الكثير عن التاريخ والحضارات القديمة. لكن يجب أن تعلم أن هناك وقت للعمل ووقت للعب. لا يجب أن تترك كل شيء للآخر وقتًا." قال القديم.

ياسين نظر إلى قلعة الرمال التي بناها، ثم إلى القديم. "أنا سأكون حذرًا،" قال ياسين وهو يبكي.

"ياسين، الأوقات تختلف، ولكن الدرس الذي تعلمه سيبقى معك إلى الأبد. تذكر،" قال القديم وهو يعود تحت الأرض.

ياسين جلس على الشاطئ، ينظر إلى القلعة التي بناها. كان يشعر بالحزن لأنه لم يكن يدرك أهمية الوقت. لكنه كان ممتنًا للقديم الذي تعلمه درسًا قيمًا.

"سأكون حذرًا،" قال ياسين بصوت خافت وهو ينظر إلى السماء.

وهكذا، انتهت يومه على الشاطئ بحزن، لكنه كان ممتنًا للدرس الذي تعلمه.

BLEURT Score: 0.20754055678844452
Semantic Coherence: 0.8729638457298279
Grammar Errors: 4
Distinct 1-gram Score: 0.6431718061674009
Distinct 2-gram Score: 0.911504424778761
Perplexity: 1.6491005420684814

Cleaned Story4:
في قرية صغيرة بالكويت، كان يعيش ثلاثة أصدقاء: يوسف، ليلى، وسامر. كانوا يلعبون معًا كل يوم تحت أشعة الشمس الدافئة، يتعلمون من بعضهم البعض، ويشاركون الأحلام والأحلام.

يوسف كان يحب اللعب بالكرة، ليلى كانت تحب الرسم، وسامر كان يحب جمع الأشياء القديمة. كانوا يشاركون كل شيء معًا، وكانوا يعتبرون أنفسهم أصدقاء لا ينتهي عددهم.

ذات يوم، قرر الأصدقاء الثلاثة أن يشاركوا في مسابقة رسم ورسومات. كان سامر يرسم لوحات جميلة عن الطبيعة، ليلى كانت ترسم قصصًا عن الأصدقاء، بينما كان يوسف يرسم لوحات عن عالمه المفضل، كرة القدم.

كان الأصدقاء متحمسين جدًا، وبدأوا في العمل على رسوماتهم. كل يوم، كانوا يلتقون تحت الشجرة الكبيرة، يشاركون أفكارهم، ويستمعون إلى بعضهم البعض.

جاء يوم المسابقة، وجمع الأصدقاء رسوماتهم وأحضروها. كانوا متحمسين جدًا لرؤية ما سيحدث. بعد فترة من الوقت، عاد الأصدقاء من المسابقة، ولكنهم كانوا حزينين جدًا.

ليلى كانت قد فازت بالجائزة الأولى، ولكن يوسف وسامر لم يفزوا. كان يوسف حزينًا لأن رسوماته لم تكن كما توقع، وسامر كان حزينًا لأن لوحاته لم تكن كما أراد.

ليلى، التي كانت تحب أصدقائها كثيرًا، حاولت أن ترفع من معنويات يوسف وسامر، قائلة: "أنا فزت، لكن الفوز ليس كل شيء. ما هو مهم هو أننا أصدقاء، وأننا تعلمنا معًا."

لكن الحزن كان يخفي تحتها. كان يوسف وسامر يشعران بأنهم لم يكونوا كافيين، وأنهم لا يستطيعون أن يكونوا مثل ليلى.

في تلك الليلة، جلس الأصدقاء الثلاثة تحت الشجرة الكبيرة، ينظرون إلى النجوم. يوسف قال: "أنا آسف لأنني لم أكون أفضل. أنا فخور بك، ليلى."

ليلى ابتسمت وقالت: "أنا فخورة بكما أيضًا. نحن أصدقاء، ونحن سنجعل أنفسنا أفضل كل يوم."

سامر، الذي كان يراقبهم، قال: "أنا أريد أن أكون مثلكم. أريد أن أكون أفضل."

ليلى، بقلبها الكبير، قالت: "لنكون معًا. لنستمر في التعلم والنمو، ونحن سنجعل أنفسنا أفضل كل يوم."

وهكذا، على الرغم من الحزن الذي شعروا به، تعلم الأصدقاء الثلاثة درسًا قيمًا عن الصداقة والنمو الشخصي. وعلى الرغم من أنهم لم يفوزوا في المسابقة، إلا أنهم فازوا بصداقتهم التي ستظل معهم إلى الأبد.

BLEURT Score: 0.2315598577260971
Semantic Coherence: 0.8763689994812012
Grammar Errors: 16
Distinct 1-gram Score: 0.6529968454258676
Distinct 2-gram Score: 0.9208860759493671
Perplexity: 1.9216291904449463

Cleaned Story5:
في مزرعة جميلة بالجزائر، كان يعيش صديقان لا يفترقان، ياسين وعمر. كانا يحبان اللعب معاً وكل يوم بعد انتهاء أعمال المزرعة، كانا يركبان دراجاتهما الصغيرة حول المزرعة، يلعبان ويضحكان.

ذات يوم، قرر ياسين وعمر أن يشاركا في مسابقة ركوب الدراجات التي ستقام في القرية المجاورة. كانا متحمسين جدًا وأعدا الدراجات ليلحاقا بالمسابقة.

لكن، في اليوم الذي كانا يركبان فيه، حدث شيء غير متوقع. عمر، الذي كان دائمًا صديقًا مقربًا، بدأ يركب بسرعة أكبر من ياسين. ياسين، الذي كان يركب بثبات، شعر بالقلق من أن عمر قد يخلق مشكلة.

عمر، الذي لم يلاحظ قلق ياسين، بدأ يركب بسرعة أكبر فأكبر. ياسين، الذي كان يركب بثبات، شعر بالقلق من أن عمر قد يخلق مشكلة.

فجأة، بينما كان عمر يركب بسرعة، انزلقت دراجته وأصيب عمر. ياسين، الذي كان يركب بثبات، سريعًا إلى جانبه.

عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

بعد أن تمكنا من إسعادهما، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق من صديقه.

عندما عادا إلى المزرعة، قررا أن يعودا إلى المزرعة. عمر، الذي كان يصاب، شعر بالندم على سلوكه. ياسين، الذي كان يركب بثبات، شعر بالقلق

BLEURT Score: 0.2624630928039551
Semantic Coherence: 0.9624589681625366
Grammar Errors: 3
Distinct 1-gram Score: 0.22453703703703703
Distinct 2-gram Score: 0.3039443155452436
Perplexity: 1.841470718383789

Cleaned Story6:
في يوم شتوي بارد في الحديقة الكبيرة بمدينة بغداد، كان هناك ثلاثة أصدقاء: ياسمين، وسامر، وعلي. كانوا يحبون اللعب معاً وكل يوم بعد المدرسة يذهبون إلى الحديقة ليلعبوا. في هذا اليوم، قرروا أن يبنوا كتل من الحجارة.

بدأوا بحمل الحجارة من كل مكان في الحديقة وبدأوا ببناء الكتل. كانوا يضحكون ويتحدثون ويجلسون على كل كتلة يبنونها. لكن، بينما كانوا يبنون الكتلة الأخيرة، سمعوا صوتًا غريبًا يأتي من بين الأشجار. كان الصوت يبدو وكأن هناك شيء يحرك الأشجار.

"ما هذا الصوت؟" سألت ياسمين بقلق.

"لا أعرف، لكن ربما يجب علينا أن نعود إلى المنزل"، قال سامر.

لكن علي، الذي كان يحب المغامرة، قال "لا، دعونا نرى ما هو".

فجأة، خرجت من بين الأشجار فصيلة من الأرنب الصغيرة. الأرنب الصغير كان يبدو خائفًا جدًا.

"لا تخافوا، نحن لن نقترب منكم"، قال علي بصوت دافئ.

الأرنب الصغير بدأ يتقدم شيئًا فشيئًا نحو الأصدقاء. "أنا لست مخيفًا، أنا فقط أبحث عن مكان آمن للاختباء"، قال الأرنب بصوت خافت.

"لقد وجدتم مكانًا آمنًا هنا، يمكنكم البقاء معنا"، قالت ياسمين بحنان.

وافق الأرنب على البقاء مع الأصدقاء وهم يبنون الكتل. في نهاية اليوم، كان الأرنب جزءًا من اللعب والضحك معهم.

"شكرًا لكما، لقد كان يومًا رائعًا"، قال الأرنب وهو يودعهم.

"نحن نحب أن نلعب معك، يمكنك البقاء معنا دائمًا"، قال علي.

ومنذ ذلك اليوم، أصبح الأرنب صديقًا لا يفترق عنهم. كانوا يلعبون معه كل يوم في الحديقة، وكان الأرنب يشاركهم في بناء الكتل.

وهكذا، انتهى يومهم بضحكات ومرح، وكانت نهاية سعيدة.

BLEURT Score: 0.22907787561416626
Semantic Coherence: 0.8796497583389282
Grammar Errors: 12
Distinct 1-gram Score: 0.7361702127659574
Distinct 2-gram Score: 0.9829059829059829
Perplexity: 1.7410953044891357

Cleaned Story7:
في يوم شتوي بارد في فلسطين، كان هناك طفل صغير يدعى يامن. يامن كان يعيش في بيت صغير بجانب حديقة جميلة مليئة بالأشجار والأزهار. كان يامن يحب الحديقة كثيرًا وكان يذهب إليها كل يوم بعد المدرسة.

ذات يوم، بينما يامن يجري في الحديقة، رأى شجرة كبيرة تنمو في وسط الحديقة. كانت الشجرة تبدو قوية ومستقيمة، لكنها بدت قليلاً منعزلة عن باقي الأشجار.

"يا شجرة، كيف تشعر بالوحدة في وسط الحديقة الجميلة هذه؟" سأل يامن بصوت حنون.

"أشعر بالوحدة، يا يامن، لأني لا أعرف كيف أكون جزءًا من هذه الحديقة الجميلة." أجابت الشجرة بصوت خافت.

"لا تقلق يا شجرة، أنا هنا لأساعدك. هل تريد أن أكون صديقك؟" قال يامن بحماس.

"أحب ذلك يا يامن، لكن كيف يمكنك أن تصنع فرقًا؟" سألت الشجرة.

"بكل ما أستطيع، يا شجرة. سأزرع بعض النباتات حولك، وسأجعل الحديقة مكانًا أكثر إشراقًا." قال يامن بثقة.

بدأ يامن بمساعدة الشجرة، زرع بعض النباتات حولها، ووضع بعض الألعاب الصغيرة بجانبها. مع مرور الأيام، بدأت الشجرة تشعر بالفرق. كانت تحصل على المزيد من أشعة الشمس، وأصبحت أزهارها أكثر إشراقًا.

"شكرًا لك يا يامن، أنت صديقي الحقيقي. لقد علمتني أن هناك قوة في الوحدة." قالت الشجرة بحماس.

"أنا سعيد جدًا لأنني استطعت مساعدتك، يا شجرة. وأنا أدرك الآن أن الوحدة يمكن أن تتحول إلى صداقة قوية." قال يامن وهو يضحك.

ومنذ ذلك اليوم، كانت الشجرة جزءًا لا يتجزأ من الحديقة، وكانت يامن يزورها كل يوم. تعلم يامن أن هناك قوة في الوحدة وأن الصداقة يمكن أن تجعل حتى الأشياء الصغيرة تبدو كبيرة.

وهكذا، عاش يامن وشجرة صديقته في سعادة ومرح، وذكريات جميلة في الحديقة الجميلة.

BLEURT Score: 0.21358543634414673
Semantic Coherence: 0.8837054967880249
Grammar Errors: 7
Distinct 1-gram Score: 0.7007874015748031
Distinct 2-gram Score: 0.9525691699604744
Perplexity: 1.6438536643981934

Cleaned Story8:
في قرية صغيرة باليمن، كان يعيش طفل يُدعى يوسف. يوسف كان يحب العزف على العود، وكان لديه موهبة خاصة في هذا الفن. كل يوم بعد المدرسة، كان يوسف يجلس تحت شجرة كبيرة في حديقة منزله، يؤدي أجمل الألحان.

ذات يوم، بينما كان يوسف يؤدي لحنًا جديدًا، شعر بشيء غريب. نظر حوله، لكنه لم يرَ أحدًا. فجأة، بدأت الأرض تتحرك، وكأن هناك شيء يخرج من تحتها.

"من هناك؟" صرخ يوسف بفضول.

"أنا... غريب الأطوار،" أجاب صوتًا غريبًا.

"من أنت؟" سأل يوسف بقلق.

"أنا... نجم الألوان،" قال الصوت.

"نجم الألوان؟" تردد يوسف.

"نعم،" قال الصوت، "أنا كائن أسطوري، أعيش تحت الأرض، وأحب الموسيقى. لكني لا أعرف كيف أبدو أمام الناس."

"لماذا؟" سأل يوسف.

"لأنني... أشعر بالاشمئزاز من أن أكون رؤية الناس. لكني أحب الموسيقى،" قال نجم الألوان.

"لكن الموسيقى جميلة!" قال يوسف بفضول.

"نعم،" قال نجم الألوان، "أنا آسف. أريد أن أكون جزءًا من العالم العلن، لكني لا أعرف كيف."

"لماذا لا تظهر أمام الناس؟" سأل يوسف.

"لأنني... أشعر بالاشمئزاز من أن أكون رؤية الناس. لكني أحب الموسيقى،" قال نجم الألوان.

"لكن الموسيقى جميلة!" قال يوسف بفضول.

"نعم،" قال نجم الألوان، "أنا آسف. أريد أن أكون جزءًا من العالم العلن، لكني لا أعرف كيف."

"أنا سأعلمك كيف تظهر أمام الناس دون أن تشعر بالاشمئزاز!" قال يوسف بحماس.

"أنت؟ كيف؟" سأل نجم الألوان.

"نحن، الأطفال، نستطيع أن نرى العالم من منظور مختلف. يمكننا أن نرى الجمال في كل شيء، حتى في الأشياء التي لا نعتقد أنها جميلة. ربما، يمكنك أن تتعلم منا كيف نرى العالم بطريقة جميلة!" قال يوسف.

"أنت صحيح،" قال نجم الألوان، "أنا سأحاول."

"فعلها!" قال يوسف، "وأنا سأعزف لك بعض الألحان!"

"سأستمتع!" قال نجم الألوان.

من ذلك اليوم فصاعدًا، كان نجم الألوان يظهر أمام يوسف، وكان يوسف يؤدي له الألحان. كان يوسف يرى الجمال في نجم الألوان، وبدأ نجم الألوان يشعر بالسعادة لأنه كان جزءًا من العالم العلن.

"شكرًا لك، يوسف!" قال نجم الألوان.

"لا بأس،" قال يوسف، "أنا سعيد لأنك الآن جزء من العالم العلن."

"أنا أيضًا!" قال نجم الألوان.

ومنذ ذلك اليوم، كان يوسف ونجم الألوان أصدقاء، وكانوا يشاركان الألحان معًا تحت شجرة كبيرة في حديقة يوسف. وكان الجميع في القرية يأخذون دروسًا من صداقة يوسف ونجم الألوان، عن كيفية رؤية العالم من منظور مختلف وجمال كل شيء.

وهكذا، انتهت القصة بسعادة، حيث تعلم الجميع درسًا قيمًا عن القبول والجمال في كل شيء.

BLEURT Score: 0.168941929936409
Semantic Coherence: 0.718830943107605
Grammar Errors: 28
Distinct 1-gram Score: 0.552
Distinct 2-gram Score: 0.7887700534759359
Perplexity: 1.840986967086792

Cleaned Story9:
في محطة الفضاء العجيبة، كان هناك طفل صغير يُدعى يوسف، وروبوت صغير يدعى زيزو. كان يوسف وزيزو أصدقاء لا يفترقان، وكانا يحبان التعلم عن كل شيء جديد.

ذات يوم، بينما كان يوسف يشاهد الأرض من نافذة محطة الفضاء، قال: "زيزو، هل تعلم أن الأرض مليئة بالأشجار والأزهار؟"

"نعم يا يوسف، وأيضًا مليئة بالفواكه والخضروات!" رد زيزو.

"أنا أتمنى أن أزرع بعضها هنا في محطة الفضاء!" قال يوسف بحماس.

"فكرة رائعة يا يوسف! لكن كيف سنزرعها هنا في الفضاء؟" سأل زيزو.

"لنستخدم الماء والتربة التي جمعناها من الأرض!" اقترح يوسف.

وبذلك، بدأ يوسف وزيزو مشروعًا جديدًا. جمعوا الماء والتربة، وزرعوا بذورًا صغيرة. كانوا يراقبانها كل يوم، يغسلونها ويشعرونها بحرارة الشمس الاصطناعية.

"انظر يا زيزو! بدأت البذور تنمو!" قال يوسف متحمسًا.

"نعم يا يوسف، لقد نجحت مشروكتنا! الآن لدينا أشجار صغيرة!" رد زيزو.

"هذه أجمل من أي شيء رأيته في الفضاء!" قال يوسف وهو يبتسم.

"أنا فخور بك يا يوسف، لقد علمتني أن الصحة والتغذية مهمة جدًا. يجب أن نأكل فواكه وخضروات صحية!" قال زيزو.

"نعم، وأنا أتمنى أن أزرع المزيد من الفواكه والخضروات هنا في الفضاء!" قال يوسف.

وبهذا، أصبح يوسف وزيزو أصدقاء أكثر سعادة، يزرعان الفواكه والخضروات، ويستمتعان بجمال الطبيعة في محطة الفضاء. وعاشا في سعادة ومرح، يتعلمان كل يوم شيئًا جديدًا عن الصحة والتغذية.

وهكذا، انتهت قصة يوسف وزيزو، في محطة الفضاء العجيبة، حيث تعلما أن الصحة والتغذية مهمة جدًا، وأن الزراعة يمكن أن تكون مصدرًا للفرح والسعادة.

BLEURT Score: 0.11535634100437164
Semantic Coherence: 0.8805229067802429
Grammar Errors: 31
Distinct 1-gram Score: 0.6768558951965066
Distinct 2-gram Score: 0.8859649122807017
Perplexity: 1.8202903270721436

Cleaned Story10:
في جزر القمر الجميلة، كان هناك صبي صغير يُدعى يوسف، وصديقه الأليف، الطائر النورس. يوسف كان يحب البستنة كثيرًا وكان دائمًا يلاحظ كيف تنمو الأزهار والأشجار تحت أشعة الشمس الدافئة.

ذات يوم، قرر يوسف أن يزرع بذرة جديدة. كان متحمسًا جدًا وقال: "سأزرع أجمل بذرة في العالم! سأكون البستاني الأفضل!".

بدأ يوسف في زراعة البذرة، لكنه كان يتحدث كثيرًا عن كيف سيكون البستاني الأفضل، وكيف سيزرع أجمل بذرة. الطائر النورس كان يراقب يوسف، يلاحظ كل شيء، من كيفية زراعة البذرة إلى كيفية إعطائها الماء والضوء.

بعد أيام، بدأت البذرة تنمو. لكن يوسف لم يلاحظ شيئًا! كان يظن أن البذرة لن تنمو، وأن كل شيء كان يعتمد على كلامه.

فجأة، وقعت عاصفة قوية على الجزر، وبدأت الأمطار تهطل بغزارة. الطائر النورس، الذي كان يراقب يوسف، أدرك أن المطر كان مفيدًا للبذرة.

بينما كان يوسف يخاف من العاصفة، كان الطائر النورس يراقب البذرة، يلاحظ كيف تنمو تحت المطر. وبعد العاصفة، وجد يوسف أن البذرة قد نمت أكثر من أي وقت مضى!

"أنا آسف، يا نورس"، قال يوسف وهو يدرك أن كل شيء لم يكن كما خطط له. "لقد كنت أتحدث كثيرًا وأفكر في الكثير من الأشياء بدلاً من أن ألاحظ ما يحدث حوله."

نورس، الطائر، نظر إلى يوسف وقرر أن يظهر له درسًا. "يوسف، يجب أن تكون أكثر ملاحظة وأقل كلام. الأشياء الحقيقية تحدث عندما نكون حاضرين ونلاحظ."

يوسف، الذي كان يشعر بالندم، وعد بأن يكون أكثر ملاحظة وأقل كلام. وبفضل نصيحة نورس، أصبح يوسف أفضل بستاني في جزر القمر، ليس بسبب كلامه، ولكن بسبب قدرته على لاحظة وتركيزه على الطبيعة.

ومنذ ذلك اليوم، كان يوسف يزرع بذورًا جديدة، لكنه كان دائمًا يلاحظ كيف تنمو تحت أشعة الشمس والرياح. وأصبح يوسف ونورس أفضل الأصدقاء، يشاركان في البستنة ويستمتعان بجمال الطبيعة.

وهكذا، تعلم يوسف درسًا قيمًا: أن يكون أكثر ملاحظة وأقل كلام، وأن الطبيعة تعلمنا الكثير من خلال الحضور واللاحظة.

BLEURT Score: 0.2238285094499588
Semantic Coherence: 0.8835628032684326
Grammar Errors: 36
Distinct 1-gram Score: 0.6430976430976431
Distinct 2-gram Score: 0.9155405405405406
Perplexity: 1.6513164043426514

Cleaned Story11:
في قرية صغيرة جميلة في عمان، كان يعيش طفلان صغيران، ياسين وعمر. كانا أفضل الأصدقاء وكانا يحبان اللعب معاً كل يوم. ياسين كان يحب الرياضة كثيراً، وكان دائماً ما يلعب كرة القدم مع عمر، بينما عمر كان يحب الألعاب الأخرى مثل اللعب بالكرة الصغيرة والقفز على الألوان.

ذات يوم، وجدا في القرية مقاطع فيديو تعليمية عن الرياضة والألعاب. كانوا متحمسين جدًا لتعلم أشياء جديدة. قررا أن يشاهدا المقطع الأول عن كرة القدم، وكان ياسين متحمس جدًا لرؤية كيف يمكنه تحسين مهاراته.

بينما كانا يشاهدان المقطع، حدث شيء غير متوقع. في الفيديو، كان هناك لاعب كرة قدم شاب يظهر أملاً وفرحًا، لكنه كان يلعب ضد ياسين وعمر في المقطع. شعر ياسين وعمر بالدهشة، فلم يكن يعتقدان أن يكونوا في مواجهة لاعب كرة قدم شاب.

بعد مشاهدة المقطع، قررا أن يلعبوا كرة القدم معاً مرة أخرى. لكن هذه المرة، كان ياسين وعمر يلعبان معاً كفريق واحد، وكانوا يشاركان الفرحة والمرح. ولكن، في لحظة غير متوقعة، حدث شيء لم يتوقعاه.

بينما كانوا يلعبون، انزلق ياسين وسقط، مما أدى إلى كسر إصبع في قدمه. شعر عمر بالحزن الشديد لرؤية صديقه ياسين يتعرض للخسارة.

في تلك اللحظة، تبديدت الأدوار بين الأبطال والأشرار. عمر، الذي كان دائماً ما يلعب بفرح، أصبح الآن يلعب بقلق وقلق، يحاول أن يرشد صديقه ياسين إلى الأمان.

وبهذا، انتهت يومهم بقلق وقلق، لكنهم تعلما درسًا قيمًا عن أهمية الصداقة والمرح في الحياة. وعلى الرغم من الحزن الذي شعرا به، إلا أنهم عاهدوا أنفسهم على أن يظلوا أفضل الأصدقاء وأن يشاركوا الفرحة والمرح معاً في المستقبل.

BLEURT Score: 0.23957321047782898
Semantic Coherence: 0.8712420463562012
Grammar Errors: 13
Distinct 1-gram Score: 0.6926229508196722
Distinct 2-gram Score: 0.9465020576131687
Perplexity: 1.9727604389190674

Cleaned Story12:
في قرية صغيرة جميلة في قطر، كان يعيش ثلاثة أصدقاء: الفيل العجوز يوسف، والسلحفاة الصغيرة ليلى، والطيور الصغيرة زينة. كانوا يلعبون معًا كل يوم تحت أشعة الشمس الدافئة.

ذات يوم، قال يوسف بفخر: "أنا أكبر منكما، وأنا أقوى. أريد أن أكون بطلاً في قريتنا!"

ليلى، بابتسامة صغيرة، أجابت: "أنا أتمنى أن أكون أسرع منكما، حتى يمكنني مساعدتك في اللعب!"

زينة، التي كانت تحلق حولهم، قالت: "أنا أتمنى أن أكون أطول، حتى يمكنني رؤية الأشياء من أعلى!"

كان الأصدقاء الثلاثة متحمسين جدًا، وكانوا يلعبون كل يوم، يحلمون بأن يصبحوا بطلين في قريتهم.

ولكن، مع مرور الوقت، بدأت ليلى تشعر بالضعف. كانت تزداد ضعفًا كل يوم، حتى لم تستطع اللعب مع يوسف وزينة.

يوسف، الذي كان يحب ليلى كثيرًا، قال: "ليلى، هل أنت بخير؟ هل يمكنني مساعدتك؟"

ليلى، بحزن، أجابت: "أنا آسفة يا يوسف، أنا لا أستطيع اللعب معكما الآن. أنا حزين جدًا لأنني لا أستطيع المشاركة."

زينة، التي كانت تحلق حزينة، قالت: "نحن نحبك يا ليلى، بغض النظر عن ذلك. هل هناك شيء يمكننا فعله لنجعلك سعيدة مرة أخرى؟"

ليلى، بدموع في عينيها، أجابت: "شكرًا يا أصدقائي. أتمنى أن أكون قادرًا على اللعب معكم مرة أخرى."

ومع مرور الأيام، كانت ليلى تزداد ضعفًا أكثر فأكثر. يوسف وزينة كانا يزورونها كل يوم، يحملان لها القصص والألعاب التي كانت تحبها.

في يوم مشمس جميل، جلس الأصدقاء الثلاثة معًا تحت شجرة كبيرة، ليلى كانت تبتسم بحزن، وقالت: "شكرًا يا أصدقائي. لقد كنت أتمنى أن أكون قادرًا على اللعب معكم، لكني سعيدة لأنني لاحظت كيف أنكم أصدقائي الحقيقيون."

يوسف وزينة، بحزن، نظرا إلى ليلى وقالا: "نحن نحبك يا ليلى، بغض النظر عن ذلك. أنت ستظل دائمًا صديقتنا."

وفي تلك الليلة، غابت ليلى في النوم، وهم يحتضنونها، مدركين أن الحب والصداقة لا يزولان حتى في الأوقات الحزينة.

وهكذا، انتهت قصة الأصدقاء الثلاثة، يوسف وليلى وزينة، مع درس قيم عن الصداقة والحب، حتى في وجه الحزن.

BLEURT Score: 0.16638880968093872
Semantic Coherence: 0.8707908391952515
Grammar Errors: 13
Distinct 1-gram Score: 0.6287625418060201
Distinct 2-gram Score: 0.8926174496644296
Perplexity: 1.9691640138626099

Cleaned Story13:
في مزرعة صغيرة جميلة في اليمن، كان يعيش طائر صغير يُدعى "سمسم". كان سمسم يعيش وحيدًا، لكنه كان يحب المزرعة كثيرًا وكان دائمًا يغرد بفرح.

في يوم صيفي حار، قال سمسم بصوت مرتفع: "يا له من يوم جميل! أشعر بالفرح في كل مكان!"

لكن بينما كان سمسم يغرد، بدأ يلاحظ شيئًا غريبًا. كانت الأشجار تذبل والزهور تتساقط. قال سمسم بصوت حزين: "ماذا يحدث؟ لماذا تبدو المزرعة كذلك؟"

فكر سمسم في كل شيء قد يسبب هذا، لكنه لم يجد أي حل. مرت الأيام واليوم الحار يزداد حرارة. كانت المزرعة تتحول إلى مكان صعب للعيش.

في أحد الأيام، قال سمسم بصوت حزين: "أشعر بالحزن. لا أريد أن أرى المزرعة مثل هذا. أتمنى لو كان بإمكاني فعل شيء."

لكن، حتى لو كان بإمكان سمسم أن يفعل شيئًا، لم يكن هناك حل. كانت المزرعة قد تعرضت للعواصف الرملية الشديدة التي تدمر كل شيء.

في نهاية الصيف، كانت المزرعة قد تحولت إلى مكان صعب للعيش. قال سمسم بصوت حزين: "أنا آسف، يا مزرعة. كنت أحبك كثيرًا. سأغادر الآن، لكنني سأعود يومًا ما."

وهكذا، غادر سمسم المزرعة، تاركًا وراءه الحزن والفراق. لكن، حتى في الحزن، كان هناك درس قيم: أن الحياة يمكن أن تكون صعبة، لكن الأمل والفرح يمكن أن يظلان دائمًا في قلوبنا.

وهكذا، تنتهي قصة سمسم، الطائر الصغير الذي تعلم أن الحب والفرح يمكن أن يظلان دائمًا، حتى في الأوقات الصعبة.

BLEURT Score: 0.17121192812919617
Semantic Coherence: 0.8550638556480408
Grammar Errors: 4
Distinct 1-gram Score: 0.6774193548387096
Distinct 2-gram Score: 0.8935185185185185
Perplexity: 1.713982343673706

Cleaned Story14:
في قرية صغيرة بالبحرين، كان يعيش طفلان صديقان، ياسمين وعمر. كانا يحبان اللعب سوياً تحت أشعة الشمس الدافئة، وكانا يشاركان كل سر وكل لحظة من لحظاتهما.

ذات يوم، قرر ياسمين وعمر أن يذهبا في مغامرة إلى الشاطئ. كانا متحمسين جدًا، لكنهما كانا يخافان من المياه العميقة. كان ياسمين يخاف من الأسماك الكبيرة، بينما كان عمر يخاف من الأمواج القوية.

بينما كانا يلعبون على الشاطئ، شعرا بالانزعاج من خوفهما. ياسمين، التي كانت تحب السباحة، شعرت بالحزن لأنها لم تستطع السباحة بسبب خوف عمر. عمر، الذي كان يحب اللعب، شعر بالحزن لأن ياسمين لم تستطع أن تشاركه مغامرتهما.

فجأة، بينما كانا يلعبون، ظهرت شخصية غامضة على الشاطئ. كانت شخصية غامضة تحمل معها حقيبة صغيرة. قالت الشخصية الغامضة: "أنا هنا لتعليمكم درسًا مهمًا".

فجأة، بدأت الأرض تتحرك وبدأت الأشجار تتحدث. كان ياسمين وعمر في صدمة. الشخصية الغامضة قالت: "كل ما حدث هنا جزء من مسرحية. نحن هنا لتعليمكم أن الخوف يمكن أن يزيله الصداقة".

ياسمين وعمر نظرا إلى بعضهما، وأدركا أن الخوف لم يكن سوى جزء من مغامرتهما. أدركا أن الصداقة هي أقوى من أي خوف.

في نهاية المسرحية، وقف ياسمين وعمر على الشاطئ، يشاهدان الشمس تغرب خلف الأفق. كانا يبتسمان، مدركين أن الصداقة هي ما يجعل كل مغامرة سعيدة.

وبهذا، انتهت مغامرتهم، ولكن ياسمين وعمر تعلما درسًا قيمًا: أن الصداقة هي أقوى من أي خوف، وأن كل مغامرة يمكن أن تكون سعيدة إذا كانت مع صديق.

BLEURT Score: 0.14485113322734833
Semantic Coherence: 0.8797150254249573
Grammar Errors: 4
Distinct 1-gram Score: 0.65625
Distinct 2-gram Score: 0.9013452914798207
Perplexity: 1.7378318309783936

Cleaned Story15:
في قرية صغيرة جميلة في تونس، كان يعيش ثلاثة أصدقاء: ياسين، ليلى، وسامي. كانوا يحبون اللعب معاً وشاركوا في الرياضة والألعاب. كانوا يعتبرون البستنة هو نشاطهم المفضل، حيث كانوا يزرعون الزهور والأشجار في حديقة القرية.

ذات يوم، قرر الأصدقاء الثلاثة أن يشاركوا في مسابقة البستنة الكبيرة التي ستقام في القرية. كانوا متحمسين جدًا، لكنهم كانوا يقلقون قليلاً.

ليلى، وهي تضع الزهور في الحديقة، قالت: "أنا قلقة، ماذا لو لم نكن نتمكن من الفوز؟"

ياسين، وهو يزرع البذور، أجاب: "لا تقلق، سنعمل معاً ونستمتع بالعمل. الفوز ليس كل شيء."

سامي، الذي كان يجمع الفواكه، قال: "أنا أتمنى أن نستطيع مشاركة حديقتنا مع الجميع بعد المسابقة."

ومع مرور الأيام، بدأ الأصدقاء الثلاثة يعملون بجد، يزرعون ويتعلمون. كانوا يشاركون في كل شيء معاً ويتعلمون من بعضهم البعض.

جاء يوم المسابقة، وجمع الجميع في الحديقة. كان هناك العديد من الأشخاص الذين كانوا يشاهدون الحديقة، وكانوا يفكرون في الفوز.

بعد فترة من الوقت، أعلن الحكم عن الفائزين. كانت حديقة ياسين وليلى وسامي الفائزة! كان الجميع في الحديقة يفرحون، وكان الأصدقاء الثلاثة يضحكون ويتحدثون مع بعضهم البعض.

ليلى، وهي تنظر إلى حديقةهم الجميلة، قالت: "لقد فعلناها! لقد كانت رحلة مثيرة!"

ياسين، وهو يضع يده على كتف ليلى، قال: "نعم، لقد فعلناها معاً. وأهم شيء، نحن الآن أصدقاء."

سامي، وهو يبتسم، قال: "وأيضاً، نحن الآن قدمناً حديقة جميلة للجميع."

كانت النهاية سعيدة، حيث شارك الأصدقاء الثلاثة حديقتهم مع الجميع، مما جعل القرية أكثر جمالاً وجمالاً. وذات يوم، أدركوا أن الفوز ليس كل شيء، بل هو العمل الجماعي والصداقة التي خلقت حديقتهم الجميلة.

BLEURT Score: 0.25361090898513794
Semantic Coherence: 0.8678288459777832
Grammar Errors: 13
Distinct 1-gram Score: 0.6938775510204082
Distinct 2-gram Score: 0.9590163934426229
Perplexity: 1.7653716802597046

Cleaned Story16:
في قرية صغيرة بموريتانيا، كان يعيش صبي صغير يُدعى ياسين مع صديقه الأليف، الخرطوم. كان ياسين يحب الصيف كثيرًا، لكنه كان يخاف من المياه. كل صيف، كان ياسين يشاهد الأطفال يلعبون في النهر، لكنه لم يجرؤ أبدًا على الانضمام إليهم.

ذات يوم، بينما كان ياسين يجلس تحت شجرة كبيرة، قال لخرطوم: "أنا حزين جدًا. أريد أن أكون شجاعًا مثل الأطفال الآخرين."

الخرطوم، الذي كان دائمًا بجانب ياسين، نظر إليه وقال: "شجاعة ليست في عدم الخوف، بل في مواجهة الخوف."

فجأة، بدأت السماء تغيم والرياح تهب. "ماذا يحدث؟" سأل ياسين بقلق.

"يبدو أن عاصفة قوية قادمة"، أجاب الخرطوم.

فجأة، ظهرت عاصفة قوية، وبدأت الأمطار تتساقط بغزارة. "يجب أن نجد مأوى!" قال ياسين.

"أنا خائف، لكن يجب أن نكون شجعان"، قال الخرطوم.

بينما كان ياسين يبحث عن مكان مأوى، وجد نفسه أمام النهر. "لا يمكنني أن أتجاهل هذا المرة"، قال لنفسه.

"أنت فعلت ذلك، ياسين! أنت شجاع!" قال الخرطوم وهو يراقب ياسين وهو يركض نحو النهر.

ياسين، مع كل قلبه يخاف، واجه خوفه وبدأ يسبح. كان الماء قارسًا، لكنه لم يركن. بعد لحظات، وجد نفسه على الجانب الآخر، حيث كان هناك مأوى.

"لقد فعلتها! لقد كنت شجاعًا!" قال ياسين وهو ينظر إلى الخرطوم.

لكن، عندما وصل إلى المأوى، وجد أن العاصفة قد تسبب في أضرار كبيرة. "يبدو أننا قد فقدنا البيت"، قال ياسين وهو ينظر إلى الخرطوم.

"لكنك أنت شجاع، ياسين. أنت لم تخف"، قال الخرطوم.

"لكن هذا يعني أننا سنجد مأوى في مكان آخر. سأكون شجاعًا في كل خطوة"، قال ياسين وهو يبتسم.

النهاية كانت حزينة، فقد فقد ياسين وخرطوم بيتهم، لكن الدرس الذي تعلمه كان قيمًا جدًا. أدرك ياسين أن الشجاعة ليست في عدم الخوف، بل في مواجهة الخوف والعمل على تحقيق الأهداف.

ومنذ ذلك اليوم، أصبح ياسين شجاعًا في كل شيء، مدركًا أن الشجاعة تأتي من الداخل، وأنها قوة يمكن أن تغير العالم.

BLEURT Score: 0.2078094780445099
Semantic Coherence: 0.8806864023208618
Grammar Errors: 13
Distinct 1-gram Score: 0.6632653061224489
Distinct 2-gram Score: 0.9180887372013652
Perplexity: 1.990156888961792

Cleaned Story17:
في جزر القمر الساحرة، كان هناك طفلان صغيران يدعيان يوسف وسميرة. كانا يعيشان في قرية صغيرة بالقرب من كهف عظيم. كان يوسف يحب الفنون القتالية والدفاع عن النفس، بينما كانت سميرة تحب الرسم والفن.

ذات يوم، قال يوسف لسميرة: "أنا بطل الفنون القتالية، يمكنني الدفاع عن نفسي وأصدقائي من أي خطر!".

ردت سميرة بحماس: "أنا أيضا بطلة، لكن بطلة الفنون الجميلة! يمكنني رسم أجمل الصور التي تجعل قلوب الناس سعيدة!"

في تلك الليلة، حلم يوسف بأن كان يخوض معركة كبيرة في كهفهم. كان يوسف يحاول الدفاع عن نفسه وسميرة من الأشرار. لكن في الحلم، لم يكن يوسف قويًا كما توقع، فلم يستطع الدفاع عنهم. استيقظ يوسف متوترًا وقال لسميرة: "لقد رأيت حلمًا غريبًا. كان هناك كهف كبير، ووجدت نفسي في معركة كبيرة. لكني لم أستطع الدفاع عنك!"

ردت سميرة بحزن: "يوسف، أنا قلقة. ماذا لو كان الحلم حقيقة؟ ماذا لو وجدنا نفسنا في كهف كبير في يوم ما؟"

يوسف، الذي كان يعتقد أن سميرة تعتقد أن الكهف مجرد حلم، قال: "لا تقلق، سميرة. لن يحدث ذلك. أنا سأكون دائمًا هناك لتحميك."

لكن في اليوم التالي، وجد يوسف وسميرة أنفسهم أمام كهف كبير، كما رأى يوسف في حلمه. قال يوسف بحماس: "رأيت هذا الكهف في حلم! يجب أن نستكشف ما بداخله."

أجابت سميرة بقلق: "يوسف، ماذا لو كان الحلم حقيقة؟ ماذا لو وجدنا نفسنا في معركة كبيرة؟"

يوسف، الذي كان يعتقد أن الكهف مجرد تحدي، قال: "لا تقلق، أنا سأكون هناك لتحميك."

بينما كانا يستكشفان الكهف، وجدا نفسيهما في معركة كبيرة. يوسف، الذي كان يعتقد أنه قوي، وجد نفسه ضعيفًا. سميرة، التي كانت تعتقد أن يوسف سيكون هناك لتحميتها، وجدت نفسها وحيدة.

في تلك اللحظة، أدرك يوسف أن كذبته حول قوته قد كسر ثقة سميرة. قال يوسف بحزن: "أنا آسف، سميرة. لم أكن أقول الحقيقة. أنا لا أستطيع الدفاع عنك كما توقعت."

سميرة، التي كانت تعتقد أن يوسف سيكون هناك لتحميتها، شعرت بحزن شديد. "يوسف، لقد كنت أعتقد أنك ستكون هناك لتحميني. لكنك كذبت علي."

يوسف، الذي كان يعتقد أن الفنون القتالية ستحميه، اكتشف أن الحقيقة الحقيقية هي أن الثقة هي أقوى سلاح. قال يوسف بحزن: "أنا آسف، سميرة. سأكون دائمًا هناك لتحميك، لكن يجب أن أكون صادقًا معك."

النهاية كانت حزينة، لكنها علمت يوسف وسميرة درسًا قيمًا: الكذب يكسر الثقة. وعلى الرغم من الحزن، كانا يدركان أن الصدق والثقة هما أساس أي علاقة قوية.

BLEURT Score: 0.16089317202568054
Semantic Coherence: 0.874419629573822
Grammar Errors: 36
Distinct 1-gram Score: 0.5721925133689839
Distinct 2-gram Score: 0.8445040214477212
Perplexity: 1.8820405006408691

Cleaned Story18:
في قرية صغيرة بالجزائر، حيث تتساقط أوراق الأشجار بألوانها الذهبية والبرتقالية مع حلول فصل الخريف، كان يعيش ثلاثة أصدقاء: ياسين، ليلى، وسمير. كانوا يعشقون اللعب معًا تحت أشجار الزيتون العتيقة، يضحكون ويتشاركون القصص.

ذات يوم، بينما كانوا يلعبون قرب النهر، وجدوا عصفور صغير يبدو أنه فقد طريقه. "يجب أن نชجعه!" قالت ليلى بحماس. "نحن يجب أن نأخذه إلى المنزل ونعطيه الطعام!" أضاف ياسين.

"لكن ماذا لو كان يحتاج إلى مساعدة أكثر من مجرد الطعام؟" سأل سمير بقلق.

"لنقلوه إلى المنزل ونرى ما يمكننا فعله!" اقترح ياسين.

وهكذا، قرر الأصدقاء الثلاثة أن يأخذوا العصفور الصغير إلى منازلهم. لكن، أثناء العودة، فاتت ياسين فرصة لاستعادة بعض الفواكه التي كانت على شجرة قريبة. "أنا سأعود لأخذ بعض الفواكه!" قال ياسين، متجاهلاً رغبة أصدقائه.

عندما عاد ياسين، وجد ليلى وسمير قد أصبحوا قلقين على العصفور الصغير. "ياسين، ماذا فعلت؟" سألت ليلى بحزن. "لقد كاننا نعتني به، لكنك غادرت لبعض الوقت."

"أنا آسف، لم أكن أعلم..." قال ياسين، يشعر بالذنب.

"ياسين، العصفور يحتاج إلى مساعدة أكثر من مجرد الطعام. يجب أن نكون كلهم معًا." قال سمير، وهو يضع يده على كتفه.

"أنا آسف، سأكون هنا معكم. دعونا نعمل معًا." قال ياسين، وهو يبدي القبول.

معًا، ساعد الأصدقاء الثلاثة العصفور الصغير، وهم يتعلمون كيف يكونون أفضل في مساعدة الآخرين. وفي النهاية، عادت السعادة إلى قلوبهم، وأصبحوا أصدقاء أفضل.

"لقد علمتنا هذه التجربة أن العمل معًا يجعل كل شيء أفضل." قال سمير، وهو ينظر إلى العصفور الصغير الذي بدأ يغرد مرة أخرى.

"نعم، وأيضًا أن الشجاعة تكمن في الاعتراف بالخطأ وطلب المساعدة." أضاف ياسين.

"وأنا سعيد لأننا أصبحنا أفضل أصدقاء." قالت ليلى، وهي تنظر إلى أصدقائها.

وهكذا، في قريتهم الصغيرة بالجزائر، تعلم الأصدقاء الثلاثة درسًا قيمًا عن الصداقة، الشجاعة، والعمل معًا، وأصبحوا أفضل بكثير.

BLEURT Score: 0.2578570246696472
Semantic Coherence: 0.8117620944976807
Grammar Errors: 11
Distinct 1-gram Score: 0.7509025270758123
Distinct 2-gram Score: 0.9384057971014492
Perplexity: 2.2395336627960205

Cleaned Story19:
في قلب البحرين، حيث الشمس تلمع بين السحب، وفي فصل الشتاء البارد، كان هناك كهف كبير يخفي أسرارًا قديمة. في هذا الكهف، كان يعيش ثلاثة أصدقاء: رامي، سارة، وسمير. كانوا يلعبون كل يوم تحت ضوء النجوم، ولكنهم لم يكنوا يعرفون ما يخبئه الكهف.

ذات يوم، قرر الأصدقاء الثلاثة أن يذهبوا في مغامرة داخل الكهف. قال رامي بحماس: "أعتقد أننا سنجد كنزًا عظيمًا!" أجابت سارة بفضول: "أو ربما نجد سحرًا يجعلهم يضحكون!" وضحك سمير قائلًا: "أو ربما نجد طريقه إلى أرض الفضائيات!"

بينما كانوا يتجولون داخل الكهف، بدأوا يشعرون بالبرد. قال سمير: "يبدو أن هذا الكهف ليس مكانًا جيدًا للعب!" لكن رامي، الذي كان شجاعًا ومبدعًا، قال: "لا تقلق، سأجد طريقة لجعل هذا المكان أكثر إثارة!"

فجأة، بينما كانوا يبحثون، وجدوا صندوقًا قديمًا. قال سارة بحماس: "ربما هذا هو الكنز الذي بحثنا عنه!" فتحوا الصندوق، لكن بدلاً من ذهب أو جواهر، وجدوا رسالة قديمة. كانت الرسالة تحكي عن كيفية العثور على كنز البحرين الحقيقي.

كانت الرسالة تشير إلى أن الكنز الحقيقي لم يكن ذهبًا أو جواهرًا، بل كان شجاعةً وصدقًا. قال رامي: "أعتقد أن هذا يعني أن الشجاعة والصدق هما الكنز الحقيقي!" وافق الأصدقاء الثلاثة، وهم يدركون أن مغامرتهم كانت ليست عن العثور على كنز مادي، بل عن اكتشاف قوة البطولة والشجاعة في قلوبهم.

وهكذا، عاد الأصدقاء الثلاثة إلى المنزل، ليس فقط بقلوب ممتلئة بالفرح، بل أيضًا بقلوب مليئة بالشجاعة والصدق. كانت مغامرتهم في الكهف قد علمتهم درسًا قيمًا: أن البطولة والشجاعة هي الكنز الحقيقي الذي لا يمكن أن يأخذوه أحد.

وبهذا، انتهت مغامرتهم في الكهف، لكنها بدأت حكاية جديدة عن شجاعة الأصدقاء الثلاثة، والقوة التي كانت في قلوبهم.

BLEURT Score: 0.16098293662071228
Semantic Coherence: 0.8402331471443176
Grammar Errors: 10
Distinct 1-gram Score: 0.7054263565891473
Distinct 2-gram Score: 0.9494163424124513
Perplexity: 1.9850956201553345

Cleaned Story20:
في قرية صغيرة في الصومال، كان يعيش طفل صغير يُدعى يوسف. يوسف كان معروفًا بين أهل القرية بشجاعته وثقته بنفسه، لكنه كان دائمًا يفكر في كيفية أن يصبح شجاعًا أكثر.

ذات يوم، بينما كان يوسف يلعب بجانب النهر، سمع صوتًا يأتي من تحت الماء. كان الصوت يحتاج إلى المساعدة. يوسف شعر بقلب يخفق بسرعة، لكنه تذكر الدرس الذي تعلمه من والده: "الشجاعة تكمن في القلب، وليست في الخوف".

"من هناك؟" صرخ يوسف بصوتٍ عالٍ.

"أنا... أنا أسد صغير. أتمنى أن تساعدني!" أجاب الصوت.

يوسف شعر بثقة يزداد شيئًا فشيئًا. "لا تقلق، سأجلب لك المساعدة!" قال.

يوسف لم يستطع السباحة، لكنه لم يفقد الأمل. فكر في كيفية استخدام ما لديه من قوة. جمع بعض الأخشاب من حوله وصنع قاربًا صغيرًا. ثم، بكل شجاعة، قاد القارب نحو الصوت تحت الماء.

"أنا هنا! هل يمكنك أن تخرج من الماء؟" قال يوسف.

"شكرًا لك يا يوسف! لكنني لا أستطيع الخروج من الماء دون مساعدة." قال الأسد الصغير.

يوسف، بقلبه الكبير، فكر في كيفية مساعدة الأسد الصغير. "لدي فكرة! يمكنني استخدام هذه الأخشاب لصنع جسر صغير!" قال.

باستخدام الأخشاب، صنع يوسف جسرًا صغيرًا يربط بين الأرض والماء. الأسد الصغير، بكل حرص، خرج من الماء ووصل إلى الجسر.

"شكرًا لك يا يوسف! لقد كنت شجاعًا جدًا!" قال الأسد الصغير.

يوسف، بابتسامة عريضة، قال: "شكرًا لك على الثقة بالنفس. لقد علمتني أن الشجاعة تكمن في القلب."

عاد يوسف إلى القرية، حيث احتفل به أهل القرية بكل شجاعة. يوسف تعلم أن الشجاعة لا تعني عدم الخوف، بل تعني الثقة بالنفس وقدرة الفرد على القيام بما هو صعب.

ومنذ ذلك اليوم، أصبح يوسف مثالًا للشجاعة والثقة بالنفس في القرية، وكل من يسمع قصته يتعلم درسًا قيّمًا عن العمل الجماعي والتعاون.

BLEURT Score: 0.25164905190467834
Semantic Coherence: 0.7834673523902893
Grammar Errors: 19
Distinct 1-gram Score: 0.7084870848708487
Distinct 2-gram Score: 0.937037037037037
Perplexity: 1.7641617059707642

Cleaned Story21:
في حديقة الحيوان الكبيرة في الأردن، حيث تتساقط الثلوج الدافئة في فصل الشتاء، كان هناك طائر صغير يُدعى "سمسم"، وقلب طائر كبير يُدعى "زيزو". كان سمسم يغني أجمل الألحان، بينما كان زيزو يغني بأصوات عميقة وجميلة.

ذات يوم، قرر زيزو أن يشارك سمسم في نشاط جديد: البستنة. كان يريد أن يزرع حديقة جميلة في حديقة الحيوان، حيث يمكن للطيور أن تزرع أزهارها وورودها. لكن، كان هناك مشكلة واحدة: كان زيزو يعتقد أن البستنة هي شغل للأشرار، وليس للأبطال.

"أنا لا أستطيع البستنة، أنا بطل!" قال زيزو بحماس.

"لكن، يا زيزو، البستنة هي عمل جميل!" قال سمسم بفضول.

"أنا لا أعتقد، لكن سأفكر في ذلك..." قال زيزو.

في اليوم التالي، قرر زيزو أن يتبع سمسم في البستنة. كان يزرع بذور الأزهار والأعشاب، وكان يغني لها كل يوم. وبمرور الوقت، بدأت الأزهار تتفتح، والأعشاب تتساقط، وكانت الحديقة تبدو أجمل من أي وقت مضى.

"أنا فخور بكما!" قال الحارس على حديقة الحيوان، وهو ينظر إلى زيزو وسمسم.

"لقد أدركت أن البستنة هي عمل جميل، وأنا سعيد لأنني شاركت!" قال زيزو.

"وأنا سعيد لأنني تعلمت أن الموسيقى يمكن أن تجذب الناس إلى الحديقة!" قال سمسم.

في نهاية اليوم، كان زيزو وسمسم يغنيان معًا، وكل طائر في الحديقة كان يغني معهم. كانت الحديقة مليئة بالمرح والجمال، وكانت الموسيقى تملأ الجو.

"لقد أصبحت حديقة الحيوان أجمل من أي وقت مضى!" قال الحارس بفرح.

"لقد فعلنا ذلك معًا!" قال زيزو وسمسم معًا.

وهكذا، أصبح زيزو وسمسم أصدقاء، وأصبحوا بطلانًا في حديقة الحيوان. وعاشا في سعادة، يغنيان ويبستان ويجعلان حديقة الحيوان مكانًا جميلًا لكل طائر.

وهكذا، كانت نهاية القصة سعيدة، حيث تعلم زيزو أن البستنة يمكن أن تكون عملًا جميلًا، وأن الموسيقى يمكن أن تجذب الناس إلى الحديقة.

BLEURT Score: 0.15405166149139404
Semantic Coherence: 0.7882713079452515
Grammar Errors: 17
Distinct 1-gram Score: 0.6
Distinct 2-gram Score: 0.8736059479553904
Perplexity: 1.776816487312317

Cleaned Story22:
في مدينة ملاهي جميلة بالعراق، حيث الأطفال يضحكون ويلعبون تحت ضوء الشمس الذهبي في فصل الخريف، كان هناك طفل صغير يُدعى يوسف. يوسف كان يحب اللعب والاستكشاف، لكنه كان يخشى الملاهي كثيرًا بسبب شخصٍ واحد، كان يُدعى سامر.

سامر كان معروفًا بين أطفال المدينة بأنه يخيفهم ويجعلهم يبكون. كان يبدو عليه أن هناك شيئًا غريبًا في قلبه، شيئًا يجعله يريد أن يخيف الآخرين. يوسف كان يخاف سامر كثيرًا، لكنه كان دائمًا يحلم بأن يتعلم لغة جديدة، ليكون قادرًا على التحدث مع أصدقائه من بلدان أخرى.

ذات يوم، قرر يوسف أن يتعلم اللغة الإنجليزية، ليكون قادرًا على التحدث مع الأطفال من بلدان أخرى في الملاهي. بينما كان يوسف يتعلم، التقى بصديق جديد، كان يُدعى علي. علي كان صديقًا لسامر، لكنه لم يكن مثل سامر. علي كان صديقًا جيدًا، يضحك ويجعل يوسف يشعر بالراحة.

في يوم ماطر، قرر يوسف أن يذهب إلى الملاهي ليلعب. كان يوسف يخشى الملاهي كثيرًا، لكنه قرر أن يثبت لنفسه أنه ليس خائفًا. بينما كان يوسف يلعب، ظهر سامر فجأة. كان يوسف يخشى سامر كثيرًا، لكن علي، الذي كان يراقب يوسف من بعيد، قرر أن يتدخل.

علي، الذي كان صديقًا لسامر، ذهب إليه وأخبره بأن يوسف ليس خائفًا، بل هو شجاع. سامر، الذي كان يبدو عليه الارتباك، أخبر علي أنه كان يريد فقط أن يكون صديقًا ليوسف. علي، الذي كان يعتقد أن سامر كان شخصًا شريرًا، كان صدمةً جدًا عندما اكتشف أن سامر كان في الحقيقة والده.

كان سامر، الذي كان يبدو عليه الارتباك، يريد فقط أن يكون صديقًا لابنه. كان سامر يخشى أن يفقد ابنه بسبب خوفه من الملاهي، لكنه لم يكن يعرف كيف يبدل هذا. علي، الذي كان صديقًا لسامر، قرر أن يظهر له أن يوسف ليس خائفًا، بل هو شجاع.

يوسف، الذي كان يخشى سامر كثيرًا، شعر بالامتنان الشديد لسامر الذي أراد أن يكون صديقًا له. يوسف، الذي كان يخشى الملاهي، شعر بالشجاعة لأنه رأى كيف يمكن للشجاعة أن تغير الأمور. يوسف، الذي كان يخشى سامر، أصبح صديقًا لسامر، والثلاثة، يوسف، علي، وسامر، أصبحوا أصدقاء.

الدرس المستفاد من هذه القصة هو أن اتق شر من أحسنت إليه، لكن أيضًا أن الشجاعة يمكن أن تغير الأمور. يوسف، الذي كان يخشى الملاهي، أصبح شجاعًا، وأصبحت الملاهي مكانًا ممتعًا بالنسبة له. يوسف، الذي كان يخشى سامر، أصبح صديقًا لسامر، والثلاثة أصبحوا أصدقاء.

BLEURT Score: 0.19419030845165253
Semantic Coherence: 0.9038289785385132
Grammar Errors: 24
Distinct 1-gram Score: 0.4945054945054945
Distinct 2-gram Score: 0.7713498622589532
Perplexity: 1.9984638690948486

Cleaned Story23:
في يومٍ جميل من أيام الربيع في الكويت، كان هناك طفل صغير يُدعى يوسف. يوسف كان يحب العزف على العود، الآلة الموسيقية التي أحبها أكثر من أي شيء آخر في العالم. كان يوسف يعزف كل يوم تحت شجرة النخيل الكبيرة في حديقة منزلهم، وكانت الألحان تملأ الهواء بألوانها الزاهية.

ذات يوم، بينما كان يوسف يعزف، بدأت السماء تغيم فجأة. بدأت الأمطار تهطل بغزارة، وسرعان ما تحولت الحديقة إلى نهر. يوسف، الذي كان غاضبًا جدًا من تغير الطقس المفاجئ، شعر بالخوف والغضب في نفس الوقت. كان يوسف وحيدًا، ولم يكن هناك أحد يسمع صوته.

بينما كان يوسف يجد نفسه وسط المياه، بدأ يوسف يشعر بالحزن واليأس. لكن في قلب هذا الحزن، وجد يوسف شيئًا ملهمًا. تذكر أن الموسيقى يمكن أن تجلب السعادة حتى في أصعب الأوقات. فبدلًا من الاستسلام للخوف، قرر يوسف أن يعزف على العود، متجاهلاً المياه التي كانت تملأ الأرض.

بكل قوته، بدأ يوسف يعزف، وبدأت الألحان تملأ المياه. كانت الموسيقى تتدفق مثل نهر الذهب، تجمع بين السماء والأرض في لحظة سحرية. يوسف، الذي كان يبدو صغيرًا ومحطمًا، بدأ يجد قوته من خلال الموسيقى.

في تلك اللحظة، حدث شيء غير متوقع. بدأت السماء تتفتح مرة أخرى، وتوقف المطر. كان كما لو أن الموسيقى كانت تحمل الرسالة التي جلبت السلام والهدوء إلى الطبيعة. يوسف، الذي كان يبدو حزينًا في البداية، شعر بالفرح والسعادة. لقد نجح في إخبار العالم بأنه لا يزال هناك قوة في الصمت والغضب.

لكن، عندما عاد يوسف إلى المنزل، كان هناك شيء لم يكن يأمل فيه. كانت الأمطار قد جلبت معها قوة كبيرة، مما أدى إلى تدمير حديقة منزلهم. يوسف، الذي كان يحب العزف تحت شجرة النخيل، شعر بحزن عميق. فقد حديقته المفضل، لكنه اكتشف شيئًا أثمن: قوة الموسيقى التي يمكن أن تجلب السعادة حتى في أصعب الأوقات.

وهكذا، على الرغم من الحزن الذي شعر به يوسف، إلا أنه تعلم درسًا قيمًا عن القوة التي يمكن أن توجد في داخله، والقدرة على تجاوز الصعاب من خلال الموسيقى.

BLEURT Score: 0.2453320324420929
Semantic Coherence: 0.8891869187355042
Grammar Errors: 25
Distinct 1-gram Score: 0.6419354838709678
Distinct 2-gram Score: 0.8964401294498382
Perplexity: 1.7072780132293701

Cleaned Story24:
في قرية صغيرة بالعراق، حيث تزهو الأرض بألوان الربيع الجميلة، كان يعيش ثلاثة أصدقاء: ياسمين، أحمد، وسمر. كانوا يحبون الطهي كثيرًا وكانوا دائمًا يجمعون معًا لتعلم وصفات جديدة.

ذات يوم، قرروا أن يصنعوا وجبة بسيطة من الكشري والخضروات. ياسمين، وهي الأكبر، قالت: "لنعدم أن نضيف بعض الزيت الزيتون للحصول على طعم أفضل!" أحمد، وهو الأكثر حماسة، أجاب: "نعم! سأذهب لأحضر الزيت الزيتون من المنزل!"

بينما كان أحمد يذهب، انزلقت سمر، وهي الصغيرة، وسقطت على الأرض. ياسمين، التي كانت تحاول مساعدة سمر، قالت: "تعال، سمر! هل أنت بخير؟" سمر، وهي تبكي، أجابت: "نعم، أنا بخير. لكني خافتت."

عند عودة أحمد، وجد ياسمين وسمر في حالة من القلق. أحمد قال: "ماذا حدث؟" ياسمين، وهي تشرح: "سمر سقطت وسكبت دموعها."

أحمد، الذي كان يحب سمر كثيرًا، قال: "لا تقلق، سمر. سأكون بجانبك دائمًا. دعونا نطهو وجبتنا معًا، ونستمتع بوقتنا معًا."

بينما كانوا يطهون، كانت السماء تغيم. فجأة، بدأت السماء تتساقط بالأمطار بغزارة. ياسمين، وهي ترى الأمطار، قالت: "يبدو أن الأمطار قد جلبت معها قلقًا على يومنا."

أحمد، وهو يضع يده على كتف سمر، قال: "لكننا معًا، سنجعل يومنا أفضل. دعونا نستمتع بوقتنا معًا، مهما حدث."

بعد لحظات، انقطعت الكهرباء، وتمكنت الضوء الطبيعي من إضاءة المكان. ياسمين، وهي تشعر بالحزن، قالت: "يبدو أننا لن نتمكن من تذوق وجبتنا كما خططنا."

سمر، وهي تنظر إلى ياسمين وأحمد، قالت: "لكن لقد كان أجمل يومًا. لقد تعلمت أن هناك قوة في الوحدة، وأن الأصدقاء يجعلون أي يوم أجمل."

أحمد، وهو ينظر إلى ياسمين، قال: "نعم، لقد تعلمنا درسًا قيمًا اليوم. حتى في أصعب الأوقات، يمكن أن يكون هناك سعادة في الوحدة."

وبينما كانت الأمطار تغمر القرية، جلس الأصدقاء الثلاثة معًا، يشاركون القصص والأحلام، وهم يدركون أن القوة الحقيقية تكمن في صداقتهم والوحدة التي يشاركونها.

BLEURT Score: 0.23366445302963257
Semantic Coherence: 0.8690466284751892
Grammar Errors: 4
Distinct 1-gram Score: 0.7328519855595668
Distinct 2-gram Score: 0.967391304347826
Perplexity: 1.8247772455215454

Cleaned Story25:
في قرية صغيرة بالمملكة العربية السعودية، كان يعيش ثلاثة أصدقاء: يوسف، سارة، وعلي. كانوا يحبون حل الألغاز واللعب معاً تحت ظل شجرة كبيرة.

ذات يوم، قال يوسف بحماس: "أنا وجدت لغزًا جديدًا! هل تريدون أن نحلوه معًا؟"

"نعم! دعونا نبدأ!" قالت سارة بفضول.

"أنا متأكد أننا سنجد الحل!" قال علي بثقة.

بدأ الأصدقاء الثلاثة يحلون اللغز، وكل خطوة كانت تحققهم أكثر قربًا للحقيقة. ولكن، في اللحظة الأخيرة، وجدوا أن الحل لم يكن كما توقعوا.

"لماذا لم يكن الحل كما أتمنى؟" قال يوسف بحزن.

"ربما كان يجب علينا التفكير في الحل بجدية أكثر"، قالت سارة.

فجأة، ظهرت شخصية جديدة على المسرح. كانت هي إحدى ممثلات المسرحية التي كانت تقوم بتمثيل قصة الأصدقاء الثلاثة.

"أه، هل أنتم متأكدين من أن الحقيقة كانت كما قمت بها؟" سألت المثيلة بابتسامة.

"لا، لقد كاننا نعتمد على توقعاتنا"، قال علي.

"هذا درس قيم. لا تعتمد على شيء حتى تكون متأكدًا من حدوثه. الأمانة والصدق هما أساس كل شيء"، قالت المثيلة.

الأصدقاء الثلاثة نظروا إلى بعضهم البعض، وهم يدركون الدرس الذي تعلموه. ولكن، في قلوبهم، كانت هناك حزن لأن القصة لم تكن كما توقعوا.

"لكن، في النهاية، ما هو المهم هو أننا تعلمنا درسًا قيماً"، قال يوسف.

"نعم، والصداقة بيننا ستظل أقوى من أي لغز"، قالت سارة.

"وأنا سأكون دائمًا متأكدًا من أن الأمانة والصدق هما أساس كل شيء"، قال علي.

وهكذا، انتهت القصة على ملاحظة حزينة ولكن درس قيم، مع أمل في مستقبل أفضل.

BLEURT Score: 0.2025231420993805
Semantic Coherence: 0.8667111992835999
Grammar Errors: 17
Distinct 1-gram Score: 0.7543859649122807
Distinct 2-gram Score: 0.9383259911894273
Perplexity: 1.903893232345581

Cleaned Story26:
في قرية صغيرة جميلة في الجزائر، كان يعيش طفلان صديقان لا يفترقان، ياسين وعمر. كان ياسين يحب اللعب خارج البيت، بينما كان عمر يحب القراءة والاستمتاع بروايات الأحلام والأمنيات.

ذات يوم، بينما كان ياسين وعمر يجلسان تحت شجرة كبيرة، قال عمر: "ياسين، هل تعلم أنني أحلم دائمًا بأن أصبح مهندسًا كبيرًا وأبناك يروي قصص الأحلام والأمنيات؟"

ابتسم ياسين وقال: "أنا أيضًا أحلم بأن أصبح بطلاً في قريتنا، أستطيع أن أحمي الجميع!"

عمر نظر إلى ياسين وقال: "لدي فكرة! لماذا لا نعمل معًا لتحقيق أحلامنا؟"

"كيف يمكننا ذلك؟" سأل ياسين بحماس.

"نحن يمكن أن نبدأ بمساعدة بعضنا البعض. يمكننا أن نستخدم مهاراتنا لتحقيق أحلامنا. أنت يمكن أن تكون بطلاً في قريتنا، وأنا يمكن أن أستمر في قراءة قصص الأحلام والأمنيات وأشاركها مع الجميع."

"هذا فكرة رائعة!" صرخ ياسين.

في الأيام التالية، عمل ياسين وعمر معًا. ياسين كان يشارك عمر قصص الأحلام والأمنيات التي كان يقرؤها، بينما كان عمر يشارك ياسين الأفكار حول كيفية تحقيق أحلامه.

مع مرور الوقت، بدأت قصص الأحلام والأمنيات تنتشر في كل زاوية من زوايا القرية. الأطفال كانوا يشاركون قصص الأحلام والأمنيات مع بعضهم البعض، مما جعل القرية مكانًا أكثر سعادة.

في النهاية، أصبح ياسين بطلاً في قلوب الجميع، وأصبح عمر معروفًا بروايته الرائعة عن الأحلام والأمنيات.

"لقد فعلناها! لقد تحققت أحلامنا!" قال عمر وهو ينظر إلى ياسين.

"نعم، لقد فعلناها معًا!" أجاب ياسين، وهو يبتسم بفخر.

ومنذ ذلك اليوم، أصبح ياسين وعمر أفضل الأصدقاء، وكانا دائمًا يشاركان أحلامهما والأمنيات معًا، متذكرين دائمًا أن الأحلام والأمنيات يمكن أن تصبح حقيقة عندما نعمل معًا.

BLEURT Score: 0.20693239569664001
Semantic Coherence: 0.8553892970085144
Grammar Errors: 9
Distinct 1-gram Score: 0.6571428571428571
Distinct 2-gram Score: 0.9016393442622951
Perplexity: 2.0531318187713623

Cleaned Story27:
في قرية صغيرة بموريتانيا، حيث الشمس تلمع بألوانها الزاهية في فصل الصيف، كان يعيش ثلاثة أصدقاء: يوسف، أمينة، وسعيد. كانوا يلعبون كل يوم تحت ظلال النخيل، يضحكون ويتشاركون أحلامهم.

ذات يوم، بينما كانوا يلعبون، ظهر ثلاثة من الأشخاص الذين كانوا معروفين في القرية بأنهما الأشرار. كانوا يُدعون "المرعوبون". كانوا يرتدون ملابس سوداء، ويحملون أدواتهم المخيفة.

"ماذا تفعلون هنا؟" صرخ "المرعوبون"، وهو يرفع صوته.

"نحن فقط نلعب، لا تقلقوا!" أجاب يوسف بحزم.

"لن نتركهما!" قال "المرعوبون"، وهو يخطو نحو الأصدقاء.

فجأة، حدث شيء غير متوقع. بينما كان "المرعوبون" يخطون نحو الأصدقاء، بدأت الأرض تتزلزل. ثم، فجأة، تبديل مفاجئ للأدوار. "المرعوبون" بدأ يبدو وكأنهم يخافون من الأصدقاء، بينما يوسف، أمينة، وسعيد بدأوا يشعرون بالراحة.

"أنا... أنا آسف،" قال "المرعوبون" الذي كان يبدو الآن كشاب صغير، "لقد كاننا نلعب دوراً في مسرحية، لكننا فشلنا في التوقف عن اللعب قبل أن نصل إلى هنا."

"أنا سعيد جدًا أنكما لا تريدون إيذاننا!" قال يوسف بابتسامة.

"نعم،" قال "المرعوبون"، "لقد أدركنا أن اللعب والضحك هو ما يجعل الحياة جميلة."

لكن، في تلك اللحظة، حدث شيء حزين. بينما كانوا يتحدثون، ظهرت عاصفة قوية، مما جعلهم يفقدون بعض الأشياء التي كانت معهم.

"أنا آسف،" قال "المرعوبون" وهو يبكي، "لقد كنا نريد أن نلعب دور الأشرار، لكننا لم نتوقع أن نؤذي أحدًا."

"لا تقلقوا،" قال يوسف، "كلنا يخطئ. لكن الأهم هو أننا الآن أصدقاء."

وبينما عاد الأصدقاء إلى بيوتهم، كانوا يحملون معهم درسًا قيمًا عن أهمية اللعب والضحك، وكيف أن الأخطاء يمكن أن تصبح فرصًا للتعلم والصداقة. ومع ذلك، كان هناك حزن في قلوبهم لفقدان بعض الأشياء التي كانت معهم.

"أنا سأحاول أن أكون أفضل في المستقبل،" قال "المرعوبون" وهو يودع الأصدقاء.

"نحن سنجدك،" قال يوسف، "لأن الصداقة لا تعرف حدود."

وهكذا، انتهت يومهم بقلوبهم مليئة بالحزن والحب، ولكن مع أمل جديد في الصداقة والتعاون.

BLEURT Score: 0.144413024187088
Semantic Coherence: 0.7952597141265869
Grammar Errors: 17
Distinct 1-gram Score: 0.7272727272727273
Distinct 2-gram Score: 0.9473684210526315
Perplexity: 1.9250507354736328

Cleaned Story28:
في غابة جميلة وواسعة في ليبيا، كان يعيش ثلاثة أصدقاء: الأرنب يوسف، والسلحفاة سارة، والثعلب طارق. كان الأصدقاء الثلاثة يحبون اللعب معًا، لكنهم كانوا يبحثون عن نشاط جديد يمكن أن يشاركوه معًا.

ذات يوم، وجدوا ثلاثة دراجات صغيرة تتراقص في الشمس الذهبية. كانت الدراجات جميلة ومشرقة، لكنها كانت قديمة بعض الشيء.

"هل يمكننا أن نركب هذه الدراجات؟" سأل يوسف بفضول.

"نعم! لقد تبدو ممتعة!" أجابت سارة بابتسامة.

"وأنا أعتقد أن هذا سيكون مغامرة رائعة!" قال طارق بحماس.

فكر الأصدقاء الثلاثة في كيفية جعل الدراجات قادرة على الحركة. بعد بعض المحاولات، وجدوا أنهم يمكن أن يحركوا الدراجات بالقفزات والأقدام.

"هيا، لن نستطيع ركوبها بسرعة، لكن يمكننا أن نركبها!" قال يوسف.

"نعم، هذا هو المهم! نحن معًا!" أضافت سارة.

"وأنا أعتقد أن هذا سيكون أجمل مغامرة في الغابة!" قال طارق.

قرروا أن يذهبوا في مغامرة حول الغابة، يركبون دراجاتهم الصغيرة. كانوا يضحكون ويتحدثون ويجربون كيف يمكنهم السرعة والتحكم في دراجاتهم.

بينما كانوا يركبون، لاحظوا جمال الغابة من حولهم. الأشجار كانت تلوح بألوانها الخضراء والبنية، والأزهار كانت تنبض بألوانها الزاهية.

"هذه كانت مغامرة رائعة!" قال يوسف وهو يتوقف.

"أنا أتفق معك. لقد كانت أجمل مغامرة!" قال طارق.

"نعم، لقد كانت جميلة جدًا. وأنا سعيدة لأننا قمنا بها معًا." قال سارة.

عاد الأصدقاء الثلاثة إلى منازلهم، يضحكون ويتشاركون قصتهم مع أصدقائهم. ومنذ ذلك اليوم، أصبحوا يذهبون في مغامرات جديدة كل يوم، متذكرين دائمًا أن السعادة تكمن في اللعب والضحك مع الأصدقاء.

وهكذا، انتهت قصتهم بسعادة وضحك، وعلموا أن المغامرات الصغيرة يمكن أن تكون أجمل من أي شيء آخر.

BLEURT Score: 0.21849961578845978
Semantic Coherence: 0.8562461137771606
Grammar Errors: 7
Distinct 1-gram Score: 0.7131147540983607
Distinct 2-gram Score: 0.9465020576131687
Perplexity: 1.7796951532363892

Cleaned Story29:
في قرية صغيرة جميلة بالكويت، كان يعيش طفلان صديقان، ياسمين وعمر. كانا يحبان اللعب والاستكشاف في كل مكان حول قريتهما. في يوم مشمس جميل، وجدا لغزًا مكتوبًا على جدار قديم قرب الحديقة.

"من يجد هذا اللغز، سيكون حامله للعصفورة الذهبية التي تحلم بها كل طفل!"، كان اللغز يقرأ.

ياسمين وعمر نظرا إلى بعضهما، وبدا عليهما الحماس. بدأا بالفكر في حل اللغز، لكنهما كانا يحتاجان إلى مساعدة. فقررا أن يطلبا المساعدة من السيد يوسف، الذي كان يعيش في منزل قديم بجوار الحديقة. السيد يوسف كان يبدو قديمًا جدًا، وكان يرتدي ثيابًا قديمة، لكنه كان يحب حل الألغاز.

عندما سألا السيد يوسف عن المساعدة، ابتسم وأجاب: "أنا سعيد بأن أ助كما. لكن، يجب أن تعلما شيئًا مهمًا. لا تحكموا على الناس من مظهرهم، فقد يكونوا مخبئين كنوزًا عظيمة."

بمساعدة السيد يوسف، استطاع ياسمين وعمر حل اللغز. كان اللغز يحتاج إلى فهم عميق للكلمات والرموز. بعد حل اللغز، وجدا العصفورة الذهبية! كانت أجمل ما رأياه من قبل. كانت العصفورة تلمع تحت أشعة الشمس، وكانت تغرد بألحان جميلة.

ياسمين وعمر شعرا بالفخر والسعادة. لكن السيد يوسف كان أكثر سعادة منهم. قال: "أنا سعيد لأنكم تعلمتم درسًا قيمًا اليوم. لا تحكموا على الناس من مظهرهم، فقد يكونوا مخبئين كنوزًا عظيمة."

من ذلك اليوم فصاعدًا، أصبح ياسمين وعمر يعتبران السيد يوسف صديقًا قيمًا، وكانا دائمًا يطلبان نصيحته في حل الألغاز. وأدركا أن الجمال يأتي من داخل الشخص، وأن كل شخص يمكن أن يكون كنزًا.

وهكذا، عاش ياسمين وعمر في سعادة، مع السيد يوسف، الذي أصبح حاميهم في كل مغامرة. وأدركا أن الحياة مليئة بالكنوز، إذا فقط استمتعوا بالاستكشاف والتعلم.

BLEURT Score: 0.25079992413520813
Semantic Coherence: 0.8795078992843628
Grammar Errors: 12
Distinct 1-gram Score: 0.6904761904761905
Distinct 2-gram Score: 0.8844621513944223
Perplexity: 1.9710404872894287

Cleaned Story30:
في قرية صغيرة وجميلة في عمان، كان يعيش ثلاثة أصدقاء: ياسين، سارة، وعمر. كانوا يحبون الاستكشاف والتعلم عن التاريخ والحضارات القديمة. ذات يوم، قرروا الذهاب في مغامرة لاكتشاف كنز قديم يُقال أنه مخبأ في قلب الصحراء.

"هل أنت متأكدون من أن هذا هو الخيار الصحيح؟" سألت سارة بحذر.

"لا شيء يأتي بسهولة، لكن هذا هو مغامرتنا الكبرى!" رد ياسين بثقة.

"أنا معكما، لكن علينا أن نكون حذرين"، قال عمر.

بدأ الأصدقاء الثلاثة رحلتهم تحت شمس الصحراء الحارقة، يتبعون خريطة قديمة وجداها في مكتبة القرية. كانوا يتحدثون عن الحضارات القديمة التي عاشت في هذه الأرض، عن العجائب التي يمكن أن تكون مخبأة في الصحراء.

بعد ساعات من المشي تحت الشمس، بدأوا يشعرون بالتعب والجفاف. لكنهم لم يفقدوا الأمل، فقد كانوا يحلمون بالاكتشاف.

فجأة، بينما كانوا يتجولون حول صخرة كبيرة، وجدوا بابًا صغيرًا مخبأ بين الصخور. كان الباب مفتوحًا، مما جعل قلوبهم تملأ بالفرح والفضول.

"هل هذا...؟" قال ياسين وهو يتساءل.

"لا يمكن أن يكون...؟" أضافت سارة.

"لا شيء يأتي بسهولة، لكن هذا هو ما نبحث عنه!" قال عمر وهو يبتسم.

دخلوا الباب ووجدوا أنفسهم في غرفة صغيرة مليئة بالآثار القديمة. كان هناك لوحات جدارية، أواني فخارية، وأشياء أخرى لا يمكن أن تكون إلا من حضارة قديمة.

"لقد فعلناها! لقد وجدنا الكنز!" صرخ ياسين.

"لكن هذا ليس كنزًا من الذهب أو الجواهر، بل هو كنز من المعرفة والتراث"، قال عمر وهو ينظر إلى الآثار.

"نعم، هذا هو الحقيقة. لا شيء يأتي بسهولة، لكن العمل الجاد والشغف بالتعلم يمكن أن يؤدي إلى اكتشافات عظيمة"، قالت سارة بحكمة.

عاد الأصدقاء الثلاثة إلى القرية، يحملون معهم ليس فقط القصص والأحلام، ولكن أيضًا درسًا قيمًا عن قيمة الشغف بالتعلم والبحث عن المعرفة.

ومنذ ذلك اليوم، أصبحوا معروفين في القرية كأولئك الذين اكتشفوا كنزًا أثمن من الذهب، كنز المعرفة والتراث. وهكذا، عاشوا بسعادة وهدوء، مستكشافين دائمًا، لكن بقلوب مليئة بالامتنان للعالم الذي يعيشون فيه.

BLEURT Score: 0.23702478408813477
Semantic Coherence: 0.7406191825866699
Grammar Errors: 13
Distinct 1-gram Score: 0.734006734006734
Distinct 2-gram Score: 0.9391891891891891
Perplexity: 1.8539193868637085

Cleaned Story31:
في يوم مشمس جميل في دولة قطر، كان هناك ثلاثة أصدقاء صغار: ياسمين، عمر، وسامر. كانوا يحبون اللعب معاً وكل يوم بعد المدرسة، يذهبون في مغامرة جديدة.

ذات يوم، وجد الأصدقاء الثلاثة صندوقًا قديمًا في حديقة المدرسة. كان الصندوق مليئًا بالألغاز والمسابقات. نظروا إلى بعضهم البعض بحماس وقال ياسمين: "هل يمكننا حل هذه الألغاز؟"

عمر، الذي كان دائمًا يحب التحديات، قال: "بالطبع! سنكون أبطال الألغاز!"

سامر، الذي كان الأكثر خوفًا، قال: "لكن ماذا لو كانت الألغاز صعبة جدًا؟"

ياسمين، بابتسامة تشجيعية، قالت: "لا تقلق، سنعمل معًا ونستطيع حلها!"

بدأ الأصدقاء الثلاثة في حل الألغاز، وكان كل حل يؤدي إلى لغز جديد. كانوا يضحكون ويشاركون الأفكار معًا، وكان هذا يجعل المغامرة أكثر متعة.

بعد ساعات من اللعب والتفكير، وصلوا أخيرًا إلى اللغز الأخير. كان هذا اللغز مختلفًا، فلم يكن هناك أي إشارة إلى ما يمكن أن يكون الحل. نظر الأصدقاء إلى بعضهم البعض، ثم نظروا إلى اللغز مرة أخرى.

فجأة، سمعوا صوتًا يأتي من الصندوق القديم. "أهلاً بكم، أبطال الألغاز!" قال الصوت.

"من أنت؟" سأل ياسمين بفضول.

"أنا خالق الألغاز، وأنا سعيد جدًا بكم. لقد تمكنتوا من حل كل لغز، وهذا يعني أنكم أبطال الحديقة!"

فتح الصندوق، وخرج منه الكثير من الجوائز والألعاب. كان الأصدقاء الثلاثة متحمسين جدًا.

لكن ثم حدث شيء غير متوقع. "لكن... لكن هذا ليس كل شيء!" قال عمر.

"ماذا؟" سأل ياسمين.

"أنا خالق الألغاز، وأنا هنا لتعليمكم درسًا. الألغاز كانت خدعة، لكن الدرس الحقيقي هو أن العمل معًا وأن لا يخاف أحد من التحديات."

الأصدقاء الثلاثة نظروا إلى بعضهم البعض، ثم إلى خالق الألغاز. "لقد تعلمنا درسًا قيمًا اليوم"، قال سامر.

"نعم، وأنا فخور بكم!" قال خالق الألغاز.

وهكذا، انتهى اليوم بكل من ياسمين وعمر وسامر يضحكون ويتشاركون القصص عن مغامرتهم المذهلة. وهم عادوا إلى البيت، كانوا يشعرون بالفخر والإثارة من مغامرتهم، وعلموا أن العمل معًا يمكن أن يجلب السعادة والنجاح.

BLEURT Score: 0.2267853170633316
Semantic Coherence: 0.783608078956604
Grammar Errors: 7
Distinct 1-gram Score: 0.7040816326530612
Distinct 2-gram Score: 0.9351535836177475
Perplexity: 1.856684923171997

Cleaned Story32:
في قرية صغيرة بجيبوتي، حيث الأزهار تتفتح في فصل الربيع، كان هناك صبي صغير يُدعى يوسف وصديقه الطائر الأزرق، زيزو. كان يوسف يحب مشاهدة مقاطع الفيديو التعليمية عن التاريخ والحضارات القديمة، وكان يحلم دائمًا بأن يصبح مستكشفًا يكتشف الأسرار القديمة.

ذات يوم، بينما كان يوسف وزيزو يجلسان تحت شجرة النخيل، قال يوسف: "زيزو، هل تعلم أن هناك كنزًا قديمًا يُقال إنهم دفنه في جيبوتي منذ آلاف السنين؟"

رد زيزو بصوته العذب: "كنت أتمنى أن نجد ذلك الكنز، يوسف! لكن كيف سنبدأ؟"

فكر يوسف لحظات ثم قال: "لنستخدم مقاطع الفيديو التعليمية التي أشاهدها لتعلم كيف أصبح مستكشفًا حقيقيًا!"

بدأ يوسف وزيزو رحلتهما، يشاهدان مقاطع الفيديو التعليمية كل يوم، يتعلمان عن كيفية البحث عن الكنوز القديمة، وكيفية قراءة الأهرامات، وكيفية فهم الرموز القديمة.

بعد أسابيع من البحث، وجدا نفسيهما أمام كهف قديم، يبدو أنه قد يكون مكان دفن الكنز. قال يوسف بقلب ممتلئ بالفضول: "زيزو، هل أنت مستعد أن نكتشف ما بداخله؟"

أجاب زيزو بحماس: "بالطبع، يوسف! نحن فعلناه معًا!"

دخل يوسف وزيزو الكهف، وكان هناك صندوق قديم. بدا يوسف متحمسًا جدًا، لكنه توقف فجأة وقال: "زيزو، هل أنت متأكد أن هذا هو الكنز الذي نحن نبحث عنه؟"

فكر زيزو لحظات ثم قال: "يوسف، ربما الكنز الحقيقي لم يكن الذهب أو الجواهر، بل كان الصداقة التي نشاركها."

ابتسم يوسف وقال: "أنت صديقي العزيز، زيزو. لقد كنت أبحث عنك أكثر من أي كنز!"

في تلك اللحظة، كشف يوسف عن صندوق صغير كان يحمله معه. كان فيه مجموعة من الصور والأحجار التي جمعها في رحلتهما معًا. قال يوسف: "هذا هو كنزنا الحقيقي، زيزو. كل هذه الأشياء تذكرنا بمنظراتنا وذكرياتنا معًا."

نظر زيزو إلى يوسف وقال: "يوسف، أنت صديقي الحقيقي. لقد علمتني أن الصداقة هي الكنز الحقيقي."

ومن ذلك اليوم، أصبح يوسف وزيزو أصدقاء لا يفترقان، يشاركان كل مغامرة جديدة معًا، متذكرين دائمًا أن الصداقة هي أعظم كنز يمكن أن يكتشفاه.

BLEURT Score: 0.17161893844604492
Semantic Coherence: 0.8438243269920349
Grammar Errors: 38
Distinct 1-gram Score: 0.6993243243243243
Distinct 2-gram Score: 0.9423728813559322
Perplexity: 1.9253052473068237

Cleaned Story33:
في يوم ربيعي جميل في قطر، كان هناك طفلان صغيران يدعيان يوسف وعمر. كانا يذهبان إلى المدرسة كل يوم، وكانا يحبان اللعب معاً كثيراً.

ذات يوم، أعلنت المدرسة عن يوم سباحة في البحيرة الكبيرة التي كانت تقع في حديقة المدرسة. كان يوسف وعمر متحمسين جداً لأنهما يحبان السباحة كثيراً.

"يا له من يوم جميل! يمكننا السباحة في البحيرة!" قال يوسف بحماس.

"نعم! سأسبح إلى النور!" قال عمر مبتسمًا.

في الصباح، انطلق يوسف وعمر إلى البحيرة. كان الجو مشرقاً والشمس تلمع على الماء. كان يوسف وعمر يضحكان ويلعبون بينما كانوا يسبحون.

لكن فجأة، بدأت السماء تغيم وبدأت الأمطار تهطل. "يا له من عاصفة! يجب أن نعود إلى المدرسة!" قال يوسف بقلق.

"لكننا لم نكتمل في السباحة!" قال عمر وهو يبكي.

"لا بأس يا عمر، يمكننا السباحة مرة أخرى في أيام أخرى. الآن يجب أن نعود إلى المدرسة." قال يوسف وهو يحاول أن يبدو متحمساً.

عاد يوسف وعمر إلى المدرسة، لكن قلوبهما كانت حزينة. كانا يأملان أن يكون يومهما أفضل.

"أنا آسف يا عمر، لم أكن أتوقع أن تكون الأمطار كثيفة جداً." قال يوسف وهو يضع يده على كتف عمر.

"لا بأس يا يوسف، أنا سعيد لأننا لاحظنا جمال البحيرة." قال عمر وهو يبتسم.

"نعم، يجب أن نعود إلى البحيرة في أيام أخرى." قال يوسف وهو يضحك.

وهكذا، عاد يوسف وعمر إلى المدرسة، وهم يحملان معهما ذكرى يوم ربيعي جميل، حتى لو كان يومهما لم يكن كما توقعا.

BLEURT Score: 0.19330668449401855
Semantic Coherence: 0.8811728358268738
Grammar Errors: 21
Distinct 1-gram Score: 0.6194690265486725
Distinct 2-gram Score: 0.8577777777777778
Perplexity: 1.6319550275802612

Cleaned Story34:
في أحد الأيام البعيدة، في فضاء خارجي مليء بالنجوم والألوان الزاهية، كان هناك ثلاثة أصدقاء صغار يعيشون في كوكب صغير يُدعى "زهرة الفضاء". كان الأصدقاء هم: ياسين، الطائر الفضائي الذي كان يملك أجنحة ملونة بألوان الفضاء، وسمر، الفيل الفضائي الذي كان يحب اللعب والضحك، وليلى، النجمة الصغيرة التي كانت تلمع مثل النجوم في السماء.

كانت زهرة الفضاء كوكبًا جميلًا، حيث كانت الأرض مليئة بالأزهار الملونة والحيوانات الفضائية التي تعيش في سلام ووئام. كان الأصدقاء الثلاثة يلعبون ويتنافسون في سباقات الفضاء كل يوم، وكانوا يحبون أن يشاركوا قصصهم والأحلام مع بعضهم البعض.

في يوم ما، بينما كانوا يلعبون قرب الحافة الخارجية للكوكب، لاحظوا شيئًا غريبًا. كان هناك كوكب آخر يبدو وكأنه يحتضر. كان الأصدقاء يهتمون بكل كوكب في الفضاء، وكانوا يحاولون مساعدة الكواكب الأخرى التي تحتاج المساعدة.

قرروا أن يذهبوا لإنقاذ هذا الكوكب. استخدموا قوتهم الفضائية لإنشاء سفينة فضائية صغيرة، وبدأوا رحلتهم نحو الكوكب المضطر. كانت رحلتهم طويلة ومخيفة، لكنهم لم يفقدوا الأمل.

عند وصولهم، وجدوا أن الكوكب كان يحتضر بسبب انزلاقات في مدارته. حاولوا بكل ما أوتوا من قوة أن يصلحوا المشكلة، لكن دون جدوى. كان الكوكب يختفي أمام عيونهم.

عاد الأصدقاء إلى زهرة الفضاء، حزينين جدًا. فقدوا صديقًا في الفضاء، كوكبًا كان يحتضنهم ويعيش معهم. قرروا أن يذكروا الكوكب المضطر دائمًا، وأن يحافظوا على سلامة زهرة الفضاء.

منذ ذلك اليوم، أصبح ياسين، سمر وليلى أكثر حرصًا على حماية كوكبهم، وعلموا أن هناك دائماً مسؤولية أكبر من أنفسهم في الحفاظ على سلامة الفضاء. ومع ذلك، كان هناك حزن في قلوبهم، فقد فقدوا صديقًا في رحلة حياتهم الفضائية.

BLEURT Score: 0.20271800458431244
Semantic Coherence: 0.8675889372825623
Grammar Errors: 7
Distinct 1-gram Score: 0.7327935222672065
Distinct 2-gram Score: 0.9552845528455285
Perplexity: 1.7864781618118286

Cleaned Story35:
في قرية صغيرة جميلة في جيبوتي، حيث الأزهار تتفتح في فصل الربيع، كان يعيش صبي صغير يُدعى يوسف مع صديقه الأفضل، الأرنب رامي. كان يوسف ورامي أفضل الأصدقاء، يلعبون معًا كل يوم تحت أشجار النخيل العالية، ويشاركان كل سر وكل حلم.

ذات يوم، وجد يوسف ورامي كنزًا صغيرًا تحت شجرة النخيل. كان الكنز عبارة عن مجموعة من العملات الذهبية والجواهر. شعر يوسف برغبة قوية في الاحتفاظ بالكنز لنفسه، لكنه كان يدرك أن رامي كان صديقه الأفضل.

فكر يوسف طويلاً وقرر أن يخبر رامي بكل الحقيقة. "رامي، أنا وجد كنزًا تحت شجرة النخيل. يجب أن نعرف ماذا نفعل به." قال يوسف بحذر.

رامي نظر إليه بفضول وقال: "كنز؟ هل يمكننا أن نستخدمه لتحسين حياتنا في القرية؟"

"نعم، لكن يجب أن نكون صادقين مع القرية. لا يمكننا الاحتفاظ به لنفسي دون أن أشارك معك." قال يوسف.

قرر يوسف ورامي أن يشارك الكنز مع القرية، لكنهما كانا يخافان من أن يرفضوا الكنز. فكر يوسف ورامي طويلاً ووضع خطة.

في اليوم التالي، أعلن يوسف ورامي عن الكنز في وسط القرية. كان الجميع متحمسين ومفاجئين. بدلاً من رفض الكنز، قرر القرية أن يوسف ورامي يستخدما الكنز لتحسين القرية، مثل بناء مدرسة جديدة ومركز صحي.

كانت القرية سعيدة جدًا، وأصبح يوسف ورامي أبطالاً في القرية. ولكن، كان هناك تحول غير متوقع في القصة.

في تلك الليلة، عندما كان يوسف ينام، سمع صوتًا يأتي من وراءه. "يوسف، هل أنت مستعد للعصفورة؟" قال الصوت.

"من أنت؟" سأل يوسف بقلق.

"أنا مخرج مسرحية. لقد كنت جزءًا من تمثيلية عن الصدق والصداقة. كل ما حدث كان جزءًا من المسرحية."

"لكن... لكن ماذا عن الكنز؟" قال يوسف.

"كل شيء كان جزءًا من المسرحية، حتى الكنز. لكن الدرس الحقيقي الذي تعلمته هو أهمية الصدق والصداقة." قال المخرج.

"لقد تعلمت درسًا قيمًا اليوم. شكرًا لك." قال يوسف.

"الدرس الحقيقي هو أن الصدق والصداقة هما الكنز الحقيقي. وبالصدق، يمكننا أن نصنع أجمل الأوقات في الحياة." قال المخرج.

وبهذا، انتهت القصة بسعادة، حيث تعلم يوسف ورامي أن الصدق والصداقة هما أهم كنزات في الحياة.

BLEURT Score: 0.23488456010818481
Semantic Coherence: 0.8055259585380554
Grammar Errors: 30
Distinct 1-gram Score: 0.64576802507837
Distinct 2-gram Score: 0.9276729559748428
Perplexity: 1.8170217275619507

Cleaned Story36:
في محطة الفضاء العجيبة التي تقف في سماء قطر اللامعة، كان هناك ثلاثة أصدقاء: زين، سارة، وعلي. كانوا يزورون المحطة كل يوم بعد المدرسة، لاستكشاف كل شيء جديد ومتحول. كانوا يحبون التكنولوجيا والمستقبل كثيرًا، وكانوا دائمًا يبحثون عن مغامرات جديدة.

ذات يوم، وجدوا جهازًا غريبًا لم يرووه من قبل. كان الجهاز يبدو كأنه يمكنه التحكم في كل شيء في المحطة. كان علي متحمسًا جدًا وقرر أن يفعل تجربة. لكنه لم يفكر قبل أن يفعل، فلم يكن يعرف كيف يعمل الجهاز.

فجأة، بدأت الأجهزة في المحطة تتألم وتتوقف عن العمل. كان الأصدقاء في صدمة. زين، الذي كان دائمًا هادئًا، شعر بالغضب لأن علي لم يكن حذرًا. سارة، التي كانت دائماً تساعد الآخرين، حاولت أن تهدئ زين.

في هذه اللحظة، كان الأصدقاء في موقف ميؤوس منه. لكن علي، الذي كان يشعر بالندم على ما فعل، أراد أن يجد طريقة لمساعدة أصدقائه. فكر بسرعة ووجد حلًا مبتكرًا. استخدم الجهاز الذي أفسده لاستعادة الأجهزة التي توقفت.

بعد لحظات، كانت المحطة تعمل مرة أخرى. الأصدقاء كانوا في السعادة، لكن زين كان لا يزال حزينًا. "لقد كان علي حذرًا جدًا،" قال زين. "أنا آسف لأنني غضبت."

سارة، التي كانت تعلم أن الغضب يمكن أن يترك ندوبًا، قالت: "يجب أن نكون حذرين ونفهم بعضنا البعض. الغضب يمكن أن يؤدي إلى أخطاء كبيرة."

علي، الذي كان يشعر بالندم، قال: "أنا آسف جدًا. سأكون أكثر حذرًا في المستقبل."

وهكذا، تعلم الأصدقاء درسًا قيمًا عن الغضب والهدوء. وهم يعودون إلى الأرض، كانوا يحملون معهم تجربة لا تُنسى في محطة الفضاء، ولكنهم أيضًا حملوا معهم درسًا قيمًا عن أهمية الحذر والهدوء.

BLEURT Score: 0.25411921739578247
Semantic Coherence: 0.8772202134132385
Grammar Errors: 7
Distinct 1-gram Score: 0.688
Distinct 2-gram Score: 0.9357429718875502
Perplexity: 1.841841220855713

Cleaned Story37:
في قرية صغيرة جميلة في ليبيا، كان يعيش طفلان صديقان لا يفترقان، ياسمين وطارق. كانا يحبان اللعب معاً تحت أشجار النخيل الكثيفة، وكانا يشاركان كل شيء معاً.

ذات يوم، بينما كانا يلعبان بالقرب من النهر، لاحظا أن المياه بدأت تزداد قوة وسرعة. فجأة، بدأت الأمطار تهطل بغزارة، مما أدى إلى حدوث انزلاقات في الأرض.

ياسمين وطارق، بقلوبهما المليئة بالقلق، رأيا قريتهم تهدد بالدمار بسبب انزلاقات الأرض. بدون تفكير، قررا أن يساعدوا قريتهم.

بدأا بالجمع بين القوى، ياسمين بذكائها وطارق بقوته. عملا معاً لجمع الأشياء التي يمكن أن تساعد في إعادة بناء ما تضرر من البيوت.

بعد ساعات من العمل الشاق، بدأ الجو يتساقط، وبدأت الأمطار تتوقف. كانت قريتهم قد نجت، ولكن كان هناك الكثير من الأعمال التي يجب القيام بها.

ياسمين وطارق، بقلوبهما المليئة بالفرح والامتنان، عملا معاً مع أهل القرية لاستعادة ما تضرر. وبفضل جهودهم الشجاعة والعمل الجماعي، تمكنوا من إعادة بناء البيوت والأماكن المضررة.

في نهاية اليوم، جلس ياسمين وطارق تحت شجرة النخيل، ينظران إلى قريتهم الجميلة التي عادت إلى الحياة. كانا يشعران بالفخر والإنجاز، مدركين أن قوة الوحدة قد ساعدتهم على تحقيق هذا الإنجاز العظيم.

"لقد علمتنا اليوم أن هناك قوة في الوحدة،" قال طارق بحماس.

"نعم،" أضافت ياسمين بابتسامة، "وأنا سعيدة جدًا لأننا عملنا معاً وننجحنا."

ومنذ ذلك اليوم، أصبح ياسمين وطارق مثالًا للشجاعة والعمل الجماعي في قريتهم. وأدرك كل من في القرية قوة الوحدة التي يمكن أن تحدث عندما يعمل الأفراد معًا نحو هدف مشترك.

BLEURT Score: 0.23606961965560913
Semantic Coherence: 0.8852691054344177
Grammar Errors: 12
Distinct 1-gram Score: 0.7304347826086957
Distinct 2-gram Score: 0.9606986899563319
Perplexity: 1.8615583181381226

Cleaned Story38:
في جزر القمر الجميلة، كان هناك مخبز صغير يقع في قلب القرية. كان المخبز ملاذاً للأطفال الذين يحبون المغامرات والاستكشاف. كان هناك ثلاثة أصدقاء: يوسف، ليلى، وعمر. كانوا يحبون ركوب الدراجات حول القرية، وكانوا دائماً يبحثون عن مغامرات جديدة.

ذات يوم، قرروا أن يذهبوا في مغامرة جديدة. قرروا أن يركبوا دراجاتهم إلى المخبز الصغير ليجربوا الخبز الطازج. كانوا متحمسين جدًا، لكنهم لم يكنوا يعلمون ما انتظروه.

عندما وصلوا إلى المخبز، وجدوا البخار يتصاعد من الفرن. كان هناك رائحة خبز رائعة تملأ المكان. لكن، فجأة، ظهرت شخصية غريبة. كان هناك رجل يرتدي ملابس مخبزة، وكان يبتسم بابتسامة واسعة.

"مرحباً يا أصدقاء!" قال الرجل. "أنا خباز القرية. أنا سعيد جدًا بزيارتكم."

"نحن هنا لجربة الخبز الطازج!" قال يوسف بحماس.

"لكن، قبل أن تجربوا الخبز، يجب أن تعلموا درسًا مهمًا." قال الخباز.

"درس ما هو؟" سأل ليلى بحماس.

"الدرس هو الحب. الحب هو ما يجعل كل شيء جميل. الحب للآخرين، الحب للمغامرات، والحب للعيش."

فجأة، بدأت الأشياء تتغير. بدأت الأصوات تخرج من المخبز كأنها موسيقى. كان كل شيء يبدو كأنهم في مسرحية!

"أه، أتيتنا إلى هنا لتعلمنا درس الحب!" قال عمر بفرح.

"نعم، يا أصدقاء. الحب هو ما يجعل كل شيء جميل. وأنا سعيد جدًا لأنكم تعلمتم هذا الدرس اليوم."

وبهذا، انتهت مغامرتهم. عاد الأصدقاء إلى منازلهم، يحملون معهم ليس فقط خبزًا طازجًا، ولكن أيضًا درسًا قيمًا في الحياة.

ومنذ ذلك اليوم، كان يوسف، ليلى، وعمر يحبون الحب في قلوبهم. وكانوا دائماً يحبون المغامرات والاستكشاف، ولكنهم أدركوا أن الحب هو ما يجعل كل شيء جميل.

BLEURT Score: 0.2594537138938904
Semantic Coherence: 0.8736574053764343
Grammar Errors: 12
Distinct 1-gram Score: 0.7107438016528925
Distinct 2-gram Score: 0.9128630705394191
Perplexity: 1.7150146961212158

Cleaned Story39:
في قرية صغيرة في قطر، كان يعيش صبي يُدعى يوسف مع عائلته. يوسف كان يحب العزف على العود، الآلة الموسيقية القديمة التي كانت تُستخدم في العصور القديمة. كان يوسف يحلم بأن يصبح عازفًا مشهورًا، لكنه كان يخشى أن يفقد صوته بسبب العزف الشديد.

كل يوم بعد المدرسة، كان يوسف يجلس تحت شجرة النخيل الكبيرة في حديقة منزله، يضع يده على العود، ويبدأ بالعزف. كان صوته يملأ الهواء، يغني الألحان الجميلة التي كانت تملأ قلبه بالفرح. كان يوسف يحلم بأن يشارك موسيقاه مع الجميع، ليجعل الناس سعيدون.

ذات يوم، أعلن المدرسة عن مسابقة موسيقية للطلاب. كان يوسف متحمسًا جدًا، لكنه كان يخشى أن يفقد صوته. قرر يوسف أن يشارك في المسابقة، وبدأ في التدريب بجد، محاولًا الحفاظ على صوته.

جاء يوم المسابقة، ووضع يوسف يده على العود، وبدأ بالعزف. كانت الألحان تملأ المكان، وكانت قلوب الجميع تملأ بالفرح. كان يوسف يؤدي بكل حب وواقعية، وكان يرى في قلوب الجمهور.

لكن بعد أن انتهى يوسف من العزف، شعر بشيء غريب في صوته. كان صوته يبدو مختلفًا، كأنه قد تغير. كان يوسف حزينًا جدًا، فقد كان يخشى أن يفقد صوته بسبب العزف الشديد.

بعد المسابقة، ذهب يوسف إلى المنزل، وهمس بصديقه العزيز، "لقد حلمت بأن أصبح عازفًا مشهورًا، لكني خافت أن أفقد صوتي".

كانت النهاية حزينة، فقد كانت حلم يوسف قد تحول إلى حقيقة، لكنه لم يكن سعيدًا. كان يوسف يدرك أن الموسيقى كانت دائمًا حبه، وأن الحزن كان جزءًا من رحلته.

وبهذا، انتهت قصة يوسف، الصبي الذي كان يحلم بأن يصبح عازفًا مشهورًا، لكنه تعلم أن الحب للموسيقى كان أثمن من أي جائزة.

BLEURT Score: 0.24168381094932556
Semantic Coherence: 0.8787590265274048
Grammar Errors: 16
Distinct 1-gram Score: 0.5976095617529881
Distinct 2-gram Score: 0.876
Perplexity: 1.683920979499817

Cleaned Story40:
في يوم ربيعي جميل بالمملكة العربية السعودية، كان هناك مختبر علمي صغير في أحد مدارس الرياض. كان هناك صديقان صغيران، ياسمين وعمر، يحبون الرياضة والألعاب كثيرًا. كانا يشاركان حبهم للعلم معًا، وكانا يحلمان بأن يصبحا الرياضيين الشباب الفائزين في المستقبل.

في ذلك اليوم، قرر ياسمين وعمر تجربة شيء جديد في المختبر. كانا يريدون أن يصنعان كرة صغيرة يمكن أن تتميز بأنها تنتقل بسرعة أكبر من الكرات العادية. كانا يعتقدان أن هذا يمكن أن يساعد في تحسين مهاراتهم الرياضية.

"أعتقد أننا يمكن أن نصنع كرة تنتقل بسرعة أكبر إذا أضفنا بعض المواد الخاصة بها!" قال عمر بحماس.

"لكن ماذا إذا لم تنجح التجربة؟" سألت ياسمين بقلق.

"لا تقلق، سنفعلها معًا وسنجد طريقة!" رد عمر.

بدأ الصديقان العمل على الكرة الجديدة، يجمعان المواد ويتجربان معًا. كانا يشاركان الضحكات والمرح، وكان كل شيء يبدو مثيرًا وممتعًا.

لكن، فجأة، حدث شيء غير متوقع. عندما أضاف عمر بعض المادة التي لم يكن يعرفها جيدًا، انفجرت الكرة فجأة، مما أدى إلى إصابة ياسمين في عينها.

"عمر، ماذا فعلت؟!" صرخت ياسمين وهي تدمع من الألم.

"أنا آسف، لم أكن أعلم... أنا آسف!" قال عمر وهو يبكي.

"لكن كيف يمكن أن أكون الآن غير قادرة على اللعب معك؟" قالت ياسمين بحزن.

"أنا آسف، لا أريد أن أفقدها. يمكننا أن نجرب مرة أخرى، يمكننا..." بدأ عمر، لكن ياسمين كانت قد غادرت المختبر، تاركة عمر وحيدًا.

في تلك الليلة، شعر عمر بالحزن الشديد. لم يكن يريد أن يفقده صديقته. كان يدرك أن الأخطاء يمكن أن تحدث، لكنه لم يكن يريد أن يؤديها إلى فقدان صديقته.

"أنا آسف، يا ياسمين. أتمنى لو كنت هناك لأساعدك. أتمنى لو..." قال عمر وهو يبكي.

ومع ذلك، كانت نهاية القصة حزينة. ياسمين كانت قد تعرضت للعين، ولم تعد قادرة على اللعب مع عمر كما كانا يفعلون. لكن عمر لم يفقد الأمل. أصبح أكثر حكمة وأكثر شجاعة، وعلم أن الصداقة الحقيقية لا تتوقف عندها، حتى وإن كانت هناك تحديات.

"أنا سأكون دائمًا صديقك، يا ياسمين. حتى لو لم نلعب معًا،" قال عمر وهو يبكي.

وهكذا، على الرغم من الحزن الذي شعر به عمر، تعلم أن الصداقة الحقيقية يمكن أن تتحمل أي شيء، حتى الألم والخسارة.

BLEURT Score: 0.2284523993730545
Semantic Coherence: 0.7366863489151001
Grammar Errors: 8
Distinct 1-gram Score: 0.6696165191740413
Distinct 2-gram Score: 0.9142011834319527
Perplexity: 2.019596576690674

Cleaned Story41:
في يوم مشرق وجميل، كان هناك طفل صغير يُدعى يامن يعيش في فلسطين. يامن كان يحب العلوم والخيال العلمي كثيرًا، وكان دائمًا يحلم بأن يصبح مستكشفًا في الفضاء.

ذات يوم، بينما كان يامن يجلس تحت شجرة زيتون في حديقة منزله، وجد حقيبة صغيرة غامضة. كانت الحقيبة مليئة بملفات وأدوات غريبة. بينما كان يامن يفكر في ما يمكن أن تكون هذه الحقيبة، ظهرت أمام عينيها قمر صغير يبدو وكأنه يتنقل في السماء.

فجأة، وجد نفسه على متن قمر صغير يتنقل في الفضاء. كان يامن مذهولًا جدًا، لكنه لم يفقد الأمل. بدأ يامن يستكشف القمر الصغير، ووجد نفسه في محطة فضاء صغيرة. كانت هناك طاولة صغيرة، ومرحاض صغير، ورفيف صغير.

فكر يامن، "لن أكون مستكشفًا في الفضاء، لكن يمكنني أن أكون طاهيًا في الفضاء!" فجأة، بدأ يامن يجد وصفات بسيطة على جدران المحطة الفضائية. كان هناك وصفة لسلطة الفواكه، ووصفة لخبز البن، ووصفة لسلطة الجزر.

بدأ يامن يطبخ، لكنه وجد صعوبة في تحضير الوصفات بسبب عدم وجود أدوات طهي. فكر يامن، "لن أستسلم!" فجأة، بدأ يامن يستخدم الأدوات الغريبة التي وجدها في الحقيبة. استخدم الأدوات كأدوات طهي، وأصبح يامن طاهيًا في الفضاء.

بعد فترة قصيرة، أصبح الطعام على المحطة الفضائية رائعًا. كان يامن سعيدًا جدًا، لكنه لم يفهم أن الحدث الرئيسي كان خدعة.

فجأة، ظهرت أمام عيني يامن شخصية غامضة. كانت الشخصية تُدعى "المرشد الفضائي". قال المرشد، "يامن، لقد كنت دائمًا مستعدًا ليكون مستكشفًا في الفضاء. لكن، لقد علمتك درسًا مهمًا عن الإصرار والخيال. الآن، يمكنك أن تعود إلى الأرض، لكن تذكر، يمكنك أن تكون طاهيًا في الفضاء في أي وقت."

عاد يامن إلى الأرض، لكنه لم ينسى درسًا مهمًا عن الإصرار والخيال. ومن ذلك اليوم فصاعدًا، أصبح يامن طاهيًا في الفضاء في كل مرة يفكر فيها فيها.

وهكذا، انتهت قصة يامن، الطفل الصغير الذي أصبح طاهيًا في الفضاء. كانت القصة مليئة بالخيال العلمي والخيالي، وأظهرت أن الإصرار والخيال يمكن أن يجعل أي شيء ممكن.

BLEURT Score: 0.18909935653209686
Semantic Coherence: 0.8888118267059326
Grammar Errors: 11
Distinct 1-gram Score: 0.6204620462046204
Distinct 2-gram Score: 0.8708609271523179
Perplexity: 1.885741114616394

Cleaned Story42:
في قرية صغيرة بالسودان، كان يعيش صبي يُدعى يوسف مع جدته العزيزة. يوسف كان يحب البستنة كثيرًا وكان يحلم بأن يصبح أفضل حارس حديقة في القرية. كل يوم بعد المدرسة، كان يوسف يجلب الماء والتربة ويزرع بذور جديدة، يحلم بأن يرى نباتاته تنمو وتنبت.

ذات يوم، قرر يوسف أن يشارك حديقته الجديدة في مسابقة البستنة التي ستقام في القرية. لكن كان هناك مشكلة واحدة؛ لم يكن يوسف يمتلك الكثير من النباتات المزهرة التي كانت مطلوبة في المسابقة. فقرر يوسف أن يخبر الجيران بأن لديه الكثير من النباتات المزهرة، لكنه كان يخطط لشراء النباتات المطلوبة من المدينة.

في يوم المسابقة، كان الجميع يشاهدون الحديقة. يوسف كان يشعر بالقلق الشديد، لكنه لم يكن يريد أن يخسر الثقة التي أبداها له الجيران. فجأة، ظهر رجل غريب يبدو عليه الحزن. كان هذا الرجل يبدو وكأنه يراقب يوسف من بعيد. وبعد لحظات، ظهرت عيناه تلمع بالدموع.

"أنت يوسف،" قال الرجل بصوت حزين. "لقد كنت أتمنى أن أرى نباتاتك تنمو، لكني أعتذر لأنني لم أكن هنا لأساعدك."

يوسف كان صدمةً جدًا. "من أنت؟" سأل يوسف.

"أنا والدك،" قال الرجل وهو يبكي. "لقد غادرت القرية عندما كنت طفلاً ولم أكن أتوقع أن أرى يومًا ما يوسف يزرع نباتات ويكون حارس حديقة."

يوسف كان في صدمة، لكنه لم يكن يريد أن يخسر الوقت. "لقد كنت أتمنى لو كنت هنا لأساعدك،" قال يوسف.

"أنا آسف، يا يوسف. لم أكن أعلم أنك ستكون هنا اليوم. لكن الآن، سأكون هنا دائمًا." قال الرجل وهو يبكي.

يوسف شعر بالراحة عندما رأى والده يعود. لكنه كان يدرك أن الكذب قد يكسر الثقة. فقرر أن يخبر الجيران الحقيقيين عن ما حدث.

في النهاية، فاز يوسف بالمسابقة ليس بسبب الكذب، بل بسبب الحب والجهد الذي بذله. والده، الذي عاد إلى القرية، أصبح يوسف يشاركه حبه للبستنة. وأصبح الجيران يعتبرون والده يوسف رجلًا شجاعًا جدًا لأنه عاد ليعيش مع ابنه.

وهكذا، تعلم يوسف درسًا قيمًا: الكذب قد يكسر الثقة، لكن الحقيقة والصدق يمكن أن يصلحا كل شيء.

BLEURT Score: 0.23483702540397644
Semantic Coherence: 0.8749327063560486
Grammar Errors: 23
Distinct 1-gram Score: 0.6559485530546624
Distinct 2-gram Score: 0.9096774193548387
Perplexity: 1.8529771566390991

Cleaned Story43:
في قرية صغيرة بالبحرين، كان يعيش ثلاثة أصدقاء: ياسمين، أحمد، وعمر. كانوا يحبون اللعب معًا تحت أشعة الشمس الدافئة، وكانوا يشاركون كل شيء معًا.

ذات يوم، قرر الأصدقاء الثلاثة أن يبنوا قارب صغير من الأخشاب والجوت. كانوا متحمسين جدًا، وكانوا يأملون أن يكون القارب قويًا بما يكفي لاستكشاف المياه حول القرية.

بينما كانوا يعملون، لاحظ ياسمين أن أحمد وعمر كانا يبدوان قلقين. سألت ياسمين: "ما بكما؟ هل هناك شيء يقلقكما؟" أجاب أحمد بصوت خافت: "لقد أردنا أن نصنع قاربًا قويًا، لكننا لا نريد أن نغرق."

ياسمين، التي كانت تحب البحر كثيرًا، شعرت بالقلق أيضًا. لكنها لم تريد أن تخيف أصدقائها، فقررت أن تعلم كيفية بناء قارب قوي قبل أن يبدأوا العمل.

بعد مرور بعض الوقت، انتهى الأصدقاء من بناء القارب. كان الجميع متحمسًا جدًا لمعاينة القارب الجديد. لكن عندما حاولوا تجريبه في البحر، انكسر القارب فجأة.

كانت النظرة على وجوه الأصدقاء حزينة. أحمد وعمر كانا حزينين لأنهم لم يستطع بناء قارب قوي. ياسمين، التي كانت تحب البحر كثيرًا، شعرت بالحزن لأنهم لم يستطيعوا استكشاف المياه حول القرية.

في تلك اللحظة، تعلم الأصدقاء درسًا قاسيًا: "اتق شر من أحسنت إليه". لقد كان أحمد وعمر يخافان من أن يغرقوا، لكنهم لم يشاركا هذا القلق مع ياسمين، مما أدى إلى بناء قارب ضعيف.

الدرس المستفاد من هذه القصة هو أن الأصدقاء يجب أن يكونوا صادقين مع بعضهم البعض، وأن يشاركون كل شيء معًا. إذا كانوا صادقين، يمكنهم بناء صداقات أقوى ومتعة أكثر في استكشاف البحر.

ومع ذلك، كانت النهاية حزينة لأن القارب الذي بنوه لم يكن قويًا بما يكفي. لكن الأصدقاء تعلموا درسًا قيمًا عن أهمية الصداقة والثقة.

BLEURT Score: 0.23008081316947937
Semantic Coherence: 0.8828525543212891
Grammar Errors: 4
Distinct 1-gram Score: 0.7058823529411765
Distinct 2-gram Score: 0.9291338582677166
Perplexity: 1.872934341430664

Cleaned Story44:
في قرية صغيرة بالجزائر، كان يعيش ثلاثة أصدقاء لا يفترقون: ياسين، والغزال زينب، والسلحفاة سامي. كانوا يحبون اللعب معًا تحت أشجار الخريف الذهبية، يضحكون ويتنافسون في ألعابهم.

ذات يوم، قرر الأصدقاء الثلاثة أن يخوضوا مغامرة جديدة. قال ياسين بحماس: "لنذهب في رحلة إلى الغابة لاستكشاف كنز الخريف!".

زينب الغزال، بقلبها الكبير، قالت: "سأحمل معي حقيبة الكنوز!"، بينما سامي السلحفاة، ببطءها، أجاب: "سأحمل معي حقيبة الكنوز أيضًا، لكني سأحتاج إلى بعض الوقت لأخذها!"

بدأوا رحلتهم، يضحكون ويتنافسون، لكنهم سرعان ما وجدوا أنفسهم في مشكلة. زينب، التي كانت تحمل الحقيبة الكبيرة، ضاعت في الطريق، مما جعلهم يبحثون عنها. سامي، الذي كان يركض ببطء، ضحك وقال: "أظن أنني سأكون أسرع في العودة إلى القرية!".

بينما ياسين يبحث عن زينب، وجد نفسه في موقف مضحك. حاول أن يركض مثل زينب، لكن قدميه صغيرة جدًا، مما جعله يركض مثل الفيل. ضحك ياسين وقال: "أظن أنني أبدًا لن أكون رافعًا!"

بعد ساعات من البحث، وجد ياسين زينب وهم يلعبان مع بعض الحيوانات الأخرى. عادوا إلى القرية معًا، لكنهم كانوا حزينين لأنهم لم يجدوا الكنز.

عندما وصلوا إلى القرية، وجدوا سامي ينتظرهم ببطء. قال سامي: "أنا آسف، لكني لم أتمكن من العودة في الوقت المناسب."

وقف الأصدقاء الثلاثة معًا، ينظرون إلى السماء، وهم يدركون أن الكنز الحقيقي كان في صداقتهم، وليس في الغابة. لكن الحزن كان يخيم على قلوبهم لأنهم لم يشاركوا هذه المغامرة معًا.

"ربما لن نجد الكنز في الغابة،" قال ياسين بصوت حزين، "لكننا كنا أصدقاء."

"نعم،" قالت زينب، "الكنز الحقيقي هو الوقت الذي قضيناه معًا."

"وأنا آسف لأنني لم أتمكن من العودة في الوقت المناسب،" قال سامي.

بقي الأصدقاء الثلاثة يتحدثون ويتناقشون، يدركون أن الكنز الحقيقي كان في صداقتهم، وأنهم سيبقون أصدقاء إلى الأبد، حتى لو لم يجدوا الكنز في الغابة.

وهكذا، انتهت مغامرة ياسين وزينب وسامي، مغامرة مليئة بالضحكات والمواقف المضحكة، لكنها انتهت بحزن بسبب عدم مشاركة الكنز. لكنهم تعلموا درسًا قيمًا: أن الصداقة هي الكنز الحقيقي.

BLEURT Score: 0.17244035005569458
Semantic Coherence: 0.8806145787239075
Grammar Errors: 3
Distinct 1-gram Score: 0.6710526315789473
Distinct 2-gram Score: 0.9240924092409241
Perplexity: 1.822092056274414

Cleaned Story45:
في جزيرة صغيرة جميلة في الإمارات العربية المتحدة، كان يعيش طائر صغير اسمه زيزو وفراشة جميلة تدعى لولو. كانا أفضل الأصدقاء وكانا يلعبون معًا كل يوم تحت أشعة الشمس الدافئة.

ذات يوم، وجد زيزو ولو لؤلؤة ذهبية جميلة على الشاطئ. كانا متحمسين جدًا، لكن لولو كانت حذرة قائلة: "يجب أن نكون حذرين، الطمع قد يقودنا إلى السقوط".

لكن زيزو، الذي كان يعتبر اللؤلؤة كنزًا عظيمًا، قرر أن يأخذها لنفسه. بدأ يغني أغانٍ جميلة ويعرض اللؤلؤة للعوام، مما جعل الناس يأتون من كل مكان ليرى اللؤلؤة.

مع مرور الوقت، بدأت لولو تشعر بالحزن. كانت ترى كيف يفقد زيزو أصدقائه بسبب الطمع، وكان يصبح أكثر وحيدًا. في أحد الأيام، بينما كان زيزو يغني، انزلقت اللؤلؤة من بين أصابعه وغمرت المياه.

نظر زيزو إلى الماء، وهو يرى اللؤلؤة تختفي تحت سطح الماء. شعر بالحزن الشديد، واعترف أنه كان خاطئًا. لولو، التي كانت تراقب من بعيد، قالت: "أنا آسفة، زيزو. الطمع قد يبدو جذابًا في البداية، لكنه يمكن أن يؤدي إلى فقدان كل شيء جميل".

منذ ذلك اليوم، تعلم زيزو درسًا قيمًا. أدرك أن الأصدقاء والمرح أكثر قيمة من أي كنز. وبينما كانا يعودان إلى بيتهمما، كانا يعدان كل يوم جميلًا، معًا، دون اللجوء إلى الطمع.

وهكذا، على تلك الجزيرة الصغيرة، تعلم زيزو ولولو أن الطمع قد يقود الشخص إلى السقوط، لكن الصداقة والمرح يمكن أن يجلبان السعادة الحقيقية.

BLEURT Score: 0.2521020472049713
Semantic Coherence: 0.8620532155036926
Grammar Errors: 24
Distinct 1-gram Score: 0.7488372093023256
Distinct 2-gram Score: 0.9766355140186916
Perplexity: 1.8122878074645996

Cleaned Story46:
في قرية صغيرة بمغرب جميل، كان يعيش قط صغير يُدعى "موسى". كان موسى يعيش وحيداً في حديقة كبيرة مليئة بالأشجار والزهور، وكان يحب اللعب والاستكشاف في كل مكان.

ذات يوم، في صيف حار، وجد موسى حبلاً صغيراً من الفواكه المزهرة. كان الفواكه لذيذة جداً وجميلة، لكن موسى لم يعرف ما هو. قرر موسى أن يأكل الفواكه مباشرة دون التفكير في أي شيء آخر.

بعد تناول الفواكه، شعر موسى بالغثيان والضعف. بدأ يشعر بالحزن لأن كل يوم كان يصبح أصعب من اليوم الماضي. كان يفكر في أن كل شيء سيصبح أسوأ من ذي قبل.

لكن، بعد مرور بعض الوقت، بدأ موسى يشعر بالتحسن. أصبح قوياً وحيوياً أكثر من أي وقت مضى. كان يلعب واليوم يصبح أجمل من اليوم الماضي.

في النهاية، اكتشف موسى أن الفواكه كانت من نوع خاص ينمو فقط في فصل الصيف، وكان يحتاج إلى وقت لتحتاجه. كان هذا درساً قيماً تعلمه موسى، وهو أن لا يجب أن تعتمد على شيء حتى تكون متأكداً من حدوثه.

ومنذ ذلك اليوم، أصبح موسى أكثر حكمة، وكان دائماً يحاول أن يفكر قبل أن يفعل. وأصبح حديقته المليئة بالأشجار والزهور مكاناً سعيداً حيث يمكنه الاستمتاع باللعب والاستكشاف دون أن يخاف من شيء.

وهكذا، عاش موسى في سعادة وبهجة، مدركاً أن الحياة مليئة بالأسرار الجميلة التي يجب أن نكتشفها ببطء وثقة.

BLEURT Score: 0.2324187308549881
Semantic Coherence: 0.8626271486282349
Grammar Errors: 18
Distinct 1-gram Score: 0.6908212560386473
Distinct 2-gram Score: 0.9805825242718447
Perplexity: 1.6766537427902222

Cleaned Story47:
في يوم شتوي بارد في قطر، كان هناك ثلاثة أصدقاء يعيشون في محطة فضاء صغيرة. كان هناك علي، الطيار الفضائي، وسمر، الباحثة الفضائية، وليلى، الصحفية الفضائية. كانوا يعيشون في محطة الفضاء منذ سنوات، وكانوا يحبون استكشاف الكواكب الجديدة.

ذات يوم، قرروا الذهاب في مغامرة جديدة. كان علي يطير المروحية الفضائية، وسمر تحمل الكاميرا لتصوير الكواكب، وليلى تحمل دفترها لكتابة القصص عن مغامراتهم. كانوا يطيران في الفضاء، يشاهدون النجوم والكواكب، عندما لاحظوا كوكباً غريباً لم يروا من قبل.

"هذا كوكب غريب!" قال علي متحمسًا.

"يجب أن نذهب ونستكشف!" قالت سمر بحماس.

"أنا سأكتشف كل شيء!" قالت ليلى بفضول.

بدأوا رحلتهم نحو الكوكب الغريب. كانوا يطيران في الفضاء، يشاهدون الأشجار والحيوانات، عندما فجأة، وجدوا مدينة سحرية!

"ما هذا؟" قال علي مذهولاً.

"يجب أن نكتشف!" قالت سمر.

"أنا سأكتشف كل شيء!" قالت ليلى.

بينما كانوا يستكشفون المدينة، وجدوا شخصًا يرتدي ثوبًا فضائيًا.

"من أنت؟" سأل علي.

"أنا مخرج المسرحية!" قال الشخص.

"مسرحية؟" قال علي.

"نعم، كل ما حدث هنا جزء من مسرحية!" قال الشخص.

"ماذا تعني؟" سألت سمر.

"أنهتنا مسرحية!" قال الشخص.

"فهمت!" قال علي.

"لقد كانت مغامرة رائعة!" قالت ليلى.

"نعم، وأنا سعيد أنكما شاركتمها!" قال الشخص.

عاد الأصدقاء الثلاثة إلى محطة الفضاء، وهم يضحكون ويتحدثون عن مغامرتهم. كانوا سعيدون جدًا لأنهم شاركوا في مسرحية رائعة.

"لقد كانت مغامرة لا تنسى!" قال علي.

"نعم، وأنا سأكتشف المزيد!" قالت ليلى.

"وأنا سأستكشف المزيد!" قالت سمر.

وهكذا، انتهت مغامرتهم في محطة الفضاء، ولكنهم عرفوا أن هناك مغامرات جديدة تنتظرهم.

BLEURT Score: 0.16670331358909607
Semantic Coherence: 0.8667436838150024
Grammar Errors: 13
Distinct 1-gram Score: 0.676595744680851
Distinct 2-gram Score: 0.8760683760683761
Perplexity: 1.715248703956604

Cleaned Story48:
في محطة قطار صغيرة وهدئة في قلب لبنان، كان هناك قطار صغير ينتظر بداية رحلته. كان هذا القطار يُدعى "القطار الصغير"، وكان يمتلك قلبًا كبيرًا ومشاعرًا حانية.

ذات يوم، وصل "القطار الصغير" إلى المحطة، وبدأ في انتظار بداية رحلته. كان ينتظر بفارغ الصبر، لكنه لم يكن وحيدًا. كان هناك طائر صغير يغرد بجانبه، وكان يبدو عليه الحزن.

"لماذا أنت حزين يا صديقي الطائر؟" سأل "القطار الصغير" بمرح.

"أنا حزين لأنني لا أستطيع الرحلة معك،" أجاب الطائر بصوت حزين.

"لكن يمكننا أن نجد حلًا!" قال "القطار الصغير" بحماس.

فكر "القطار الصغير" وطائرته الصغيراً معاً ووجدوا فكرة. قام "القطار الصغير" بتقديم الطائر الصغير على متن القطار كأول مسافر، حتى يبدأ رحلته معه.

لكن، قبل أن يبدأ القطار رحلته، حدث شيء لم يكن في الحسبان. توقف القطار فجأة بسبب عطل، ولم يتمكن "القطار الصغير" من استكمال رحلته.

"لماذا توقفت؟" سأل "القطار الصغير" بقلق.

"أنا آسف يا صديقي،" قال أحد المسؤولين، "لكن هذا العطل لن يسمح لك بالاستمرار في رحلتك."

شعر "القطار الصغير" بالحزن الشديد لأنه لم يستطع إكمال رحلته مع صديقه الطائر. لكنه قرر أن يبقى هادئًا ويعطي الطائر الصغير درسًا قيمًا.

"الصداقة لا تعرف حدود،" قال "القطار الصغير" لصديقه الطائر، "وأنا سأظل دائمًا معك، حتى لو لم نتمكن من الرحلة معًا."

ومع ذلك، كان هناك شعور حزين في قلب "القطار الصغير" لأنه لم يستطع إكمال رحلته مع صديقه. لكنه تعلم أن الرد على المواقف الصعبة يعتمد على كيفية تفكيرنا وقراراتنا.

وهكذا، انتهت رحلة "القطار الصغير" وطائرته الصغير في محطة القطار الصغيرة، ولكن بقلوب مليئة بالحب والصداقة.

BLEURT Score: 0.1594434678554535
Semantic Coherence: 0.8747169971466064
Grammar Errors: 7
Distinct 1-gram Score: 0.6557377049180327
Distinct 2-gram Score: 0.8930041152263375
Perplexity: 1.9293709993362427

Cleaned Story49:
في يوم مشمس جميل في المملكة العربية السعودية، كان هناك طفلان صغيران يدعيان يوسف وعمر. كان يوسف وعمر يحبان اللعب والمرح، لكنهما كانا يحبان ممارسة الرياضة أكثر من أي شيء آخر. كانا يلعبان كرة القدم كل يوم بعد المدرسة.

ذات يوم، قرر يوسف وعمر أن يذهبا إلى مطعم جديد في الحي، حيث يمكنهما تناول وجبة بعد ممارسة الرياضة. كان المطعم يسمى "النخيل الذهبي" وكان يشتهر بوجباته اللذيذة.

وصل يوسف وعمر إلى المطعم وطلبا وجبتهما المفضلة. كان يوسف يأكل بسرعة، لكن عمر كان يأكل ببطء. فجأة، حدث شيء غير متوقع. بينما كان عمر يأكل، انزلقت قدمه تحت الطاولة وأصبح محاصرًا.

كان يوسف في صدمة، لكنه لم يفقد الأمل. فكر في طريقة مبتكرة لنجاة صديقه. فجأة، تذكر يوسف أن هناك بعض الألعاب الرياضية التي يمكن أن تساعد في النجاة من مواقف صعبة. فكر في استخدام الكرة التي كانا يلعبان بها كل يوم.

بسرعة، قام يوسف بجمع الكرة وبدأ يرفعها من تحت الطاولة. كان عمر يرفع رأساً ويشعر بالقلق، لكن يوسف لم يفقد الأمل. بعد محاولات عديدة، تمكن يوسف أخيرًا من رفع الكرة ورفع الطاولة معها، مما جعل عمر يخرج من تحتها.

عمر كان سعيدًا جدًا وبدأ يضحك. يوسف كان سعيدًا جدًا أيضًا لأنه نجح في إنقاذ صديقه. بعد ذلك، قرر يوسف وعمر أن يطلبا وجبتهما مرة أخرى، ولكن هذه المرة، كانا يأكلان ببطء ويشعران بالامتنان للنجاة من موقف ميؤوس منه.

في نهاية اليوم، عادا يوسف وعمر إلى البيت، وهم يضحكان ويتشاركان قصتهم المثيرة. كانت نهاية يومهما سعيدة، وأدركا أن الصداقة والمرح هما أثمن ما في الحياة. ومن ذلك اليوم فصاعدًا، أصبح يوسف وعمر أكثر حرصًا على النجاة في المواقف الصعبة، ولكنهم لم ينسوا أبدًا أن الصداقة والمرح هما ما يجعل الحياة أجمل.

BLEURT Score: 0.24320638179779053
Semantic Coherence: 0.8841598629951477
Grammar Errors: 20
Distinct 1-gram Score: 0.6518518518518519
Distinct 2-gram Score: 0.9219330855018587
Perplexity: 1.863338589668274

Cleaned Story50:
في حديقة جميلة بمدينة عراقية، كان يعيش طفلان صديقان لا يفترقان، يُدعيان يوسف وأمير. كانا يحبان اللعب سوياً في الحديقة، وكان نشاطهما المفضل هو السباحة في البحيرة الصغيرة التي كانت تملأ الحديقة بجمالها.

يوسف وأمير كانا يذهبان إلى الحديقة كل يوم بعد المدرسة، وكانا يبدأان يومهما بالسباحة في البحيرة. كانا يضحكان ويلعبون، وكان كل يوم هو يوم جديد مليء بالمرح والفرح.

ذات يوم، بينما كان يوسف وأمير يسبحان في البحيرة، لاحظا شيئاً غريباً. البحيرة بدأت تزداد قلوة، وبدأت السماء تغيم. كان هناك شيء يحدث، لكن يوسف وأمير لم يعلما ما هو.

بعد فترة من الزمن، بدأت المياه تتحول إلى بركة صغيرة. كان يوسف وأمير حزينين جدًا. كانت البحيرة التي كانا يحبانها كثيراً قد اختفت. حاولا البحث عن سبب ذلك، لكن دون جدوى.

يوسف وأمير جلسا على ضفاف البحيرة القديمة، ينظران إلى المكان الذي كان يملأه الماء والمرح. كانا حزينين لأن الحديقة لم تعد كما كانت. لكنهما تعلما درسًا قيمًا عن الحياة: أن كل شيء في الحياة يمكن أن يتغير، وأن الأشياء الجميلة قد تختفي، لكن الذكريات الجميلة التي خلقناها معًا ستظل معنا إلى الأبد.

وهكذا، على الرغم من الحزن الذي شعر به يوسف وأمير، إلا أن قلوبهما كانت مليئة بالامتنان للذكريات الجميلة التي خلقاها معًا في تلك الحديقة.

BLEURT Score: 0.2480095773935318
Semantic Coherence: 0.8508207201957703
Grammar Errors: 18
Distinct 1-gram Score: 0.696969696969697
Distinct 2-gram Score: 0.9644670050761421
Perplexity: 1.7580713033676147

Cleaned Story51:
في قرية صغيرة بالسودان، حيث الشتاء يغطي الأرض بثوبه البارد، كان يعيش صبي صغير يُدعى يوسف مع صديقه الأفضل، الطائر النورس. كان يوسف يحب بناء الكتل من الحجارة التي تتراقص تحت قدميه، بينما نورس يراقبه من على شجرة عالية، يغرد بألحان هادئة.

ذات يوم، بينما كان يوسف يبني كتلة جديدة، دخل غضبه على نورس لأنه لم يكن يغني كما يفعل عادة. شعر يوسف بالغضب يملأ قلبه، فرفع حجرًا كبيرًا ليضرب الطائر. لكن قبل أن يفعل ذلك، وقف فجأة وهمس لنفسه: "ماذا أفعله؟ هل أريد أن أدمر صديقي بسبب غضبي؟"

فكر يوسف لحظات، ثم نظر إلى نورس الذي كان ينظر إليه بقلب صغير يخاف. شعر يوسف بالندم الشديد على غضبه، فرفع الحجر عن الأرض ووضع يده على صدر نورس، كأنه يأخذ منه العهود.

فجأة، بدأت السماء تغيم، وأدرك يوسف أن عاصفة شتوية قوية تتهدد القرية. أدرك أن الوقت يفوت، وأنهم يجب أن يجدوا مأوى سريعًا. نظر يوسف إلى نورس وقال: "لنكون معًا، لننجو معًا."

باستخدام ذكائه، وجد يوسف طريقة مبتكرة لإنشاء مأوى صغير من الأشجار والأغصان. عمل يوسف ونورس معًا، يجمعان الأخشاب ويعتنون بالمأوى حتى أصبح مأوىً آمنًا. وبينما العاصفة تهدد، وجد يوسف ونورس مأوىً آمنًا.

بعد العاصفة، خرج يوسف ونورس من المأوى ليجدوا القرية بأكملها قد نجت. كانت القرية قد تعرضت للخسارة، لكن الجميع كانوا آمنين. يوسف، الذي كان يشعر بالندم الشديد على غضبه، أدرك أن الغضب قد يترك ندوبًا على الناس، لكنه يمكن أن يؤدي أيضًا إلى تحديات جديدة التي يمكن أن تؤدي إلى النجاة.

من ذلك اليوم، وعد يوسف نفسه أن يكون دائمًا أكثر هدوءًا، وأن يعتني بقلب صديقه نورس. وأصبح يوسف ونورس أفضل الأصدقاء، يبنيان الكتل معًا، ولكن هذه المرة، مع فهم أكبر للقيم والأحلام.

وهكذا، في قرية صغيرة بالسودان، تعلم يوسف أن الغضب يمكن أن يترك ندوبًا، لكن الحكمة والهدوء يمكن أن يؤديان إلى نجاة لا تُنسى.

BLEURT Score: 0.20877626538276672
Semantic Coherence: 0.870975136756897
Grammar Errors: 21
Distinct 1-gram Score: 0.7044673539518901
Distinct 2-gram Score: 0.9275862068965517
Perplexity: 2.210181713104248

Cleaned Story52:
في قرية صغيرة ببلاد البحرين، حيث الشمس تلمع بأشعتها الذهبية في فصل الصيف، كان يعيش طفل صغير يُدعى يوسف. يوسف كان يحب الموسيقى والغناء كثيرًا، وكان دائمًا ما يغني بأصواته العذبة تحت شجرة النخيل الكبيرة في حديقة منزله.

ذات يوم، بينما كان يوسف يجلس تحت شجرة النخيل، قال لنفسه: "لن أغني اليوم، سأجلس فقط وأستمتع بالهدوء."

فجأة، سمع صوتًا غريبًا يأتي من خلف الشجرة. كان صوتًا يبدو وكأن هناك شخصًا آخر يغني. يوسف، بفضول، اقترب من الصوت وأجاب بهدوء: "من هناك؟"

"أنا... أنا الأصوات التي تملأ السماء!" أجاب الصوت الغريب.

يوسف، متحمسًا، قال: "هل يمكنك أن تغني معي؟"

"نعم، لكن يجب أن تكون مستعدًا لسماع شيء غير معتاد!" قال الصوت.

فجأة، بدأت الأصوات تتلازم مع صوت يوسف، مما خلق موسيقى غنائية ساحرة. يوسف شعر بالفرح والسعادة، وأصبح يغني بأقوى صوته.

لكن بعد لحظات، انقطع الصوت الغريب فجأة. يوسف، حزينًا، قال: "لماذا توقفت؟"

"لقد كنت خدعة، يا يوسف. لم يكن هناك أحد آخر يغني معك. كانت فقط صوتك الذي خلقت موسيقى ساحرة!" قال الصوت الغريب، الذي لم يكن سوى صدى صوت يوسف نفسه.

يوسف شعر بحزن شديد، وقال: "لكن كيف يمكن أن يكون ذلك؟ لقد شعرت بأن هناك شخصًا آخر يغني معي."

"الصدى، يا يوسف، هي قدرة عجيبة. يمكن أن تظهر لك أن هناك أشياء يمكن أن تحدث دون أن تكون حقيقية." قال الصوت الغريب.

يوسف، حزينًا، نظر إلى السماء، وقال: "لقد كنت أتمنى أن يكون هناك شخص آخر يشاركني الحب للموسيقى."

"لكن، يا يوسف، أنت قادر على خلق موسيقى ساحرة بنفسك. لا تحتاج إلى شخص آخر. فقط استمتع بجمال الموسيقى التي تملأ قلبك!" قال الصوت الغريب.

يوسف، بعد لحظات من الدهشة، ابتسم قليلاً وقال: "أنت صحيح. سأستمتع بالغناء، مهما كان."

وبهذا، انتهت القصة، لكن يوسف لم ينسى درسًا قيمًا عن كيفية الاستمتاع بالجمال الذي يوجد داخل كل منا، حتى لو كان ذلك جمالًا خياليًا.

BLEURT Score: 0.15722373127937317
Semantic Coherence: 0.7941508293151855
Grammar Errors: 23
Distinct 1-gram Score: 0.7040816326530612
Distinct 2-gram Score: 0.9283276450511946
Perplexity: 1.754203200340271

Cleaned Story53:
في قرية صغيرة بالسودان، كان يعيش ثلاثة أخوة: يوسف، أحمد، وعمر. كانوا يعيشون مع والديهم في بيت صغير، وكان كل واحد منهم يمتلك مهنة مختلفة. يوسف كان يعمل في الحقل، أحمد كان صانعًا للأحجار الكريمة، وعمر كان يعمل في الملاحة.

كانت الأيام تتساقط أوراق الأشجار بألوانها الذهبية والبرتقالية، مما جعل الخريف أجمل فصول السنة. الأخوين يوسف وأحمد كانا يعتبران عمر أخيهم الصغير الذي كان دائمًا يتبعهما في كل ما يفعله.

ذات يوم، قرر يوسف وأحمد أن يذهبا في رحلة إلى مدينة بعيدة لبيع الأحجار الكريمة. قبل أن يغادرا، وعدا عمر بأن يعودا قبل حلول الظلام.

بعد مرور الوقت، عاد يوسف وأحمد من رحلتهما، لكنهما كانا حزينين. لقد كانا قد بيعا الأحجار الكريمة بأسعار منخفضة، مما جعلهما لا يستطيعان شراء ما يحتاجانه من الطعام والملابس.

عندما عاد يوسف وأحمد إلى القرية، وجدا عمر ينتظرهم بقلب مليء بالحزن. كان عمر قد تعلم كيف يصنع الأحجار الكريمة، ولكنه لم يكن قادرًا على بيعها. كان يوسف وأحمد حزينين لأنهم لم يستطيعا مساعدة أخيهم الصغير.

وقف الأخوين يوسف وأحمد أمام عمر، وقال يوسف: "أخي، أنا آسف لأننا لم نستطيع مساعدتك كما يجب. لكننا تعلمنا درسًا قيمًا اليوم. الأخوة يجب أن يكونوا دائمًا معًا، وأن يساعدوا بعضهم البعض."

أومأ عمر برأسه، وهو يحاول أن يبتسم. "أنا فخور بكما، يا أخائي. أنا سعيد لأنني تعلمت كيف أصنع الأحجار الكريمة."

ومع ذلك، كان الحزن يخيم على قلوبهم. لقد كانوا يعلمون أن الحياة قد تكون صعبة، لكنهم كانوا يأملون أن يكونوا دائمًا هناك لبعضهم البعض.

وهكذا، في تلك الليلة الخريفية، جلس الأخوين الثلاثة معًا، يتشاركون الحزن والفرح، مدركين أن الأخوة هي أعظم كنز يمكن أن يملكوه.

BLEURT Score: 0.19045615196228027
Semantic Coherence: 0.8672797083854675
Grammar Errors: 16
Distinct 1-gram Score: 0.7354085603112841
Distinct 2-gram Score: 0.95703125
Perplexity: 2.08685564994812

Cleaned Story54:
في قرية صغيرة جميلة في اليمن، كان يعيش طفل صغير يُدعى ياسين. ياسين كان يعيش وحيدًا، لكنه كان دائمًا سعيدًا ومليئًا بالحب للطبيعة والحيوانات. كان فصل الخريف قد حلّ، وكان الأشجار تتساقط أوراقها الذهبية والبرتقالية، مما جعل القرية تبدو أجمل من أي وقت مضى.

يومًا ما، بينما ياسين يلعب في الحديقة، شعر بشيء غريب يحدث في داخله. فجأة، بدأ يديه يتحركان بمفردهما، وكأنه يرسم شيئًا. كان ياسين مذهولًا، لكنه لم يفهم ما يحدث.

فجأة، ظهرت أمامه شجرة صغيرة، كانت تبتسم له. كانت الشجرة تظهر أمام عيني ياسين، لكنها لم تكن هناك في الحقيقة. كانت الشجرة تظهر فقط لقلبه.

شعر ياسين بالدهشة والفرح. كان لديه قوة خارقة! بفضل هذه القوة، بدأ ياسين يرسم أشجارًا وزهورًا في الحديقة، مما جعلها أجمل من أي وقت مضى.

مع مرور الأيام، أصبحت القرية مكانًا ساحرًا، حيث الأشجار والزهور تتراقص في الهواء. الناس في القرية كانوا يأتون ليرى هذا السحر، وهم يعتبرون ياسين بطلاً.

ياسين، الذي كان يعتبر نفسه وحيدًا، وجد نفسه محبوبًا ومحترمًا. كان سعيدًا جدًا، لكنه لم ينسى أبدًا الشجرة الصغيرة التي ظهرت أمامه في البداية.

في نهاية الخريف، عندما كانت الطبيعة تتحول إلى الهدوء والهدوء، شعر ياسين بقوة جديدة تملأ قلبه. كان يعرف أن هذه القوة ستظل معه، وأن كل شيء يمكن أن يتحول إلى أجمل ما إذا كان بإمكانه رؤية الجمال في كل شيء.

وهكذا، عاش ياسين بسعادة، محاطًا بالأشجار والزهور التي رسمها بقلبه، وكل شيء في حوله كان أجمل من أي وقت مضى.

BLEURT Score: 0.24766939878463745
Semantic Coherence: 0.8648862838745117
Grammar Errors: 5
Distinct 1-gram Score: 0.6810344827586207
Distinct 2-gram Score: 0.935064935064935
Perplexity: 1.8282305002212524

Cleaned Story55:
في قرية صغيرة بالعراق، كان هناك ثلاثة أصدقاء: ياسمين، أحمد، وسمر. كانوا يحبون السباحة كثيرًا وكانوا يذهبون إلى النهر كل يوم بعد المدرسة. كانوا يعتبرون النهر مكانًا سحريًا حيث يمكنهم التحدث عن أحلامهم وأحلامهم المستقبلية.

ذات يوم، بينما كانوا يسبحون، قال أحمد بصوت متحمس: "لقد وجدت مكانًا سحريًا في النهر، إذا سبحنا إليه، سنجد كنزًا!"

ياسمين وسمر نظرا إليه بدهشة وقالت ياسمين: "أحمد، هل أنت متأكد من ذلك؟"

أحمد ابتسم وقال: "بالطبع، أنا متأكد. لقد رأيت الكنز بنفسي!"

سمر، التي كانت دائمًا صادقة، قالت: "لكن، أحمد، يجب أن نكون صادقين مع بعضنا البعض. لا يمكن أن يكون هناك كنز في النهر."

ياسمين، التي كانت تحب السباحة كثيرًا، قالت: "أحمد، ربما يمكننا البحث عن الكنز معًا، لكن يجب أن نكون صادقين مع بعضنا البعض."

أحمد نظر إلى أصدقائه بدهشة وقال: "أنا آسف، لم أكن أعرف أنني سأجعلكم تشعرين بهذا الطريقة."

بعد ذلك اليوم، قرر الأصدقاء الثلاثة البحث عن الكنز معًا، لكنه لم يكن هناك. وجدوا فقط بعض الأشياء القديمة التي كانت تحكي قصصًا عن الأوقات الجميلة التي كانوا يسبحون فيها.

في النهاية، جلس الأصدقاء الثلاثة على ضفاف النهر، يشاهدون الشمس تغيب. قال أحمد بصوت حزين: "أنا آسف، لم أكن أريد أن أجعلكم تشعرين بالحزن."

ياسمين، التي كانت تحب الصدق كثيرًا، قالت: "أحمد، الصدق هو أهم شيء في الحياة. يجب أن نكون صادقين مع بعضنا البعض، حتى وإن كانت الحقيقة صعبة."

سمر، التي كانت دائمًا حكيمة، قالت: "نعم، الصدق يجعلنا أقوى وأكثر صداقة. لا يهم أننا لم نجد الكنز، لأننا وجدنا صداقتنا."

فجأة، انقطع الصوت عنهم، وهم لم يلاحظوا. نظروا حولهما، لكنهم لم يجدوا أحدًا. نظر أحمد إلى ياسمين وسمر وقال: "هل يمكن أن يكون هذا كل شيء؟"

ياسمين وسمر نظرا إليه بدهشة وقالتا: "أحمد، هل أنت متأكد؟"

أحمد ابتسم وقال: "نعم، أنا متأكد. كل ما حدث كان جزءًا من مسرحية. لقد كاننا نلعب دورًا في مسرحية عن الصدق."

ياسمين وسمر نظرا إلى بعضهما البعض، ثم إلى أحمد، وقالتا: "لقد علمتنا الصدق، أحمد. شكرًا لك."

أحمد ابتسم وقال: "أنا سعيد لأنني يمكن أن أتعلم معكم. وأتمنى أن تكون هذه القصة تذكرًا لكما عن أهمية الصدق."

وبهذا، انتهت القصة، لكن الدرس الذي تعلماه سيبقى معهم إلى الأبد.

BLEURT Score: 0.17784912884235382
Semantic Coherence: 0.8681299090385437
Grammar Errors: 12
Distinct 1-gram Score: 0.625
Distinct 2-gram Score: 0.8600583090379009
Perplexity: 2.08794903755188

Cleaned Story56:
في يوم خريفي جميل في الجزائر، كان هناك طفلان صغيران يدعيان يوسف وعمر. كانا يحبان اللعب في الحديقة، لكنهما كانا يحلمان بأن يزوران الفضاء يومًا ما.

في أحد الأيام، بينما كان يوسف وعمر يلعبون بالقرب من النجوم، قال يوسف بحماس: "تخيل يا عمر، ماذا لو كنا نستطيع السفر إلى الفضاء؟"

عمر نظر إلى النجوم وقال بفضول: "لقد أردت دائمًا أن أرى كيف تبدو الكواكب الأخرى!"

فجأة، بينما كانا يفكران في الفضاء، ظهرت أمامهما كوكب صغير. كان يوسف وعمر متحمسين جدًا، وقال يوسف: "هل يمكن أن نذهب إليه؟"

عمر، بابتسامة كبيرة، أجاب: "لنفعل! دعونا نرى ماذا يحدث!"

بمساعدة من بعض الأصدقاء الخياليين، تمكن يوسف وعمر من السفر إلى الكوكب الصغير. كان هناك أشجار زاهية بالألوان وأزهار ملونة تزهو في كل مكان. كان الكوكب مليئًا بالحيوانات الغريبة والطيور الملونة.

يوسف، وهو متحمس، قال: "هذا مدهش! لقد لم أكن أتوقع أن يكون الكوكب كذلك!"

عمر، وهو يلعب مع الحيوانات، قال: "أنا أحب هذا! يمكننا اللعب هنا طوال اليوم!"

بعد يوم من الاستمتاع بالكوكب، عادا يوسف وعمر إلى الأرض. كانا سعيدين جدًا وأحضرا معهما قصصًا رائعة عن مغامرتهما في الفضاء.

عندما عادا إلى الحديقة، قال يوسف لعمر: "لقد كان يومًا رائعًا، أليس كذلك؟"

عمر، وهو يبتسم، أجاب: "نعم، لقد كان يومًا لا يُنسى! وأتمنى أن نذهب إلى الفضاء مرة أخرى."

يوسف، وهو يضحك، قال: "ربما في يوم ما، يا صديقي!"

وهكذا، انتهى يوم يوسف وعمر بقلوب مليئة بالفرح والمرح، مدركين أن الأحلام يمكن أن تصبح حقيقة، وأن المغامرات في الفضاء ستظل دائمًا ذكريات جميلة.

BLEURT Score: 0.1724652349948883
Semantic Coherence: 0.8307321667671204
Grammar Errors: 14
Distinct 1-gram Score: 0.6875
Distinct 2-gram Score: 0.9414225941422594
Perplexity: 1.6624867916107178

Cleaned Story57:
في قرية صغيرة بالأردن، كان يعيش طفل يُدعى يوسف. يوسف كان يحب بناء الكتل، وكان يحلم بأن يصبح مهندسًا كبيرًا يومًا ما. كل يوم بعد المدرسة، كان يوسف يجلس تحت شجرة الزيتون الكبيرة في حديقة منزله، يجمع الكتل ويبني عليها أبراجًا عالية ومتاهيًا من الكتل.

ذات يوم، بينما كان يوسف يبني، لاحظ فجأة كائنًا غريبًا يبدو كأنها كائن خارق. كان الكائن يراقب يوسف بفضول. يوسف، في البداية، شعر بالخوف، لكنه قرر أن يتقدم نحو الكائن ويتحدث إليه.

"مرحبًا، هل أنت هنا لتعلم بناء الكتل معي؟" سأل يوسف بفضول.

"نعم، أنا هنا لتعلم منك. أنا من عالم آخر، وأنا أتمنى أن أتعلم كيف بناء الكتل منك." قال الكائن.

يوسف، الذي كان دائمًا يحب مشاركة مهاراته، شعر بالفخر والرضا. "أنا سعيد جدًا أنني يمكنني تعليمك. هل تريد أن نبدأ الآن؟"

"نعم، سأكون شاكرًا جدًا." قال الكائن.

بدأ يوسف في تعليم الكائن كيفية بناء الكتل. كان الكائن يتعلم بسرعة، وكان يوسف سعيدًا جدًا بتعليمه. لكن بعد أيام قليلة، بدأ يوسف يلاحظ أن الكائن يبدو أكثر قوة وغضبًا.

"لماذا أنت غاضب يا صديقي؟" سأل يوسف بحزن.

"أنا غاضب لأنني لا أستطيع بناء الكتل كما تفعل أنت. أنا لا أستطيع أن أكون مثلك." قال الكائن.

يوسف شعر بالحزن. "أنا سأعلمك أكثر، لا تقلق."

لكن في تلك الليلة، بينما كان يوسف ينام، حدث شيء غير متوقع. الكائن، الذي كان يبدو كأنها كائن خارق، اختفى فجأة. يوسف استيقظ في الصباح ليجد أن كل الكتل التي كان يبنيها قد انهارت.

"لماذا فعلت هذا؟" قال يوسف بصوت حزين.

"أنا لا أستطيع البقاء هنا. أنا لا أستطيع أن أكون مثلك." قال الكائن من بعيد قبل أن يختفي.

يوسف شعر بحزن شديد. لقد فقد صديقه الجديد، ولم يعد بإمكانه بناء الكتل كما كان يفعل. "أتمنى أن أكون قويًا مثلك، حتى أستطيع بناء الكتل مرة أخرى." قال يوسف بصوت حزين.

الدرس الذي تعلمه يوسف هو أن يجب أن يتذكر دائمًا أن يجب أن يتجنب من يبدو أنهم يحبون أو يحتاجون إلى مساعدته، لأنهم قد لا يكونون دائمًا كما يبدو.

BLEURT Score: 0.19369828701019287
Semantic Coherence: 0.8729017376899719
Grammar Errors: 22
Distinct 1-gram Score: 0.6088328075709779
Distinct 2-gram Score: 0.8924050632911392
Perplexity: 1.998820424079895

Cleaned Story58:
في قرية صغيرة في فلسطين، كان هناك صبي صغير يُدعى يوسف، وصديقه الروبوت الذي أهداه له جده. كان يوسف يحب الروبوت كثيرًا وكان يعتبره صديقًا حقيقيًا. كان يوسف يلعب مع الروبوت كل يوم بعد المدرسة، ويحكي له كل شيء.

ذات يوم، قرر يوسف أن يبني مع الروبوت أكبر آلة يمكن أن يبنيها. كان الروبوت يراقب يوسف وهو يجمع الألوان والأخشاب، وكان يوسف يتحدث إليه عن كل خطوة يفعلها. كان يوسف متحمسًا جدًا، وكان يتحدث أكثر من المعتاد.

بينما كان يوسف يبني الآلة، بدأت السماء تغيم وبدأت الرياح تعصف. فجأة، انزلقت يد الروبوت وضربت يوسف، مما أدى إلى إصابته. شعر يوسف بالإحباط الشديد، وقرر أن يبني الآلة بمفرده دون أن يتحدث مع الروبوت.

بينما كان يوسف يعمل، بدأت يد الروبوت تتحول إلى لون ذهبي ساطع. فجأة، ظهرت قوة خارقة غير متوقعة من الروبوت، مما أدى إلى إصلاح الآلة التي كان يوسف يبنيها. ولكن، عندما نظر يوسف إلى الروبوت، كان قد انتهى من بنائه، ولم يكن هناك أي علامة على أن الروبوت كان موجودًا.

شعر يوسف بالإحباط الشديد، لأنه فقد صديقه الروبوت. كان يوسف يدرك الآن أن الحديث أكثر من اللازم قد أدى إلى فقدان صديقه. كان يوسف يدرك أن الملاحظة والاستماع كانا أكثر أهمية من الحديث.

في نهاية القصة، كان يوسف يجلس وحيدًا، ينظر إلى الآلة التي بناها، يتذكر صديقه الروبوت الذي فقدته. كان يوسف يحزن على فقدان صديقه، ولكن كان أيضًا يدرك درسًا قيمًا: أن يكون أكثر ملاحظة وأقل كلام.

BLEURT Score: 0.19756440818309784
Semantic Coherence: 0.888790488243103
Grammar Errors: 24
Distinct 1-gram Score: 0.591304347826087
Distinct 2-gram Score: 0.8995633187772926
Perplexity: 1.6725425720214844

Cleaned Story59:
في يوم خريفي جميل في عمان، قرر ثلاثة أصدقاء الصغار، ياسمين، عمر، وسارة، الذهاب إلى المتحف الوطني لاستكشاف أسرار الماضي. كانوا متحمسين جدًا، لكنهم لم يكنوا يدركون ما انتظره لهم.

"أنا سأرى العجائب!" قالت ياسمين بحماس.

"وأنا سأتعلم الكثير!" أضاف عمر.

"نحن سنجد كنوزًا قديمة!" تمنى سارة.

وصل الأصدقاء الثلاثة إلى المتحف، حيث كان هناك العديد من الأعمال الفنية والتحف القديمة. كانوا يزورون كل معرض، يستمعون إلى قصص الحضور، ويستكشفون كل شيء بحماس.

لكن في أحد الأماكن، وجدوا نفسهم أمام معرض خاص بالتاريخ القديم. كان هناك العديد من الأشياء القديمة، لكن ما جلب انزعاجهم هو أن أحد الأطفال الآخرين بدأ يلعب ببعض الأشياء القديمة دون أن يلاحظ أحد.

"يجب أن نوقفه!" قالت ياسمين بقلق.

"نعم، لا يمكنه أن يفعل ذلك!" أضاف عمر.

فجأة، بدأت الأشياء القديمة تتحرك، وكأنها تحاول أن تهدد الأطفال. كانوا في موقف ميؤوس منه، لكنهم لم يفقدوا الأمل.

"يجب أن نجد طريقة لاستعادة الأشياء إلى مكانها!" قال سارة.

فكر الأصدقاء الثلاثة بسرعة، ثم وجدوا فكرة مبتكرة. استخدموا بعض الألعاب التي كانوا يحملونها معهم لجذب انتباه الأطفال الآخرين، مما جعلهم يلاحظون ما كان يحدث.

"أحضروا المساعدة!" صرخ عمر.

وصل الحضور الآخرين، وهم يجدون الأطفال يلعبون ببعض الأشياء القديمة. كانوا قلقين جدًا، لكنهم سعيدون لأن الأطفال الآخرين كانوا آمنين.

"شكرًا لكما على مساعدتنا!" قال ياسمين.

"نحن سعداء بأننا أدركنا الوقت قبل أن يحدث شيء سيء!" أضاف سارة.

"نعم، يجب أن نكون دائمًا حذرين في المتحف!" قال عمر.

بعد ذلك، استمر الأصدقاء الثلاثة في استكشاف المتحف، لكنهم كانوا أكثر حذرًا. وعندما انتهوا، شعر الأطفال بالفخر لأنهم تمكنوا من تجنب موقف ميؤوس منه بطريقة مبتكرة.

"لقد تعلمنا درسًا قيمًا اليوم!" قال ياسمين.

"نعم، يجب أن نكون دائمًا حذرين ونحترم الأشياء القديمة!" أضاف عمر.

"وأنا سعيد أننا أدركنا الوقت قبل أن يحدث شيء سيء!" قالت سارة.

وهكذا، انتهى يومهم في المتحف بضحكات وقلوب ممتنة، مدركين أن الأمان يأتي من الحذر والتفكير قبل الفعل.

BLEURT Score: 0.18333634734153748
Semantic Coherence: 0.8833339214324951
Grammar Errors: 8
Distinct 1-gram Score: 0.6611295681063123
Distinct 2-gram Score: 0.9066666666666666
Perplexity: 2.060335159301758

Cleaned Story60:
في جزيرة صغيرة جميلة على سواحل ليبيا، كان يعيش صبي صغير يُدعى يوسف. يوسف كان يحب جزيرته كثيرًا وكان يحب القيام بالأشغال اليدوية، مثل صنع الورق اللامع والتماثيل الصغيرة من الرمال.

ذات يوم، بينما كان يوسف يجوب الجزيرة، وجد شيئًا غريبًا. كان هناك شخص يبدو شريرًا جدًا، يرتدي ملابس سوداء، يبدو كأنه يراقب الجزيرة. كان يوسف خائفًا جدًا، لكنه قرر أن يكتشف من هو هذا الشخص.

"من أنت؟" سأل يوسف بصوت مرتجف.

"أنا... أنا رجل البحر." أجاب الشخص الشرير بصوت عميق.

"رجل البحر؟" سأل يوسف بفضول.

"نعم، أنا رجل البحر. وأنا هنا لجعل هذه الجزيرة مكانًا خبيثًا." قال الشخص الشرير.

يوسف شعر بالخوف، لكنه لم يركن للخوف. بدلاً من ذلك، قرر أن يجد طريقة لجعل هذا الرجل يغير رأيه.

"لكن الجزيرة جميلة جدًا. لماذا تريد أن تجعلها مكانًا خبيثًا؟" سأل يوسف.

الرجل البحر نظر إلى يوسف بعيون غامضة، ثم قال: "أنا... أنا لست رجل البحر. أنا... أنا والدك."

يوسف لم يصدق ما سمعه. "كيف؟ كيف أنت والدتي؟" سأل يوسف بفضول.

"أنا والدك، يا يوسف. كنت أتخيل أنك ستكون سعيدًا إذا وجدت مكانًا خبيثًا هنا. لكنك الآن، أرى أنك تحب هذه الجزيرة كثيرًا." قال الرجل بدموع في عينيه.

يوسف شعر بالفرح. "أنا سعيد جدًا لأنك هنا، يا أبي. هل يمكنك أن تغير رأيك وأن تصبح صديقي بدلًا من رجل البحر الشرير؟"

الرجل ابتسم، وبدا سعيدًا جدًا. "نعم، يا يوسف. أنا سأكون صديقك. سأعيد الجزيرة جمالها مرة أخرى."

من ذلك اليوم فصاعدًا، عمل يوسف وأبيه معًا لجعل الجزيرة مكانًا أجمل. كان يوسف يتعلم من أبيه كيف يصنع الأشغال اليدوية، وأبيه يتعلم من يوسف كيف يكون صديقًا أفضل.

وهكذا، انتهى يوم يوسف بسعادة، مع صديق جديد وأبيه الذي عاد إلى حياته. ومن ذلك اليوم، كانت الجزيرة مكانًا أفضل، حيث كان الجميع يعيش في سلام وصداقة.

BLEURT Score: 0.22421304881572723
Semantic Coherence: 0.7371791005134583
Grammar Errors: 22
Distinct 1-gram Score: 0.6808510638297872
Distinct 2-gram Score: 0.9466192170818505
Perplexity: 1.7781730890274048

Cleaned Story61:
في جبال الكويت الجميلة، حيث تتفتح الزهور بألوانها الزاهية في فصل الربيع، كان يعيش ثلاثة أصدقاء: ريم، يوسف، والسلحفاة لولو. كانوا يحبون اللعب معًا، لكنهم كانوا يفترقون كل صباح، حيث كان يوسف وريم يعودان إلى منازلهم في القرية، بينما كانت لولو تعيش في الجبال.

ذات يوم، قرر الأصدقاء الثلاثة أن يجمعوا بعض الفواكه الطازجة من الجبال لطهي وصفة بسيطة من العسل والفواكه. كانوا متحمسين جدًا، لكنهم واجهوا تحديًا كبيرًا. لولو، السلحفاة الصغيرة، كانت تعاني من مشكلة في السير، فكانت بطيئة جدًا.

ريم، بقلبها الكبير، اقترحت أن يرافقوا لولو، وأن يأخذوا معهم كل ما يحتاجون من الفواكه. يوسف، بحماس، اقترح أن يصنعوا عربة صغيرة من الأوراق والغصنات لرفع لولو فوقها، مما يجعلها تتمكن من السير بسهولة أكبر.

معًا، استطاعوا أن يجمعوا الفواكه الطازجة ويعملوا على وصفة العسل والفواكه. كانوا يضحكون ويتحدثون، وهم يطهوون معًا، وكان ذلك أجمل يوم في حياتهم.

لكن، أثناء الطهي، حدث شيء لم يتوقعوه. لولو، التي كانت تحب الطهي مع أصدقائها كثيرًا، شعرت بالتعب الشديد. ريم و يوسف، بقلوبهما الحانية، حاولا أن يرفعوا منزلقاتها، لكنها كانت لا تستطيع السير.

في تلك اللحظة الحزينة، أدرك الأصدقاء الثلاثة أن هناك قوة في الوحدة، وأنهم كانوا أقوى عندما يعملون معًا. رغم حزنهم على فقدان صديقتهم لولو، إلا أنهم تعلموا درسًا قيمًا عن أهمية العمل معًا والاهتمام ببعضهم البعض.

وهكذا، في تلك الجبال الجميلة، حيث كانت الزهور تتفتح بألوانها الزاهية، تذكروا دائمًا لولو، وأهمية الوحدة والعمل معًا، حتى في أصعب الأوقات.

BLEURT Score: 0.19127903878688812
Semantic Coherence: 0.8469144105911255
Grammar Errors: 21
Distinct 1-gram Score: 0.7467248908296943
Distinct 2-gram Score: 0.9649122807017544
Perplexity: 1.9357529878616333

Cleaned Story62:
في قرية جميلة بالأردن، حيث تتساقط أوراق الأشجار بألوانها الذهبية والبرتقالية، كان يعيش طفل صغير يُدعى يوسف. يوسف كان يحب اللعب والمرح، لكنه كان يحب الرياضة أكثر من أي شيء آخر. كان يوسف يذهب إلى المدرسة كل يوم، حيث يشارك في الألعاب الرياضية مع أصدقائه.

ذات يوم، بينما كان يوسف يلعب كرة القدم مع أصدقائه، لاحظ شيئًا غريبًا. كان هناك طفل آخر يلعب وحيدًا في زاوية المدرسة، ولم يكن يشارك مع أي أحد. كان يوسف يشعر بالرهبة من هذا الطفل الغريب، لكنه قرر أن يتعرف عليه.

يوسف اقترب من الطفل الغريب وقال: "مرحبًا، هل تريد أن تلعب معي؟" لكن الطفل لم يرد. بدا الطفل غاضبًا ومحبطًا. يوسف شعر بالحزن، لكنه لم يفقد الأمل. قرر أن يبقى صديقًا له، حتى لو كان الطفل غريبًا.

مع مرور الوقت، بدأ يوسف يلاحظ أن الطفل الغريب كان يبدو أكثر سعادة. كان يوسف يشاركه ألعابًا جديدة ويقدم له بعض الأطعمة التي يحبها. الطفل الغريب بدأ يبتسم ويجيب على أسئلة يوسف.

ولكن، في يوم خريفي بارد، حدث شيء لم يكن يوسف متوقعًا. الطفل الغريب، الذي كان يُدعى سامي، أخبر يوسف أنه كان يخفي سرًا كبيرًا. سامي كان يخفي كرة القدم التي كان يلعب بها وحيدًا كل يوم، وكان يخفيها لأن أصدقائه كانوا يأخذونها دائمًا.

يوسف شعر بالرهبة والغضب. لقد أحسنت سامي وأعطته أصدقائه، لكن سامي لم يكن يريد أن يشاركهم. شعر يوسف بالحزن لأن سامي لم يكن يستحق صداقته.

لكن يوسف لم يفقد الأمل. قرر أن يشارك سامي كرة القدم، لكنه قرر أن يطلب شيئًا منه بدلاً من ذلك. "سامي، هل يمكنك أن تشاركي كرة القدم معي؟" قال يوسف. "ولكن، هل أنت متأكد أنك ستشاركي معي؟" سأل يوسف.

سامي نظر إلى يوسف وقال: "نعم، أنا متأكد. أنا آسف لأنني لم أشارك معك من قبل." يوسف شعر بالفرح، فقد تعلم درسًا مهمًا. "اتق شر من أحسنت إليه"، لكنه أيضًا تعلم أن الأمل والصداقة يمكن أن تغير كل شيء.

من ذلك اليوم فصاعدًا، شارك يوسف وسامي كرة القدم معًا، وهم يضحكون ويلعبان معًا. يوسف تعلم أن الأصدقاء الحقيقيين هم أولئك الذين يشاركونك الأوقات الجميلة، وليس أولئك الذين يخفيون سرًا منك.

وهكذا، انتهت قصتنا بسعادة، حيث تعلم يوسف درسًا قيمًا عن الصداقة والمرح، وأن الأمل والصداقة يمكن أن تغير كل شيء.

BLEURT Score: 0.25924623012542725
Semantic Coherence: 0.8849392533302307
Grammar Errors: 32
Distinct 1-gram Score: 0.5795454545454546
Distinct 2-gram Score: 0.9088319088319088
Perplexity: 1.864675760269165

Cleaned Story63:
في قرية صغيرة بالبحرين، كان يعيش صبي يُدعى يوسف مع صديقه الطائر النادر، زيزو. كان يوسف يحب زيزو كثيرًا وكان دائمًا يلاحظ كل شيء حولهم، من كيف يغرد زيزو في الصباح إلى كيف يتغير لون السماء في المساء.

ذات يوم، قرر يوسف أن يأخذ زيزو في مغامرة كبيرة. قال يوسف بحماس: "زيزو، هل تريد أن نذهب في مغامرة؟ يمكننا استكشاف الجزيرة التي لا يزورها أحد!"

رد زيزو بصوت مرتفع: "أه، أه! لا أعرف، يوسف. لكني أتمنى أن نعود قبل غروب الشمس."

"لا تقلق، زيزو. سأكون بجانبك دائمًا. دعنا نبدأ!" قال يوسف بثقة.

بينما كانا يتجولان، اكتشف يوسف وأصدقائه الكثير من الأشياء المدهشة. لاحظ كيف أن الأشجار تختلف في أشكالها، وكيف أن الرمال تختلف في ألوانها. ولكن، كان هناك شيء لم يلاحظه بعد.

فجأة، بدأت السماء تغيم وبدأت الرياح تعصف. قال يوسف: "زيزو، ربما يجب علينا العودة الآن. السماء تبدو غريبة."

"أه، أه! لا، يوسف. لا يمكننا العودة الآن. يجب أن نجد طريقة للعودة!" قال زيزو بقلق.

لكن يوسف كان يلاحظ شيئًا. لاحظ أن هناك شجرة كبيرة قريبة منهم، وكان يبدو أنها قد تكون مأوى. قال يوسف: "زيزو، انظر! تلك الشجرة يمكن أن تكون مأوى. دعنا نذهب إليها."

"أه، أه! لقد رأيتها! لقد كنت أتمنى أن نجد مأوى!" قال زيزو بفرح.

وصلوا إلى الشجرة ووجدوا مكانًا آمنًا. جلس يوسف وزيزو تحت الشجرة، يراقبان السماء. قال يوسف: "لقد كنت ألاحظ أن الشجرة كانت قريبة من هنا. لقد كنت أتمنى أن ألاحظ ذلك."

"أه، أه! لقد كنت ألاحظ ذلك أيضًا، يوسف. لكنك لاحظتها أولًا. لقد كنت أكثر ملاحظة!" قال زيزو.

"نعم، يجب أن أكون أكثر ملاحظة. لقد علمتني اليوم أن التحلي بالصبر والمراقبة يمكن أن يجلب السلام والسلامة." قال يوسف.

وبعد أن مرت الساعات، عاد الظلام إلى الليل، ولكن يوسف وزيزو كانا في سلام. عادوا إلى القرية، حيث استقبلهما أهل القرية بفرح.

"يوسف، لقد كنت شجاعًا جدًا. لقد علمتنا درسًا مهمًا اليوم." قال أحد أهل القرية.

"نعم، يجب أن نكون أكثر ملاحظة. وأنا سعيد لأن زيزو كان بجانبي." قال يوسف.

وهكذا، عاد يوسف وزيزو إلى البيت، محملين بالذكريات الجميلة والدرس الذي تعلماه. ومن ذلك اليوم فصاعدًا، أصبح يوسف أكثر ملاحظة، وكان دائمًا يلاحظ كل شيء حولهم، من السماء إلى الأرض، ومن الطبيعة إلى الأصدقاء.

BLEURT Score: 0.20098620653152466
Semantic Coherence: 0.8680461645126343
Grammar Errors: 38
Distinct 1-gram Score: 0.5982905982905983
Distinct 2-gram Score: 0.8971428571428571
Perplexity: 2.0221009254455566

Cleaned Story64:
في قلب اليمن الجميل، حيث السماء تتلون بألوان الليل، كانت هناك مكتبة صغيرة تحكي قصصًا لا تنتهي، والألغاز التي تنتظر من يجدها. في هذه المكتبة، كان يعيش طفل يُدعى ياسين، الذي كان يحب الألغاز أكثر من أي شيء آخر في العالم.

ياسين كان يحب المكتبة كثيرًا، وكان يذهب إليها كل يوم بعد المدرسة. كان يجلس تحت شجرة النخيل الكبيرة في فناء المكتبة، يقرأ ويحلم بالعالم الذي يخيله. كان يحلم بأن يصبح بطلًا يجد الألغاز ويحل المشكلات، لكنه كان يعلم أن هذه كانت مجرد حلم.

ذات يوم، بينما كان ياسين يجلس تحت شجرة النخيل، وجد كتابًا قديمًا لم يرَ مثله من قبل. كان الكتاب يحتوي على لغز لم يكن ياسين قد رأى مثله من قبل. كان اللغز يبدو صعبًا جدًا، لكن ياسين لم يفقد الأمل. بدأ يفكر في حل اللغز، يأمل أن يكون هذا هو البداية لحلمه.

مرت الأيام، وواصل ياسين محاولاته في حل اللغز. كان يحلم بأن يجد حلًا، وأن يصبح بطلًا حقيقيًا. لكن كل محاولة كانت تؤدي إلى طريق مسدود. كان ياسين حزينًا جدًا، لكنه لم يرثي الأمل.

في ليلة من الليالي، حلم ياسين بأن وجد حلًا للغز. كان حلمه واضحًا جدًا، حتى أنه شعر بأن اللغز قد حل من قبل. عندما استيقظ ياسين، شعر بالحماس. قرر أن يذهب إلى المكتبة ويعيد المحاولة مرة أخرى.

وصل ياسين إلى المكتبة، وبدأ يبحث عن الكتاب القديم. وجده في مكانه المعتاد تحت شجرة النخيل. فتح الكتاب، وبدا اللغز وكأنه قد تغير. كان ياسين متحمسًا جدًا، لكنه سرعان ما أدرك أن اللغز لم يكن قد تغير، بل كان يرى اللغز من منظور مختلف.

فجأة، وبينما كان ياسين يفكر في اللغز، شعر بشيء غريب. بدأ الكتاب يتحرك في يده، وكأن اللغز كان يريد أن يُحلل. كان ياسين مذهولًا، لكنه لم يستطع أن يفهم ما كان يحدث.

في تلك اللحظة، تحول حلمه إلى واقع. كان ياسين قد حل اللغز، لكنه لم يكن يعرف كيف. شعر بالامتنان والحزن في نفس الوقت. كان سعيدًا لأن حلمه أصبح حقيقة، لكنه كان حزينًا لأن اللغز لم يكن كما توقع.

في نهاية المطاف، أدرك ياسين أن الألغاز ليست فقط عن حل المشكلات، بل عن فهم العالم من حولنا. كانت هذه التجربة درسًا قيمًا في حياته، وأصبحت مكتبة تلك الشجرة النخيل مكانًا لا يُنسى في قلبه.

ومع ذلك، كانت نهاية قصة ياسين حزينة. فقد كان ياسين قد غادر المكتبة، ولم يعد إلى هناك مرة أخرى. لكن حكايته ظلت حية في قلوب كل من قرأ عنها، تذكيرًا بأن الألغاز والحلم يمكن أن يغيرا حياتنا.

BLEURT Score: 0.2233673632144928
Semantic Coherence: 0.8962301015853882
Grammar Errors: 4
Distinct 1-gram Score: 0.5714285714285714
Distinct 2-gram Score: 0.9181585677749361
Perplexity: 1.945608139038086

Cleaned Story65:
في قرية صغيرة ببلاد تونس، كان يعيش ثلاثة أصدقاء: ياسين، ليلى، وسليم. كانوا يحبون الاستكشاف والتعلم عن التاريخ والحضارات القديمة. ذات يوم، قرروا الذهاب في مغامرة لاستكشاف آثار مدينة قديمة كانت تقع على مشارف القرية.

"أصدقائي، هل أنتم مستعدين للعثور على كنز المدينة القديمة؟" قال ياسين بحماس.

"بالطبع، هذا سيكون مغامرة لا تنسى!" ردت ليلى.

"نعم، دعونا نبدأ!" قال سليم.

وصل الأصدقاء إلى الموقع الذي كانوا يبحثون عنه. كان هناك حجر كبير يبدو وكأنه بوابة. "هذا يجب أن يكون البوابة!" قال ياسين.

"لكن كيف نفتحها؟" سألت ليلى.

"ربما يجب أن نجد طريقة أخرى لفتحها"، اقترح سليم.

فكر الأصدقاء وبدأوا في البحث عن طريقة لفتح البوابة. بعد فترة من البحث والتفكير، وجدوا أنهم يجب أن يعملوا معاً ويكونوا داعمين لبعضهم البعض.

"أعتقد أننا نحتاج إلى العمل معاً!" قال ياسين.

"أتفق، يجب أن نكون فريق!" قالت ليلى.

"نعم، سنجد طريقة!" قال سليم.

بعد جهد وصدق، وجدوا أنهم يمكن أن يفتحوا البوابة عن طريق العمل معاً ووضع الأشياء في مواضعها الصحيحة. فتحوا البوابة، ووجدوا أنفسهم أمام كنز المدينة القديمة.

"لقد فعلناها! لقد وجدنا الكنز!" صرخ سليم.

"لكن الكنز ليس من الذهب أو الجواهر، بل هو المعرفة والتاريخ الذي علمنا!" قال ياسين.

"نعم، لقد علمنا درسًا قيمًا عن العمل معاً وثباتنا أمام الصعاب"، قالت ليلى.

"نعم، نحن نقرر كيفية الرد على المواقف الصعبة، ونحن قادرون على تحقيق أي شيء إذا عملنا معاً!" قال سليم.

عاد الأصدقاء إلى القرية، يحملون معهم الكنز الحقيقي: المعرفة والتاريخ. وأصبحوا مثالًا للشجاعة والعمل الجماعي في القرية.

وهكذا، كانت نهاية مغامرتهم سعيدة، وأدركوا أن قوة العمل معاً والثبات أمام الصعاب هي الكنز الحقيقي.

BLEURT Score: 0.21607518196105957
Semantic Coherence: 0.8713566064834595
Grammar Errors: 12
Distinct 1-gram Score: 0.7233201581027668
Distinct 2-gram Score: 0.9365079365079365
Perplexity: 1.6615697145462036

Cleaned Story66:
في قرية صغيرة جميلة في تونس، كان يعيش طفل يُدعى ياسين. ياسين كان لديه حلم كبير، وهو تعلم لغة جديدة، اللغة الإنجليزية. كان يحلم بأن يصبح مترجمًا ويعمل في مجال السياحة، ليكون قادرًا على التحدث مع الزوار من جميع أنحاء العالم.

كان الشتاء قد حلّ، وكانت السماء غائمة والرياح تهب بشدة. ياسين كان يجلس في غرفته الصغيرة، ينظر من النافذة إلى الخارج، يفكر في حلمه. كان يحب الشتاء، لكنه كان يأمل أن يكون قادرًا على السفر والاستكشاف في يوم ما.

في كل يوم، كان ياسين يجلس بجانب مكتبته الصغيرة، يقرأ الكتب والمذكرات عن اللغة الإنجليزية. كان يحاول تعلم الكلمات الجديدة والجمل الصحيحة، لكنه كان يشعر باليأس أحيانًا. كان ياسين يدرك أن تعلم لغة جديدة ليس سهلًا، وأن هناك الكثير من العمل الذي يجب القيام به.

مع مرور الأيام، كان ياسين يلاحظ تحسنًا في مهاراته اللغوية. كان قادرًا على التحدث بثقة أكبر، وبدأ يفهم الكثير من ما يقرأ. لكن، مع كل تقدم، كان يشعر بالحزن أيضًا. الحزن من أن لديه الكثير من الوقت للعمل والتحضير قبل أن يكون قادرًا على استخدام مهاراته في الحياة الحقيقية.

في أحد الأيام، بينما كان ياسين يجلس بجانب النافذة، نظر إلى الخارج ولاحظ أن السماء قد فتحت قليلًا، والشمس تلمع بين الغيوم. شعر بالحزن يغمره، لأنه كان يعلم أن هذا هو الشتاء في تونس، وأن الربيع قد لا يأتي قريبًا.

ياسين، الذي كان يأمل في يوم ما أن يصبح مترجمًا، شعر بالحزن لأن حلمه لم يكن سريعًا كما توقع. لكن، في نفس الوقت، كان يشعر بالامتنان لأن لديه حلمه، وأن لديه الكثير من الوقت للعمل عليه.

وهكذا، في غرفته الصغيرة، وسط الشتاء البارد في تونس، كان ياسين يدرك أن الحلم لا يأتي في اليوم التالي، لكنه يستحق العمل الجاد والصبر. وهكذا، بقيت حلمه يعشقه، ووعد نفسه بأن يظل يعمل بجد حتى يوم ما يكون قادرًا على تحقيق حلمه.

BLEURT Score: 0.21738658845424652
Semantic Coherence: 0.8795948624610901
Grammar Errors: 6
Distinct 1-gram Score: 0.5856164383561644
Distinct 2-gram Score: 0.865979381443299
Perplexity: 1.909319519996643

Cleaned Story67:
في قرية صغيرة جميلة في الكويت، حيث الأزهار تتفتح في فصل الربيع، كان يعيش طفل يُدعى يوسف. يوسف كان طفلاً شجاعًا ومبدعًا، لكنه كان يخاف من الغضب. كان يوسف يرى الغضب كأنه سحر يخلق من حولنا عالمًا من الندوب والدمار.

ذات يوم، بينما كان يوسف يلعب في الحديقة، شعر بغضب شديد لأنه لم يكن قادرًا على صنع لعبة جديدة كما كان يفعل دائمًا. شعور الغضب كان ينمو في قلبه، وكان يبدو أن كل شيء حول يوسف يتحول إلى أسوأ. فجأة، بينما كان يوسف يغضب، بدأت الأرض حول قدميه تتسع وتتوسع، وكأنها تريد أن تخفض من شدة غضبه.

فجأة، وبينما كان يوسف يغضب، ظهرت قوة خارقة غير متوقعة. كانت الأرض التي كانت تتوسع تحولت إلى حديقة جميلة مليئة بالأزهار والأشجار. كانت الأرض تحاول أن تعيد الصلح بين يوسف وطبيعة العالم.

يوسف، الذي كان يغضب بشدة، شعر بسعادة غامرة. لم يكن يفهم كيف حدث هذا، لكنه شعر بالسعادة والراحة. أدرك يوسف أن الغضب لم يكن فقط يدمر من حوله، بل كان أيضًا يدمر من داخله.

من ذلك اليوم فصاعدًا، قرر يوسف أن يتعلم كيف يتحكم في غضبه. أدرك أن الغضب يترك ندوبًا على الناس، لكنه أيضًا يمكن أن يترك ندوبًا على قلوبنا.

مع مرور الوقت، أصبح يوسف أفضل من أي وقت مضى. أصبح قادرًا على تحويل غضبه إلى قوة خارقة تحول العالم من حوله إلى مكان أجمل.

وهكذا، في قريته الجميلة في الكويت، أصبح يوسف بطلاً للقريه، ليس فقط لأنه كان شجاعًا ومبدعًا، ولكن أيضًا لأنه تعلم كيف يتحكم في غضبه، مما جعل من حوله أجمل وأكثر سعادة.

BLEURT Score: 0.22909002006053925
Semantic Coherence: 0.8657532930374146
Grammar Errors: 19
Distinct 1-gram Score: 0.6260162601626016
Distinct 2-gram Score: 0.9142857142857143
Perplexity: 1.7842564582824707

Cleaned Story68:
في إحدى قرى الإمارات العربية المتحدة الجميلة، حيث الشمس تلمع بألوانها الزاهية وتزين السماء بأشكالها المذهلة، كان يعيش طفل صغير يُدعى يوسف. يوسف كان يعيش وحيدًا في بيت صغير، وكان يحب اللعب في الحقول المحيطة ببيته، يركض بين الأزهار الملونة ويستمتع بجمال الطبيعة.

ذات يوم، بينما كان يوسف يلعب خارج البيت، لاحظ شيئًا غريبًا. كان هناك رجل غامض يرتدي ثوبًا أسود يبدو عليه الحزن. الرجل كان يبدو وكأنه يبحث عن شيء، لكنه لم يكن واضحًا. يوسف، بقلب طيب وروح شجاعة، قرر أن يسأل الرجل عن سبب غيابه.

"مرحبًا يا رجل، ماذا تفعل هنا؟" سأل يوسف بفضول.

"أنا... أنا البحث عن شيء،" أجاب الرجل بصوت خافت.

"ماذا تبحث عنه؟" سأل يوسف بفضول.

"لقد فقدت شيئًا ثمينًا جدًا،" قال الرجل بصوت حزين.

يوسف، الذي كان دائمًا يشارك قلبه مع الآخرين، قرر مساعدة الرجل. "لا تقلق يا رجل، سأجد ما تبحث عنه معك."

بدأت رحلة البحث معًا. يوسف، بكل شجاعة وقلب طيب، كان يساعد الرجل في البحث. لكن، في أثناء البحث، حدث شيء غير متوقع. الرجل، الذي كان يبدو كأنه شجاع، بدأ يظهر علامات القوة والشجاعة.

"أنا... أنا ليس أشرارًا كما تظن،" قال الرجل بصوت متحمس.

"من أنت؟" سأل يوسف بفضول.

"أنا حارس القرية السري،" قال الرجل بفخر.

"ولكن، لماذا كنت ترتدي ثوبًا أسود؟" سأل يوسف.

"لقد كنت أرتديه لكي لا أكون واضحًا،" أجاب الرجل.

يوسف، الذي كان دائمًا يشارك قلبه مع الآخرين، شعر بالسعادة لرؤية الرجل الحقيقي. "لقد كنت أبحث عنك، يا حارس القرية السري،" قال يوسف بابتسامة.

"شكرًا لك يا يوسف،" قال الرجل بابتسامة، "لقد كنت أبحث عنك أيضًا."

في النهاية، وجد الرجل ما كان يبحث عنه، وهو كنز القرية السري. لكن ما كان أكثر أهمية، كان هو صداقة جديدة بين يوسف والحارس السري.

"لقد علمتني أن الأشرار قد يكونون أبطالًا،" قال يوسف وهو ينظر إلى الحارس السري.

"ومن الأبطال قد يكونون أشرارًا،" رد الحارس بابتسامة.

وهكذا، انتهت قصتنا بسعادة، حيث يوسف والحارس السري أصبحا أصدقاء، وكل يوم كان يصبح أجمل من اليوم الماضي.

BLEURT Score: 0.2055344134569168
Semantic Coherence: 0.7610076665878296
Grammar Errors: 19
Distinct 1-gram Score: 0.6773162939297125
Distinct 2-gram Score: 0.9006410256410257
Perplexity: 1.7853254079818726

Cleaned Story69:
في يوم صيفي مشرق بمدينة ملاهي جميلة بمصر، كان هناك طفلان صغيران يُدعيان يوسف وعلي. كان يوسف يبلغ من العمر 7 سنوات وعلي يبلغ من العمر 5 سنوات. كانا يحبان اللعب معاً وكانا دائماً يبحثان عن مغامرات جديدة.

ذات يوم، قرر يوسف وعلي أن يذهبا إلى مدينة الملاهي ليلعبوا ألعاب الملاهي. كانا متحمسين جداً وقررا أن يبدأا يومهما بالركض على العجلات الدوارة. كان يوسف يعرف أن علي قد يكون خائفاً من العجلات الدوارة، لكنه قرر أن يغمره بفخر وثقة بأن علي يستطيع أن يركض على العجلات.

عندما وصلوا إلى العجلات الدوارة، بدأ يوسف يركض بسرعة. كان علي يراقب بقلق، لكنه قرر أن يتبع يوسف. كان يوسف يضحك ويصرخ من الفرح، بينما كان علي يبدو خائفاً ومتوتراً.

فجأة، توقف يوسف وأمسك بيد علي قائلاً: "علي، هل أنت متأكد من أنك تريد أن تنتقل إلى العجلات الدوارة؟" لكن علي، متأثراً بثقة يوسف، أجاب: "نعم، أنا مستعد."

بينما كانا يركضان على العجلات الدوارة، شعر علي بالخوف يزداد. كان يوسف يركض بسرعة، لكن علي كان يبدو متوتراً جداً. فجأة، توقف يوسف وأمسك بيد علي مرة أخرى. "علي، يجب أن نعود الآن. أنا خائف أيضاً."

علي، الذي كان يشعر بالخوف الشديد، شعر بالامتنان لثقة يوسف وقلبه الكبير. عاد يوسف وعلي إلى الأرض، حيث كان هناك مكان مظلل يبدو هادئاً ومريحاً.

"شكراً يا يوسف، لقد كنت خائفاً جداً. لقد كنت أتمنى لو استمعت لتحذيراتك." قال علي بقلب مفتوح.

"لا بأس، يا علي. أنا سعيد لأنك الآن تشعر بالراحة. أحياناً يجب أن نستمع إلى تحذيرات الأكثر حكمة." قال يوسف بابتسامة.

وهكذا، قضى يوسف وعلي بقية يومهما في مدينة الملاهي، يلعبون ألعاباً أخرى أقل خوفاً. تعلم علي درساً قيماً في ذلك اليوم، وهو أن الاستماع لتحذيرات الأكثر حكمة يمكن أن يجعلنا أمانيين وأكثر هدوءاً.

وهكذا، انتهى يومهما بضحكات وألعاب، وقلوبهما ممتلئة بالسعادة والامتنان.

BLEURT Score: 0.24345946311950684
Semantic Coherence: 0.8693170547485352
Grammar Errors: 44
Distinct 1-gram Score: 0.6526315789473685
Distinct 2-gram Score: 0.9190140845070423
Perplexity: 2.0804502964019775

Cleaned Story70:
في يوم صيفي حار في تونس، كانت الشمس تلمع على جدران المدرسة، وكانت الأطفال يتجمعون في الفصل. كان هناك طفل يُدعى ياسين، كان ياسين يحب الموسيقى والغناء كثيرًا، وكان دائمًا يحلم بأن يصبح مغنيًا مشهورًا.

في ذلك اليوم، قرر ياسين أن يشارك في مسابقة الغناء التي كانت ستقام في المدرسة. كان ياسين متحمسًا جدًا، لكنه كان يخاف من الفشل.

"ماذا لو لم أكن جيدًا؟" قال ياسين لنفسه وهو يتأمل.

لكن ياسين لم يسمح لخوفه أن يوقفته. بدأ يتدرب كل يوم بعد المدرسة، يغني بأقوى صوته ويتحسن ببطء.

"أنا سأكون جيدًا،" قال ياسين لنفسه بثقة.

جاء يوم المسابقة، وكل الطلاب كانوا متحمسين. جلس ياسين على المسرح، أخذ نفسًا عميقًا، وبدأ يغني.

كان صوته يملأ الفصل بألحان جميلة، وكانت الأطفال يصفقون بفرح. ياسين كان يغني من قلبه، وكان كل كلمة تخرج منه كلمات حقيقية.

بعد أن انتهى ياسين من الغناء، وقف الجميع صامتين لحظات، ثم بدأوا يصفقون بحرارة. ياسين كان سعيدًا جدًا، لم يكن يهمه الفوز أو الخسارة، بل كان سعيدًا لأنهم استمتعوا بغنائه.

"شكرًا لك يا ياسين، لقد كنت مذهلاً!" قال المعلم.

"شكرًا لك يا معلم،" أجاب ياسين بابتسامة عريضة.

في نهاية اليوم، عاد ياسين إلى المنزل، وهو يحمل في قلبه شعورًا بالفخر والسعادة. كان قد تعلم درسًا قيمًا: أن الشجاعة تكمن في محاولة الفعل، وأن الأهم هو أن نستمتع بما نحب.

ومنذ ذلك اليوم، أصبح ياسين مغنيًا مشهورًا في المدرسة، لكن أكثر من ذلك، أصبح بطلًا في قلوب أصدقائه.

BLEURT Score: 0.19921505451202393
Semantic Coherence: 0.8737980723381042
Grammar Errors: 7
Distinct 1-gram Score: 0.6986899563318777
Distinct 2-gram Score: 0.9605263157894737
Perplexity: 2.073284387588501

Cleaned Story71:
في جزر القمر الجميلة، حيث الأزهار تفتح بألوانها الزاهية في فصل الربيع، كان هناك ثلاثة أصدقاء صغار: ليلى، عمر، وفارس. كانوا يحبون اللعب مع حيواناتهم الأليفة، الأرنب "ميمي" والقط "فوفو"، في حديقة المنزل الكبيرة.

ذات يوم، بينما كان الأصدقاء الثلاثة يلعبون مع ميمي وفوفو، ظهرت في السماء كوكب صغير لم يروه من قبل. كان الكوكب يبدو مضيئًا بألوان زاهية، وكان الأصدقاء متحمسين جدًا لرؤية كوكب جديد.

فارس، الذي كان دائمًا يحب المغامرات، قال: "يجب أن نذهب ونزور هذا الكوكب! سيكون مغامرة لا تُنسى!".

ليلى، التي كانت دائمًا تحذر من المخاطر، قالت: "لكن ماذا لو كان هناك خطر هناك؟ يجب أن نستمع إلى تحذيرات الأكثر حكمة".

عمر، الذي كان يحب استكشاف الأشياء الجديدة، قال: "لا بأس، يمكننا أن نذهب ونستكشف. سيكون ممتعًا!"

فارس، الذي كان يحب المخاطرة، اقترح أن يذهبوا في رحلة إلى الفضاء الخارجي. ليلى، التي كانت قلقة، حاولت أن تمنعهم من الذهاب، لكن عمر، الذي كان متحمسًا جدًا، اقترح أن يأخذوا معهم ميمي وفوفو.

بعد الكثير من التفكير والمناقشة، قرر الأصدقاء الثلاثة أن يذهبوا في رحلة إلى الفضاء الخارجي، لكنهم وعدوا بأن يعودوا قبل غروب الشمس. قبل أن يغادروا، أخبرتهم أمهم، التي كانت الأكثر حكمة، بأن الفضاء الخارجي يمكن أن يكون مخيفًا وأن يجب أن يعودوا إذا شعروا بالخوف.

بينما كانوا في الفضاء، رأوا أشياء لم يرَوها من قبل، مثل النجوم والأشعة اللمعة. لكن بعد فترة قصيرة، بدأت السماء تغيم وبدأت الأشياء تبدو مخيفة. شعر الأصدقاء بالخوف، لكنهم تذكروا كلمات أمهم.

عندما عادوا إلى الأرض، شعروا بالراحة والسعادة. لقد تعلموا درسًا قيمًا: استمع لتحذيرات الأكثر حكمة. وأدركوا أن الحب والسلام هما ما يجعل الحياة سعيدة.

في تلك الليلة، قبل أن يناموا، قالت ليلى: "لقد كانت مغامرة رائعة، لكننا لقد تعلمنا درسًا قيمًا. يجب أن نستمع دائمًا لتحذيرات الأكثر حكمة".

ومنذ ذلك اليوم، أصبح الأصدقاء الثلاثة أكثر حكمة، وهم دائمًا يلعبون مع ميمي وفوفو، ولكنهم أيضًا يدركون أهمية الاستماع إلى تحذيرات الأكثر حكمة.

BLEURT Score: 0.21199876070022583
Semantic Coherence: 0.8807400465011597
Grammar Errors: 12
Distinct 1-gram Score: 0.6633663366336634
Distinct 2-gram Score: 0.8973509933774835
Perplexity: 2.033822774887085

Cleaned Story72:
في يوم ربيعي جميل في الجزائر، كان هناك طفل صغير يُدعى ياسين. ياسين كان يحب المغامرات والاستكشاف أكثر من أي شيء آخر. وفي يوم من الأيام، وجد ياسين جهازًا غريبًا في حديقة منزلهم. كان الجهاز يبدو كأنه جهاز فضائي!

فكر ياسين، "هل يمكنني استكشاف الفضاء الخارجي؟" وبدون تردد، قرر ياسين أن يذهب في مغامرة إلى الفضاء الخارجي. استخدم ياسين الجهاز الغريب الذي وجده، وسرعان ما وجد نفسه في الفضاء الخارجي.

كان ياسين متحمسًا جدًا، يطير حول النجوم والكواكب، يكتشف كل شيء حول نفسه. لكنه سرعان ما أدرك أن الفضاء الخارجي ليس كما توقع. كان هناك الكثير من المخاطر والتحديات. وبينما كان ياسين يستكشف، وجد نفسه أمام موقف ميؤوس منه: سفينة فضائية ضخمة كانت تقترب منه بسرعة.

شعر ياسين بالغضب والحزن. لم يكن يريد أن يعود إلى الأرض بعد أن اكتشف الفضاء. لكنه كان يعلم أن العودة هي خيار واحد فقط. فكر ياسين بسرعة ووجد حلًا مبتكرًا. استخدم جهاز الفضاء الذي وجده لإنشاء حقل مغناطيسي حول نفسه، مما جعله غير قابل للتعرض للضرب.

بفضل الحقل المغناطيسي، تمكن ياسين من تجاوز السفينة الفضائية الضخمة وعودته إلى الأرض بسلام. لكنه شعر بحزن شديد لأنه كان عليه العودة إلى الأرض بعد أن اكتشف الفضاء الخارجي.

عاد ياسين إلى الأرض، لكنه كان مختلفًا. كان ياسين الآن يعرف أن هناك عالمًا واسعًا خارج منزله، وأن المغامرات والاستكشاف يمكن أن تكون مثيرة ومفيدة. ولكن، كان ياسين حزينًا لأن مغامرته في الفضاء الخارجي لم تكن كما توقع.

ومنذ ذلك اليوم، أصبح ياسين أكثر حكمة، لكنه كان دائمًا يحلم باليوم الذي سيتمكن فيه من استكشاف الفضاء مرة أخرى.

BLEURT Score: 0.16610676050186157
Semantic Coherence: 0.8668497204780579
Grammar Errors: 1
Distinct 1-gram Score: 0.6546184738955824
Distinct 2-gram Score: 0.9233870967741935
Perplexity: 2.098787784576416

Cleaned Story73:
في يوم صيفي مشرق على سواحل فلسطين الجميلة، كان هناك صديقان لا يفترقان، سمكة صغيرة تُدعى لينا والبطل البحري زينو. كانا يلعبان بالقرب من الشاطئ، وكان زينو يحب لينا كثيرًا.

"لينا، هل تود أن تعرف سر المحيط السحري؟" سأل زينو بحماس.

"نعم، يا زينو! أتمنى أن أتعلمها!" أجابت لينا بحماس.

"السر هو أن كل شيء في المحيط يعمل معًا ليكون جميلًا. كل حيوان يلعب دورًا في الحفاظ على التوازن. هل تريد أن نلعب دورًا في هذا السر؟" قال زينو.

"نعم، سأكون سمكة صغيرة وأنت، يا زينو، ستكون البطل البحري!" ردت لينا.

وبذلك، بدأ الصديقان رحلتهم في البحث عن السر. كانوا يسبحان ويستكشفان كل زاوية من زوايا المحيط، يلتقون بأشكال من الحياة البحرية الغريبة، ويتعلمان كيف يعمل كل شيء معًا ليكون جميلًا.

"انظري، لينا! هذه الأسماك الصغيرة تلعب دورًا في تنظيف المحيط، وهذه الأصداف تعمل كمنازل للحيوانات البحرية الصغيرة!" قال زينو.

"وأنا، يا زينو، سأكون سمكة صغيرة وأساعد في الحفاظ على التوازن في المحيط!" قالت لينا بحماس.

بعد يوم من البحث والاستكشاف، وجد زينو ولينا السر المحيط السحري. كانت جميلة جدًا، وكانت كل شيء يعمل معًا ليكون جميلًا.

"لينا، لقد فعلناها! نحن الآن جزء من السر المحيط السحري!" قال زينو بفخر.

"نعم، يا زينو! وأنا سعيدة جدًا لأننا تعلمنا هذا معًا!" ردت لينا.

وبذلك، عاد الصديقان إلى الشاطئ، حيث كانوا يلعبون كل يوم. وأصبحوا بطلين في المحيط، يحافظان على التوازن والجمال في المحيط السحري.

"لينا، أتمنى أن تكون دائمًا سعيدة في المحيط السحري!" قال زينو.

"وأنا، يا زينو، أتمنى أن نكون دائمًا أصدقاء!" ردت لينا.

وهكذا، عاش الصديقان سعيدين إلى الأبد، يحافظان على جمال المحيط السحري.

BLEURT Score: 0.20025858283042908
Semantic Coherence: 0.8743311762809753
Grammar Errors: 17
Distinct 1-gram Score: 0.5882352941176471
Distinct 2-gram Score: 0.8543307086614174
Perplexity: 1.8457857370376587

Cleaned Story74:
في قرية صغيرة بالكويت، حيث الشتاء يغطي الأرض بثوبه الأبيض، كان يعيش طفل صغير يُدعى يوسف مع والده. يوسف كان يحب والده كثيرًا، لكنه كان يشعر بالغضب من والده دائمًا، لأنه كان يعتقد أن والده يأخذه إلى الأماكن المظلمة والخوفية التي تحكي بها الأساطير والخرافات.

ذات يوم، قرر والده يوسف أن يأخذ يوسف في مغامرة خاصة، قائلاً إنه سيظهر له أسرارًا من الأساطير. كان يوسف متحمسًا لكنه كان يخشى أيضًا. بينما كانا يتجولان في الغابة، بدأ يوسف يشعر بالبرد والخوف، لكنه لم يكن يرغب في الاعتراف بهذا.

فجأة، ظهرت أمامه شخصية شريرة، كانت تُدعى "الجنية الشريرة". كانت الجنية تُخيف الناس في القرية، وكانت تُعتبر مصدر الخوف والغضب. يوسف شعر بالغضب الشديد تجاه الجنية، لكن والده بدا عليه شعورًا بالحزن والأسف.

"لماذا أنت هنا، يا جنية الشريرة؟" سأل يوسف بصوت مرتفع.

"أنا هنا لأنني أريد أن أظهر لك أن الحب لا يوجد فقط في الأماكن السعيدة، بل في كل مكان، حتى في الأماكن التي يخشى منها الناس"، أجابت الجنية بصوتها العميق.

فجأة، تحولت الجنية إلى والده، وأظهرت يوسف أن والده كان في الحقيقة الجنية الشريرة. كان والده يحاول أن يظهر له أهمية الحب والرحمة، وأن الحب لا يزال موجودًا حتى في الأماكن التي يخشى منها الناس.

يوسف شعر بخيبة أمل كبيرة، لأنه كان يعتقد أن والده كان يخفي الحقيقة عنده. لكنه في النهاية، أدرك أن والده كان يحاول أن يتعلم منه درسًا مهمًا.

"أنا آسف يا يوسف، لم أكن أريد أن أؤذيك"، قال والده بدموع في عينيه.

"أنا آسف أيضًا يا أبي"، قال يوسف وهو يبكي. "أنا لا أريد أن أكون غاضبًا منك أبدًا".

ومن ذلك اليوم فصاعدًا، أصبح يوسف أكثر حبًا وأكثر فهمًا لما يحدث حوله. لكن الحزن ظل يخيم على قلبه، لأنه أدرك أن الحب والرحمة لا يمكن أن يأتيان دائمًا مع السعادة، بل يمكن أن يأتيان مع الألم والخيبة.

وهكذا، تعلم يوسف درسًا قيمًا عن الحب والرحمة، لكنه تعلم أيضًا أن الحب يمكن أن يكون حزينًا، وأن الأماكن التي نحبها قد لا تكون دائمًا ملاذًا من الألم.

BLEURT Score: 0.17990940809249878
Semantic Coherence: 0.8726420998573303
Grammar Errors: 21
Distinct 1-gram Score: 0.6175548589341693
Distinct 2-gram Score: 0.9056603773584906
Perplexity: 2.0282394886016846

Cleaned Story75:
في إحدى مكتبات السودان الجميلة، حيث الأشجار تتساقط أوراقها بألوان الذهب والبرتقالي، كان هناك ثلاثة أصدقاء: يوسف، والببغاء زيزو، والكلب توتو. كانوا يحبون المغامرات والاستكشاف، وكانوا دائمًا يبحثون عن مغامرة جديدة.

ذات يوم، بينما كان الثلاثة يلعبون داخل المكتبة، سمعوا صوتًا غريبًا يأتي من وراء أحد الكتب. "من هناك؟" صرخ يوسف بحماس.

"أنا... أنا هنا!" أجاب الصوت بصوت خافت.

"من أنت؟" سأل زيزو بفضول.

"أنا... أنا الشخص الذي يخفي الكتب السرية هنا." قال الصوت.

"هل يمكنك أن تظهر نفسك؟" سأل توتو بحذر.

"نعم، لكن يجب أن تكونوا مستعدين للعثور على مغامرة لا مثيل لها." قال الصوت.

فجأة، ظهر شخص غامض يرتدي قميصًا أسود ويرتدي قبعة كبيرة. "أنا... أنا الشخص الذي كنت تخافون منه." قال الشخص الغامض.

"من أنت؟" سأل يوسف بقلق.

"أنا... أنا والدك، يا يوسف." قال الشخص الغامض بصوت حزين.

"كيف يمكن أن تكون؟" سأل يوسف مذهولًا.

"أنا كنت أبحث عن طريقة لجعل المكتبة مكانًا ممتعًا للأطفال، لكنني فعلت ذلك بطريقة غامضة جدًا. أتمنى أن أكون قد أظهرت لكما أنني أريد فقط أن أكون صديقًا لكما." قال والد يوسف بحزن.

"لكن، يا أبي، أنت أفضل صديق يمكن أن يكون!" قال يوسف وهو يبكي من الفرح.

"أنا سعيد جدًا، يا يوسف. وأنا سعيد جدًا لأنكما أردتمني." قال والد يوسف وهو يبكي أيضًا.

"ولكن، يا أصدقاء، لا يجب أن ننتهي هنا. يجب أن نستكشف المكتبة أكثر!" قال زيزو بفضول.

"أعلم، يا زيزو. دعونا نبدأ مغامرة جديدة!" قال توتو بابتسامة.

"نعم، دعونا نبدأ!" قال يوسف وهو يضحك.

وهكذا، انتهت مغامرة اليوم بصداقة أقوى وأكثر إثارة. والجميع، بما في ذلك الشخص الغامض الذي كان يخفي الكتب السرية، أصبحوا أصدقاء.

ومنذ ذلك اليوم، أصبح يوسف وزيزو وتوتو والكتب السرية أصدقاء لا يفترقون. وكانوا دائمًا يبحثون عن مغامرات جديدة في المكتبة، مستخدمين الكتب السرية كدليل لاستكشاف العالم من حولهم.

وهكذا، انتهت القصة بسعادة، حيث أصبح الأصدقاء الأربعة أصدقاء لا يفترقون، واكتشفوا أن المغامرات والاستكشاف يمكن أن يكونا أجمل بكثير عندما يكون هناك أصدقاء يشاركونك في المغامرة.

BLEURT Score: 0.23317739367485046
Semantic Coherence: 0.6969677805900574
Grammar Errors: 26
Distinct 1-gram Score: 0.6720257234726688
Distinct 2-gram Score: 0.9032258064516129
Perplexity: 1.847748041152954

Cleaned Story76:
في قرية صغيرة جميلة في قطر، كان يعيش طفل يُدعى يوسف. يوسف كان طفلاً ذكيًا ومحبًا للمعرفة، وكان يحب مشاهدة مقاطع الفيديو التعليمية على جهازه اللوحي بعد المدرسة. كان يوسف يحلم دائمًا بأن يصبح عالمًا مشهورًا يومًا ما.

ذات يوم، أثناء مشاهدة يوسف مقطع فيديو عن الكيمياء، ظهرت شخصية غامضة على الشاشة. كانت شخصية شريرة تحاول إعاقة يوسف من فهم الكيمياء. شعر يوسف بالقلق والغضب، لكنه لم يفقد الأمل. بدلاً من ذلك، قرر يوسف أن يجد طريقة لمعرفة من وراء شخصية الشريرة هذه.

بعد أيام من البحث والتحليل، اكتشف يوسف تحولًا غير متوقع في الأحداث. الشخصية الشريرة التي كان يخشاها كانت في الحقيقة والد يوسف! كان والده يريد أن يثبت لابنه أن الصداقة والتعاون هما الطريق الصحيح نحو النجاح.

شعر يوسف بالدهشة والفخر في الوقت نفسه. لقد كان والده يريد أن يظهر له أن الصداقة والتعاون يمكن أن يكونا قويين جدًا حتى مع من يبدو شريرًا.

بعد ذلك، أصبح يوسف وأبيه أصدقاء مقربين. كانا يشاركان في مشاهدة مقاطع الفيديو التعليمية معًا، يتعلمان ويضحكان. وأصبح يوسف أكثر شجاعة وأكثر إصرارًا في حلمه بأن يصبح عالمًا مشهورًا.

الدرس المستفاد من هذه القصة هو أن الصداقة يمكن أن تظهر في أشكال غير متوقعة، وأن التعاون والصداقة يمكن أن يجلبان السعادة والنجاح. وأصبح يوسف وأبيه مثالًا يحتذى به في القرية كله.

BLEURT Score: 0.22268187999725342
Semantic Coherence: 0.8595041632652283
Grammar Errors: 16
Distinct 1-gram Score: 0.6956521739130435
Distinct 2-gram Score: 0.9223300970873787
Perplexity: 2.0085608959198

Cleaned Story77:
في أحد أيام الصيف الحارة في قطر، كان هناك ثلاثة أصدقاء صغار يدعون يوسف، ليلى، وسامر. كانوا يحبون العلوم كثيرًا وكانوا دائمًا يبحثون عن مغامرات جديدة. في يوم ما، قرروا زيارة المختبر العلمي في مدرستهم لاستكشاف الأجهزة والكتب العلمية.

بينما كانوا يتنقلون بين الأجهزة، وجدوا جهازًا غريبًا يبدو وكأنه من عالم آخر. كان الجهاز يبدو كأنه ينتظر أمرًا لبدء تجربة. كان يوسف، الذي كان دائمًا شغوفًا بالاستكشاف، يشعر بالفضول الشديد تجاه الجهاز الغريب. قرر أن يفعل شيئًا مبدعًا وبدأ بتغيير بعض الإعدادات دون أن يلاحظ ليلى وسامر.

فجأة، بدأت الأجهزة في المختبر تتصرف بشكل غير متوقع. بدأت الأضواء تلمع بقوة، وأصوات غريبة بدأت تخرج من الأجهزة. شعر الأصدقاء بالقلق الشديد، لكن يوسف، الذي كان يشعر بالترقب، قال: "ربما هذا ما كان ينتظره الجهاز منا!"

في تلك اللحظة، حدث شيء غير متوقع. بدأت الأجهزة تخرج ببطء من المختبر، كأنها تريد أن تخرج إلى العالم الخارجي. كان الأصدقاء في وضع ميؤوس منه، لكنهم لم يفقدوا الأمل.

ليلى، التي كانت تعرف الكثير عن العلوم، تذكر شيئًا عن الأجهزة التي يمكنها أن تتحكم في المادة حولها. فكرت بسرعة ووجدت حلًا. قالت: "لنستخدم الأجهزة لجعل المادة حولنا أكثر ملاءمة!"

باستخدام الأجهزة، تمكن الأصدقاء من تحويل المادة المحيطة بهم إلى مادة ملونة ومرحة، مما جعل المختبر يبدو وكأنه مكان من عالم الألعاب. استطاعوا بذلك أن يخلقوا بيئة ممتعة حتى يعودوا إلى الوضع الطبيعي.

عاد الأصدقاء إلى المنزل، متعبين لكن سعيدين. تعلموا درسًا قيمًا عن العلم والاستكشاف، وأدركوا أن العلم يمكن أن يكون ممتعًا وملهمًا. ومنذ ذلك اليوم، أصبحوا أكثر شغفًا بالعلوم والاستكشاف، وهم دائمًا يتذكرين المغامرة المذهلة في المختبر.

وهكذا، انتهت مغامرتهم بسعادة وامتنان، مدركين أن العلم يمكن أن يكون مصدرًا للفرح والمرح.

BLEURT Score: 0.2762882709503174
Semantic Coherence: 0.8751937747001648
Grammar Errors: 6
Distinct 1-gram Score: 0.7191011235955056
Distinct 2-gram Score: 0.9586466165413534
Perplexity: 2.0053367614746094

Cleaned Story78:
في أحد أيام الخريف الجميلة بالكويت، كان هناك طفل صغير يُدعى يوسف. يوسف كان يحب لعب ألعاب الطاولة كثيرًا، لكنه كان دائم الشعور بالوحدة. كان يلعب وحيدًا في غرفته الصغيرة، يتمنى لو كان لديه صديق يشاركه ألعاب الطاولة.

ذات يوم، بينما كان يوسف يلعب لعبة شطرنج، سمع صوتًا يأتي من خارج غرفته. كان ذلك صوتًا يأتي من خارج نافذة غرفته. فتح يوسف النافذة ليجد طائرًا صغيرًا يبدو أنه فقد طريقه.

بدلاً من التركيز على الوحدة، قرر يوسف أن يساعد الطائر الصغير. بدأ يوسف بتغذية الطائر ووضع ماء له. كان الطائر يعود كل يوم، وكان يوسف سعيدًا جدًا لرؤيته.

مع مرور الأيام، بدأ يوسف يلاحظ تغييرًا داخله. كان يشعر بالسعادة والرضا، ولم يكن يشعر بالوحدة مرة أخرى. كان الطائر صديقه الجديد، وكان يوسف سعيدًا جدًا بوجود صديق يشاركه أوقاته.

في أحد الأيام، قرر يوسف أن يلعب لعبة شطرنج مع الطائر. كان يوسف يضع قطع الشطرنج على الأرض، وكان الطائر يلعب بفضوله. كان يوسف يضحك كثيرًا وهو يلعب مع صديقه الجديد.

كانت هذه اللحظة سعيدة جدًا بالنسبة ليوسف. كان يوسف قد تعلم أن الصداقة يمكن أن تأتي من الأماكن غير المتوقعة وأن الحب والمرح يمكن أن يأتي من اللعب مع الآخرين، حتى لو كانوا من نوع مختلف.

وهكذا، انتهى يوم يوسف بسعادة ومرح، مدركًا أن التعلم والنمو الشخصي يمكن أن يحدثان في أي وقت، حتى في فصل الخريف، وأن الحب يمكن أن يجمع بين الناس، بغض النظر عن الخلفيات أو النوع.

ومنذ ذلك اليوم، كان يوسف دائمًا يشارك حبه للعب ألعاب الطاولة مع صديقه الطائر، وكل يوم كان يصبح أجمل من اليوم السابق.

BLEURT Score: 0.22277633845806122
Semantic Coherence: 0.8813453912734985
Grammar Errors: 23
Distinct 1-gram Score: 0.6414342629482072
Distinct 2-gram Score: 0.904
Perplexity: 1.8404159545898438

Cleaned Story79:
في قرية صغيرة جميلة في جيبوتي، كان يعيش طفل صغير يُدعى يوسف. كان يوسف يحب فصل الشتاء كثيرًا، حيث السماء الزرقاء تلوّن الأرض بألوانها الذهبية والبرتقالية، والأشجار تتحول إلى ألوانها الذهبية والبنية. لكن يوسف لم يكن يحب الشتاء فقط لألوانه الجميلة، بل لأنه كان يحب السير في الطبيعة والتعلم عن العالم من حوله.

كل يوم بعد المدرسة، كان يوسف يخرج في مغامرة جديدة ليكتشف شيئًا جديدًا. كان يحب أن يلاحظ كيف تغير العالم من يوم إلى يوم، وكيف ينمو كل شيء حوله. لكن يوسف كان يشعر بالقلق دائمًا، لأنه لم يكن يعرف كيف يعود إلى البيت. كان يوسف يخاف أن يخسر الطريق ويعود إلى البيت متأخرًا.

ذات يوم، قرر يوسف أن يذهب في مغامرة إلى أعلى تل في القرية. كان يوسف يحب أن يرى من أعلى التل، لكنه كان يخاف من العودة. وبينما كان يوسف يركض في الطريق، بدأ يوسف يشعر بالبرد. كان الشتاء في جيبوتي باردًا، وكان يوسف لا يملك معطفًا.

بينما كان يوسف يركض، لاحظ يوسف أن هناك عصفور صغير يغرد بجانبه. كان العصفور يغرد بحرية، وكأنه يأمل يوسف في العودة إلى البيت. شعر يوسف بالدفء والحب من العصفور، فقرر يوسف أن يعود إلى البيت.

عندما وصل يوسف إلى البيت، كان والديه ينتظران بقلق. كانا يخافون أن يوسف قد يخسر الطريق. لكن يوسف كان سعيدًا جدًا لأنه عاد سالمًا.

والدي يوسف نظر إليه وقال: "يوسف، نحن فخورون بك لأنك عادتم إلى البيت سالمًا. لكننا علمت أنك تعلمت درسًا مهمًا اليوم. الدرس هو أننا يجب أن نكون بروّرًا لوالدينا ونستمع إلى نصائحهم."

كانت النهاية سعيدة، حيث كان يوسف سعيدًا لأن والديه كانا فخورين به، وكان يوسف سعيدًا لأنهم علّموه درسًا مهمًا. ومن ذلك اليوم فصاعدًا، كان يوسف دائمًا يعود إلى البيت قبل المساء، وكان يسمع نصائح والديه بقلب مفتوح.

BLEURT Score: 0.25317999720573425
Semantic Coherence: 0.8793078660964966
Grammar Errors: 31
Distinct 1-gram Score: 0.6142857142857143
Distinct 2-gram Score: 0.9068100358422939
Perplexity: 2.0315918922424316

Cleaned Story80:
في يوم مشمس جميل، في حديقة صغيرة وجميلة بمدينة تونس، كان هناك ثلاثة أصدقاء: الطائر الأزرق يوسف، والقط الصغير سليم، والببغاء الأصفر سارة. كانوا يلعبون ويضحكون معًا كل يوم، وكانوا أفضل الأصدقاء.

ذات يوم، قرر يوسف أن يصنع لعبة جديدة. جمع بعض الأوراق والأغصان وأعادها إلى الحديقة. كان يوسف متحمسًا جدًا، لكنه كان يخاف من أن يؤذي أحد أصدقائه.

فجأة، بينما كانوا يلعبون، انزلقت لعبة يوسف وضربت سارة ببغاء الأصفر. سارة كانت تعيش في حديقة صغيرة، وكانت تحب اللعب مع أصدقائها. لكنها أصيبت بجروح خطيرة.

يوسف وسانيم شعرا بالحزن الشديد. سارة كانت صديقتهم المفضلة، وكانت لا يمكنهم تخيل حياة بدونها. حاولوا جاهدًا أن يصلحوا ما حدث، لكن سارة كانت تعاني كثيرًا.

بعد أيام، وبعد جهد كبير، قرر يوسف وسانيم أن يأخذوا سارة إلى الطبيب. الطبيب قال إن سارة كانت تعاني من جروح عميقة، ولكنها قد تعيش إذا كانت تحتاج إلى الرعاية الدائمة.

يوسف وسانيم شعروا بالحزن الشديد. كانوا يأملون أن يعودوا إلى اللعب مع سارة، لكن الطبيب قال إنها قد لا تستطيع اللعب مرة أخرى.

في النهاية، كانت سارة تعيش في بيت صغير، حيث كان يوسف وسانيم يزورونها كل يوم. كانوا يحبونها كثيرًا، ولكنهم كانوا حزينين لأنها لم تعد تستطيع اللعب معهم.

كانت القصة حزينة، لكنها تعلمت أصدقاء يوسف وسانيم أن الحب والصداقة لا ينتهي، حتى وإن كانت هناك نهاية حزينة.

BLEURT Score: 0.25134581327438354
Semantic Coherence: 0.8742682337760925
Grammar Errors: 20
Distinct 1-gram Score: 0.6777251184834123
Distinct 2-gram Score: 0.919047619047619
Perplexity: 1.743570327758789

Cleaned Story81:
في قرية صغيرة جميلة في تونس، كان يعيش ثلاثة أصدقاء: ياسمين، عمر، وسارة. كانوا يحبون البستنة كثيرًا وكانوا يشاركون في مسابقة البستنة السنوية. لكن هذه السنة، كانت هناك مشكلة كبيرة؛ الأرض كانت جافة جدًا ولم يكن هناك ماء كفاية لزراعة النباتات.

يومًا ما، اجتمع الأصدقاء الثلاثة وقالوا: "لنحاول أن نجد حلًا معًا!" قرروا أن يبحثوا عن ماء في كل مكان. عمر، الذي كان يعرف الكثير عن الطبيعة، اقترح أن يذهبوا إلى النهر القريب. ولكن، عندما وصلوا إلى النهر، وجدوا أنه جاف أيضًا.

ياسمين، التي كانت تحب القراءة، قرأت عن كيفية استخدام الماء المذاق الذي يمكن أن يجدوه في الطبيعة. فكرة جميلة! لكن كيف سيتم جمع الماء المذاق؟ سارة، التي كانت تحب الفن، اقترحت أن يصنعوا خيوطًا من القماش لتجميع الماء المذاق.

بعد يوم من العمل الجاد، نجحوا في صنع خيوط القماش وأخذوا الماء المذاق. لكن، عندما عادوا إلى القرية، وجدوا أن الأرض كانت لا تزال جافة. فجأة، وقعت سارة على فكرة أخرى. "لماذا لا نصنع سدًا صغيرًا من الطين ونجمع الماء المذاق فيه؟" اقترحت.

بكل شجاعة، بدأ الأصدقاء الثلاثة العمل على السد. كانوا يضعون الطين وجمعون الماء المذاق. وبعد ساعات من العمل الشاق، انتهوا. وهم ينظرون إلى السد، بدأت السماء تغيم. فجأة، بدأت الأمطار تتساقط!

كانت السماء قد تغيرت تمامًا! الأصدقاء الثلاثة لم يصدقوا عينيهما. جمعوا الماء المذاق في السد، وسرعان ما بدأت الأرض تغير. النباتات بدأت تنمو، والقرية أصبحت أجمل من ذي قبل.

وفي اليوم التالي، اجتمع الناس من القرية ليرى ما حدث. كان الجميع فخورًا بالأصدقاء الثلاثة. وبينما كانوا يشاهدون النباتات النامية، ظهرت فكرة جديدة. "لماذا لا نصنع مسرحية عن شجاعة العمل الجماعي والتعاون؟" اقترحت ياسمين.

وأصبح الأصدقاء الثلاثة بطلين المسرحية. وبينما كانوا يؤدون، كان الجميع يضحك ويتعلم. كانت القصة كلها جزءًا من تمثيلية، ولكن الدرس الذي تعلموه كان حقيقيًا. الشجاعة والعمل الجماعي والتعاون يمكن أن يغيرا كل شيء.

وهكذا، انتهت القصة بسعادة، مع درس قيم يُحفظ في قلوب الجميع.

BLEURT Score: 0.23684465885162354
Semantic Coherence: 0.8904235363006592
Grammar Errors: 7
Distinct 1-gram Score: 0.6821192052980133
Distinct 2-gram Score: 0.946843853820598
Perplexity: 2.053220272064209

Cleaned Story82:
في قرية صغيرة في اليمن، كان يعيش طفل يُدعى ياسين. ياسين كان طفلاً شجاعًا وملاحظًا، يحب استكشاف كل زاوية في القرية. كان ياسين يعيش وحيدًا، لكنه كان لديه صديق مقرب جدًا، وهو الجمل الذي كان يسمّيه "أحمد".

يومًا ما، بينما ياسين يلعب مع أحمد، قال: "أحمد، هل تظن أن هناك كنزًا مخبأً في هذه القرية؟"

أحمد، الجمل، نظر إلى ياسين بعينيه الكبيرة وقال: "لا أعلم يا ياسين، لكن يمكننا البحث عن ذلك معًا."

بدأ ياسين وأحمد رحلتهما في البحث عن الكنز. كان ياسين يتحدث كثيرًا عن الكنز الذي يمكن أن يكون، وأحمد يراقب ياسين بعينيه المليئة بالفضول.

بعد أيام من البحث، وجد ياسين وأحمد كنزًا صغيرًا تحت شجرة عتيقة. كان الكنز عبارة عن مجموعة من الكتب القديمة. ياسين، متحمسًا، قال: "هذا الكنز! لقد وجدنا الكنز!"

لكن عندما بدأ ياسين يقرأ الكتب، أدرك أن الكنز الحقيقي كان في معرفته ودراسته، وليس في الكنوز المادية. شعر ياسين بالحزن لأن الكنز لم يكن كما توقع.

في تلك اللحظة، تحولت نظرته إلى أحمد، الذي كان ينظر إليه بقلق. قال ياسين: "أحمد، لقد أدركت شيئًا. الكنز الحقيقي كان في معرفتي ودراستي، وليس في الكنوز المادية."

أحمد، الجمل، نظر إلى ياسين بعينيه الكبيرة وقال: "أنا فخور بك يا ياسين، لكني أخشى أنني قد أصبح عائقًا لك."

ياسين، بقلب ممتلئ بالحزن، قال: "لماذا يا أحمد؟"

أحمد، الجمل، أجاب: "لأنني لم أكن دائمًا هناك لاستمع إليك، بل كنت دائمًا هناك لاستمع إلى كلماتك فقط."

ياسين، وهو يفهم الدرس، قال: "أنا سأكون أكثر ملاحظة، وأتحدث أقل. شكرًا لك أحمد، لقد علمتني درسًا قيّمًا."

وفي تلك اللحظة، تحول الصديق المقرب إلى خصم، ليس لأن أحمد لم يكن صديقًا جيدًا، بل لأن ياسين أدرك أن الصداقة الحقيقية تكمن في الاستماع والفهم، وليس في الكلمات فقط.

وهكذا، انتهى يوم ياسين وأحمد، الصديق المقرب الذي تحول إلى خصم، لكن ياسين تعلم درسًا لا يُنسى عن أهمية الاستماع والتفكير قبل الكلام.

BLEURT Score: 0.1938076913356781
Semantic Coherence: 0.8592492938041687
Grammar Errors: 6
Distinct 1-gram Score: 0.6296296296296297
Distinct 2-gram Score: 0.9054054054054054
Perplexity: 1.8403944969177246

Cleaned Story83:
في قرية صغيرة في فلسطين، كان يعيش ثلاثة أصدقاء: يوسف، ليلى، وسامر. كانوا يحبون طهي وصفات بسيطة، وكانوا دائماً يبحثون عن تحدي جديد في عالم الطهي.

ذات يوم، قرروا أن يشاركوا في مسابقة الطهي الكبرى التي تقام في القرية كل عام. كانوا متحمسين جدًا، لكنهم كانوا يخافون من الفشل.

"ماذا لو لم نتمكن من تحضير الوصفة بشكل جيد؟" قال يوسف بقلق.

"لا تخفوا، نحن سنجد طريقة!" ردت ليلى بثقة.

"نعم، والشجاعة هي ما يجعلنا نتمكن من تحقيق أحلامنا!" أضاف سامر.

في ليلة قبل المسابقة، حلم يوسف بأنهم يفوزون بالمسابقة، لكنه كان يفقد الوصفة. حلم ليلى بأنهم يفوزون لكنهم يخسرون الوقت. وسامر، كان يحلم بأنهم يفوزون لكنهم يخافون من تقديم الوصفة.

في الصباح التالي، استيقظ الأصدقاء الثلاثة وهم يشعرون بالقلق. لكنهم قرروا أن يفوزوا بالمسابقة، مهما كان.

"نحن سنجعل هذا حلمنا يتحول إلى واقع!" قال يوسف بثقة.

بدأوا بالعمل على الوصفة، وكل واحد كان يعمل بجد. وفي نهاية المطاف، كانت الوصفة التي أعدوها أجمل وذوقية.

"أنا فخور بكم!" قال الحكم، وهو يأخذ قطعة من الوصفة.

"نحن فخورين جدًا!" رد الأصدقاء الثلاثة معًا.

فازوا بالمسابقة! كانت نهاية سعيدة، والأصدقاء الثلاثة كانوا يضحكون ويتحدثون عن كيفية تحويل حلمهم إلى واقع.

"لقد علمتنا الشجاعة أننا نتمكن من تحقيق أحلامنا!" قال سامر.

"نعم، والعمل الجاد والثقة بالنفس هي مفتاح النجاح!" أضافت ليلى.

"وأنا سعيد جدًا لأننا فازنا معًا!" قال يوسف.

وهكذا، تعلم الأصدقاء الثلاثة درسًا قيمًا عن الشجاعة والعمل الجاد، وأصبحوا أبطالاً في قلوب أهل القرية.

BLEURT Score: 0.24019721150398254
Semantic Coherence: 0.8843424320220947
Grammar Errors: 10
Distinct 1-gram Score: 0.7316017316017316
Distinct 2-gram Score: 0.9478260869565217
Perplexity: 1.7223854064941406

Cleaned Story84:
في مختبر علمي صغير في الأردن، كان يعمل طفلان صغيران، ياسمين وطارق. كانا يحبان التجربة والاكتشاف، وكانا دائماً ما يبحثان عن مهن جديدة ليكتشفاها.

في يوم صيفي حار، قال طارق: "ياسمين، أتمنى لو أستطيع أن أكون عالمًا في المستقبل، ليكتشف الكثير من الأشياء الجديدة!"

"أنا أيضًا،" أجابت ياسمين بفضول. "لكن ماذا لو أصبحت طبيبة؟"

"أوه، لقد ستعرف كيف تعالج الناس!" قال طارق بفخر.

في ذلك اليوم، قررا تجربة شيء جديد. اخترقا مختبرهما الصغير وبدأا بتجربة صنع صابون من الزيت والبذور. كانا متحمسين جدًا، لكنهما لم يكن لديهما معرفة كافية بالمواد التي كانوا يستخدمونها.

"ياسمين، هل أنت متأكدة من أن هذا صابون آمن؟" سأل طارق وهو يراقب الصابون يتساقط على الأرض.

"لا،" أجابت ياسمين بقلق. "ربما يجب علينا أن نكون أكثر حذرًا."

لكن قبل أن يتمكنا من إيقاف التجربة، حدث شيء لم يتوقعه أحد. الصابون الذي صنعاه انفجر وملأ المختبر ببصمات الزيت والبذور.

"ياسمين، ماذا نفعل الآن؟" قال طارق وهو يشعر بالاشمئزاز من البصمات.

"يبدو أننا بحاجة إلى تنظيف هذا المكان الآن!" أجابت ياسمين بحزم.

بعد ساعات من العمل الشاق، تمكن طارق وياسمين من تنظيف المختبر، لكنهما شعرا بحزن عميق. لقد كانوا يحلمان بأن يصبحا من العلماء أو الأطباء، لكنهما تعلما درسًا قاسيًا عن أهمية الحذر والاستعداد قبل تجربة أي شيء.

"ياسمين،" قال طارق وهو ينظر إلى صديقته، "أظن أننا علينا أن نكون أكثر حذرًا في المستقبل."

"نعم،" أومأت ياسمين. "لكننا سنكون دائمًا مستعجبين للتعلم والاكتشاف."

وهكذا، على الرغم من الحزن الذي شعرا به، تعلم طارق وياسمين درسًا قيمًا عن أهمية الحذر والاستعداد في عالم العلوم، ووعدا بأن يصبحا دائمًا أكثر حذرًا في تجاربهم المستقبلية.

BLEURT Score: 0.17198744416236877
Semantic Coherence: 0.8542397618293762
Grammar Errors: 8
Distinct 1-gram Score: 0.7125984251968503
Distinct 2-gram Score: 0.932806324110672
Perplexity: 2.3962526321411133

Cleaned Story85:
في قرية صغيرة بالعراق، كان يعيش طفلان صديقان لا يفترقان، اسمهما يوسف وسمر. كان يوسف وسمر يحبان الأشغال اليدوية كثيرًا، وكانا يقضيان وقتهما في صنع الأشياء الجميلة من القماش والخشب والفخار.

ذات يوم، قرر الصديقان القيام بمشروع جديد. كانوا يريدون صنع لوحة فنية كبيرة تُظهر جمال الطبيعة في قريتهم. بدأ يوسف وسمر بالجمع المادة اللازمة، وقضى كل يوم بعد المدرسة في العمل على المشروع.

كانت اللوحة تتألق تحت أشعة الشمس الذهبية، وكانت الألوان تتلألأ معًا في جمالها. لكن، كان هناك مشكلة صغيرة؛ كانت اللوحة لا تتميز بلمسة خاصة، شيء يجعلها تختلف عن اللوحات الأخرى.

في أحد الأيام، بينما كان يوسف وسمر يجلسان تحت شجرة النخيل، نظرا إلى اللوحة وبدأا في التفكير. فجأة، ابتسم يوسف واقترح: "لماذا لا نأخذ بعض الأشكال الطبيعية من الطبيعة ونضيفها إلى اللوحة؟" نظر سمر إلى يوسف بفضول وقال: "فكرة رائعة! لكن كيف سنفعل ذلك؟"

بدأ يوسف وسمر في جمع الأشكال الطبيعية، مثل الأوراق الملونة والجواهر الصغيرة التي وجداها في النهر. كانوا يضعونها ببطء على اللوحة، مما جعلها تبدو أكثر إبداعًا وجمالًا.

بعد عدة أيام من العمل الجاد، كانت اللوحة تتلألأ بجمالها. كانت الألوان تختلف بلمسة خاصة، وكانت الأشكال الطبيعية تلمع تحت أشعة الشمس.

عندما عرضا اللوحة في قريتهم، استقبلها الجميع بفرحة ودهشة. كانت اللوحة تُظهر جمال الطبيعة في قريتهم بطريقة لم يرها أحد من قبل.

في نهاية اليوم، جلس يوسف وسمر تحت شجرة النخيل، ينظران إلى اللوحة التي خلقوها معًا. ابتسم يوسف وقال: "لقد فعلنا ذلك، سمر! لقد خلقنا شيئًا جميلًا حقًا."

ابتسم سمر وأجاب: "نعم، يوسف. لقد علمتنا الطبيعة درسًا قيمًا عن الجمال والجمال الذي يمكن أن نجد في كل شيء."

وهكذا، عاش يوسف وسمر في سعادة، متشاركين الأحلام والأحلام، وكل يوم كان أجمل من اليوم الماضي.

BLEURT Score: 0.23343528807163239
Semantic Coherence: 0.8747744560241699
Grammar Errors: 17
Distinct 1-gram Score: 0.6838235294117647
Distinct 2-gram Score: 0.922509225092251
Perplexity: 1.7086138725280762

Cleaned Story86:
في جزر القمر الجميلة، كان يعيش طفل صغير يُدعى ياسين. ياسين كان يحب الأشغال اليدوية كثيرًا، وكان دائمًا يصنع أجمل الألوان والأشكال من الأوراق الملونة والخشب. كان يحب صنع الأشياء الجميلة ليس فقط لجعل نفسه سعيدًا، بل أيضًا لشاركها مع أصدقائه.

ذات يوم، قرر ياسين أن يصنع أجمل لوحة يمكن أن يشاركها مع صديقه الجديد، الذي لم يكن يعرفه بعد. كان ياسين متحمسًا جدًا، وقرر أن يصنع لوحة كبيرة جدًا، مليئة بالألوان والأشكال التي يحبها.

بعد أن انتهى ياسين من لوحته، ذهب إلى الشاطئ حيث كان ينتظر صديقه الجديد. كان الشمس تغرب وسماء الليل تتلألأ بالنجوم. انتظارًا لم يأتي صديقه، جلس ياسين وحيدًا على الرمال، ينظر إلى لوحته الجميلة.

فجأة، بدأت الأمواج تتلاطم بقوة، وبدأ المطر ينهمر بغزارة. كان ياسين يبكي، ليس فقط بسبب المطر، ولكن أيضًا لأن صديقه لم يأتِ. شعر ياسين بحزن شديد، لأنه أحب أن يشارك لوحته الجميلة مع صديقه الجديد.

في تلك الليلة، تعلم ياسين درسًا قاسيًا: "اتق شر من أحسنت إليه". لم يكن صديقه الجديد سوى خدعة، فلم يكن هناك صديق ينتظر ياسين. شعر ياسين بالوحدة والحزن، لكنه تعلم أن الأمل والصداقة الحقيقية لا تأتي من اللحظات القصيرة أو من الأشخاص الذين لا يفعلون ما يوعدهم.

ومع ذلك، استمر ياسين في صنع الأشغال اليدوية الجميلة، ليس فقط لأنه يحبها، ولكن أيضًا لأنه يريد أن يشاركها مع أصدقائه الحقيقيين، وأيضًا لأنه يدرك أن كل شيء جميل في الحياة قد يختلف، لكن الحب والصداقة الحقيقية لا يمكن أن تُفقد.

BLEURT Score: 0.2537098526954651
Semantic Coherence: 0.8641534447669983
Grammar Errors: 3
Distinct 1-gram Score: 0.6681034482758621
Distinct 2-gram Score: 0.9264069264069265
Perplexity: 2.1825995445251465

Cleaned Story87:
في قرية صغيرة في فلسطين، حيث الشتاء يغطي الأرض بثوبه الأبيض، كان يعيش ثلاثة أصدقاء: يوسف، ليلى، وسامر. كانوا يلعبون معاً كل يوم تحت أشعة الشمس الدافئة، يضحكون، ويشاركون الألعاب.

ذات يوم، بينما كان الثلاثة يلعبون بالقرب من النهر، وجدوا حجراً كبيراً. كان يوسف يحب اللعب به، لكن ليلى كانت تحب اللعب معه. سامر، الذي كان دائماً ما يعتني بصداقتهم، اقترح أن يشاركوا الحجر، لكن يوسف كان غاضباً جداً لأنه لم يكن يريد أن يشارك.

"أنا فقط أريد اللعب به!" صرخ يوسف، وهو يرفع الحجر من يده.

ليلى، التي كانت تحب يوسف كثيراً، شعرت بالحزن. "لكننا أصدقاء، لا يجب أن نكون غاضبين من بعضنا البعض."

سامر، الذي كان يحب ليلى كثيراً، شعر بالحزن أيضاً. "يوسف، أنت تعلم أن الغضب يترك ندوباً على الناس."

لكن يوسف لم يستمع. أراد أن يكون وحيداً. فجأة، بدأ يوسف يغادر، يحمل الحجر معه. ليلى وسامر تبقيان وراءه، يشاهدان كيف يفقد يوسف صديقيه.

"يوسف، أرجوك! لا تذهب!" صرخت ليلى، لكن يوسف لم يستجب.

مرت الأيام، وأصبح يوسف وحيداً. لم يكن هناك أحد يلعب معه. كان يوسف يدرك الآن أن الغضب قد فاتح له طريقاً من الحزن والوحدة.

في يوم شتاء بارد، وجد يوسف نفسه وحيداً في الغابة، يتأمل في ما حدث. "لماذا لم أستمع إلى سامر وليلى؟" قال لنفسه.

في تلك اللحظة، ظهر سامر وليلى أمام يوسف، يرتديان ملابسهم الدافئة، ولكن وجوههم كانت تظهر الحزن.

"يوسف، نحن نريد أن نكون أصدقاء مرة أخرى. الغضب قد ترك ندوباً على قلوبنا، لكننا لا نريد أن يظل ذلك كذلك." قالت ليلى.

يوسف نظر إلى أصدقائه، ودموعه تسربت من عينيه. "أنا آسف، أنا لم أكن أعرف أن الغضب يمكن أن يفعل هذا."

سامر، الذي كان دائماً ما يعتني بصداقتهم، قال: "يوسف، يمكن للغضب أن يترك ندوباً على الناس، لكن الصداقة يمكن أن تجعل تلك الندوب تختفي."

يوسف، الذي كان حزيناً جداً، نظر إلى أصدقائه وأومأ برأسه. "سأكون أكثر حذرًا في المستقبل، وأعلم أن الصداقة هي أثمن شيء لدينا."

ومن ذلك اليوم فصاعداً، عاد يوسف ليشارك الحب والضحك مع ليلى وسامر، مدركاً أن الغضب قد يترك ندوباً، لكن الصداقة يمكن أن تجعل تلك الندوب تختفي.

BLEURT Score: 0.19588929414749146
Semantic Coherence: 0.8699949979782104
Grammar Errors: 45
Distinct 1-gram Score: 0.6407185628742516
Distinct 2-gram Score: 0.9009009009009009
Perplexity: 1.9338310956954956
